In [1]:
import hashlib
import hmac
import csv
import numpy as np
import pandas as pd
import joblib
from joblib import Parallel, delayed
from multiprocessing import  Pool
import multiprocessing
import itertools
import random

In [2]:
from operator import itemgetter
import logging, sys

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
_DEBUG = False
KAGGLE = False

In [5]:
# logging.basicConfig( #stream=sys.stdout, 
#                     level=logging.DEBUG)

In [6]:
# logging.debug('A debug message!')

In [7]:
pd.options.display.max_rows=500
pd.options.display.max_columns=500
pd.set_option('display.max_colwidth', None)

In [8]:
l1=[0,1,2,3]
l2=['a','b','c','d']
l3=['hello','how','are','you']

ll=[l1,l2,l3]
# ll[0]=l1
# ll[1]=l2
# ll[2]=l3


list(zip(*ll))

[(0, 'a', 'hello'), (1, 'b', 'how'), (2, 'c', 'are'), (3, 'd', 'you')]

In [9]:
import time
from multiprocessing import Value
from ctypes import py_object

In [10]:
def init_pool_processes(shared_value):
    global p
    p = shared_value
def parallelize_dataframe(df, func, data_to_share=None,other_args=None, n_cores=4):
    start = time.time()
    df_split = np.array_split(df, n_cores)
    
    if data_to_share is not None:
        p = Value(py_object)
        p.value = data_to_share        
        pool = Pool(processes=n_cores,
                    initializer=init_pool_processes, initargs=(p,))
    else:
        pool = Pool(n_cores)
        
    arg_list=[]
    arg_list.append(df_split)
            
    if other_args is not None:
        for arg in other_args:
            arg_list.append(itertools.repeat(arg))
    
    parallel_args = zip(*arg_list)
    end = time.time()
    print(f'Parallel preparation time: {end-start}')
    df = pd.concat(pool.starmap(func, parallel_args))
    pool.close()
    pool.join()
    return df

In [11]:
def debugprint(msg):
    if _DEBUG:
        print(msg)

In [12]:
def get_bin(bins,val):
    matching_bin=-1
    for i in range(len(bins)-1):
        if bins[i] <= val < bins[i+1]:
            matching_bin= bins[i]
    if matching_bin==-1:
        matching_bin=bins[-1]
    return matching_bin

In [13]:
def calculate_roll(server_seed,client_seed,nonce):
    # Compute the HMAC-SHA512 hash of string1 using string2 as the key
    
    string1 = f"{nonce}:{server_seed}:{nonce}"
    hmac_key =   f"{nonce}:{client_seed}:{nonce}".encode()
    hmac_hash = hmac.new(key=hmac_key,
                         msg=string1.encode(),
                         digestmod=hashlib.sha512).hexdigest()

    # Convert the first 8 characters of the HMAC-SHA512 hash to an integer
    string3 = hmac_hash[:8]
    number = int(string3, 16)

    # Compute the roll value
    roll = round(number / 429496.7295)
    
    return roll

In [14]:
def compute_roll_hash_arrays(server_seed_array,client_seed_array,nonce_array):
    # Vectorize the function
    vectorized_calculate_roll = np.vectorize(calculate_roll)

    # Compute the roll values for the input arrays
    roll_array = vectorized_calculate_roll(server_seed_array,
                                           client_seed_array,
                                           nonce_array)
    
    # Compute roll for hash using the hash which is the next element in array
    # Vectorize the function
    vectorized_calculate_roll_hash = np.vectorize(calculate_roll)

    # Compute the roll values for the input arrays
    roll_array_hash = vectorized_calculate_roll_hash(server_seed_array[1:],
                                           client_seed_array[:chain_length-1],
                                           nonce_array[:chain_length-1])
    return roll_array,roll_array_hash

    

In [15]:
# trans_36 = bytes((x ^ 0x36) for x in range(256))
# trans_36

In [16]:
chain_length = 5110101 #1000001

filename = "sha256_hashchain_b9556671f785fe935bee087665b4047e421ea4491a5e2021a8152cab0b74c953.npy"

if KAGGLE:
    filepath_client ="/kaggle/input/hash-file-generation-client-seed/"
else:
    filepath_client="data/"
    
# filename_client=f'{filepath_client}sha256_hashchain_client_5M.npy'
filename_client=f'{filepath_client}sha256_hashchain_client1.npy'


# Define the number of CPU cores to use
num_cores = 8

In [17]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [18]:
# server_seed_array=np.load(filename)
# print(len(server_seed_array))
# server_seed_array[0:10]

In [19]:
client_seed_array=np.load(filename_client,
                         allow_pickle=True,fix_imports=True,encoding='latin1')
client_seed_array= client_seed_array[:chain_length]
print(len(client_seed_array))
client_seed_array[0:10]

1010102


array(['3455dda4b3aecaa36d4687277766a079feebbb4ab01dc038bc8fb8a36ddad6aa',
       '57eea50e4484ba102d776506baf4b1dc0dcb3fc74fe2ab56b10b0f427cb6409e',
       '3e49b34f2d6c7116d92ac8818c94f87d5616d7f2aea5024c2f38413605dc321c',
       'a0731d74c9501894d16ba144bc10dd49e8439c7f88304bfdbe4a755870c46da3',
       '665122519e5835c8e7ebfa0ab5b1949d75ac2c9e3dc1a6965a4b6e26bfc4ed76',
       '87fa02b24749184f0ce9183f21715ce556db72ecd1ad4f32b02227b40f3b24ab',
       'edee47aed4d9b598fa647145a17e44f9d155dc53ad231c913ff5c93fc04e56ee',
       'baf9d66be4136fe8acd30c042704b1798380fe0f3ee45d5800ae01db88e03ed3',
       '8eac9ebdcbfcf1720f6b0317fa66569c3129d278c0e2605069e7c5a0a2179c8d',
       'aeab33d31e0fb10105ce7ad75e3a40736db70a824c3ed6489957d6736243184b'],
      dtype='<U64')

In [20]:
# str_cat ='pppppp'
# # str_cat ='ppppppppppppppp'
# print(len(str_cat))
# client_seed_array_1=(str_cat + pd.Series(client_seed_array)).values
# client_seed_array_1[:2]

In [21]:
# str_cat ='ppppppppppppppp'
str_cat ='woxpwoxpwoxpwoxpwoxp'
print(len(str_cat))
client_seed_array_2=(str_cat + pd.Series(client_seed_array)).values
client_seed_array_2[:2]

20


array(['woxpwoxpwoxpwoxpwoxp3455dda4b3aecaa36d4687277766a079feebbb4ab01dc038bc8fb8a36ddad6aa',
       'woxpwoxpwoxpwoxpwoxp57eea50e4484ba102d776506baf4b1dc0dcb3fc74fe2ab56b10b0f427cb6409e'],
      dtype=object)

In [22]:
np.random.seed(5000)
low = 1500
# nonce_array = np.random.randint(low, high=low+chain_length, 
#                                 size=chain_length)

nonce_array = np.arange(low,low+chain_length,1)
np.random.shuffle(nonce_array)
print(len(nonce_array))
print(nonce_array[:10])
pd.Series(nonce_array).nunique()

5110101
[ 638661 1297273 1057802 1410000 2886373 4139846 2635511 4657935 2336816
  640280]


5110101

In [23]:
# roll_array,roll_array_hash=compute_roll_hash_arrays(server_seed_array,
#                                        client_seed_array,
#                                        nonce_array)
# print(len(roll_array[0:10]))
# print(roll_array[0:100])
# print(len(roll_array_hash[0:10]))
# print(roll_array_hash[0:100])

In [24]:
# roll_array_1,roll_array_hash_1=compute_roll_hash_arrays(server_seed_array,
#                                        client_seed_array_1,
#                                        nonce_array)
# print(len(roll_array_1[0:10]))
# print(roll_array_1[0:100])
# print(len(roll_array_hash_1[0:10]))
# print(roll_array_hash_1[0:100])

In [25]:
# roll_array_2,roll_array_hash_2=compute_roll_hash_arrays(server_seed_array,
#                                        client_seed_array_2,
#                                        nonce_array)
# print(len(roll_array_2[0:10]))
# print(roll_array_2[0:100])
# print(len(roll_array_hash_2[0:10]))
# print(roll_array_hash_2[0:100])

In [26]:
# # Vectorize the function
# vectorized_calculate_roll = np.vectorize(calculate_roll)

# # Compute the roll values for the input arrays
# roll_array = vectorized_calculate_roll(server_seed_array,
#                                        client_seed_array,
#                                        nonce_array)

# # Print the output
# print(len(roll_array[0:10]))
# print(roll_array[0:100])

In [27]:
# # Vectorize the function
# vectorized_calculate_roll_hash = np.vectorize(calculate_roll)

# # Compute the roll values for the input arrays
# roll_array_hash = vectorized_calculate_roll_hash(server_seed_array[1:],
#                                        client_seed_array[:chain_length-1],
#                                        nonce_array[:chain_length-1])

# # Print the output
# print(len(roll_array_hash[0:10]))
# print(roll_array_hash[0:100])

#### Top k hash verification of client and server

In [28]:
# # server_hash_orig = "d100f357cf1b05cc23ddc35c5f34a960911f9e67582a6d5ff27d9a210137e9c0"
# # nonce= 2079
# # client_seed_orig = "ppppppd3f19c18078cea0c25643b203c9a56dc87cc5e71fa46c2cf5520ee8477f5b43c"
# # chain_length=100

# # server_hash_orig = "ffed0cf8eb7020128c153bea86b3f29bc328af0451052576211138023bb300bd"
# # nonce= 2078
# # client_seed_orig = "ppppppc34b5f7748ee0255fc2c64af17e5cc78cd65a9c302b34652c0aec87081c91f3d"
# # chain_length=100

# server_hash_orig = "68f1ed13e2f17e3d38fd888e470eaae28720675dc7c5913a6b7fd9586df159b4"
# nonce= 2116
# # client_seed_orig = "ppppppc34b5f7748ee0255fc2c64af17e5cc78cd65a9c302b34652c0aec87081c91f3d"
# client_seed_orig = "woxpwoxpwoxpwoxpwoxp"+"c023a89e8027c289bc936fb9a6be20aa5d1dcba191e70d98004e860fb6c0d5db"
# chain_length=1000



In [29]:
# Define a function to generate a single hash in the hash chain
def generate_hash(message):
    # Hash the message using SHA-256
    hash_obj = hashlib.sha256(message)

    # Get the hexadecimal representation of the hash
    hash_hex = hash_obj.hexdigest()

    # Return the hash
    return hash_hex

In [30]:
def generate_hash_chain(seed,chain_length):
    hash_list=[seed]
    message = seed.encode()
    for i in range(chain_length):
        hash_hex= generate_hash(message)
        # Write the hash to the CSV file
        hash_list.append(hash_hex)
        # Update the message with the current hash
        message = hash_hex.encode()
    return hash_list

def compute_multirolls(hash_list,client_seed,nonce):
    rolls=[]
    for cur_hash in hash_list:
        roll = calculate_roll(cur_hash,client_seed,nonce)
        rolls.append(roll)
    return rolls
def compute_multirolls_nonce(server_hash,client_seed,hash_list_nonce):
    rolls=[]
    for cur_hash_nonce in hash_list_nonce:
        roll = calculate_roll(server_hash,client_seed,cur_hash_nonce)
        rolls.append(roll)
    return rolls

Function description 

Input
--------
Server Hash
Nonce
Client Seed

Output
----------
Category that has more number of successes - GT or LT <br>
How many number of success ?<br>

Return state of each condition: (Pass, Fail, Tie) <br>
100 values
-----------

1. Total number of low values on the given category is higher than that of other category
2. This category has the least value
3. This category has the most number of low values 
4. This category has the least value which are less than or equal to / greater than or equal to matching hash value bin
5. This category has the most number of low values which are less than or equal to / greater than or equal to matching hash value bin

1000 values
----------------
1. Total number of low values on the given category is higher than that of other category
2. This category has the least value
3. This category has the most number of low values 
4. This category has the least value which are less than or equal to / greater than or equal to matching hash value bin
5. This category has the most number of low values which are less than or equal to / greater than or equal to matching hash value bin


In [31]:
ABOVE="Above"
BELOW="Below"
INTER ="Intermediate"
def set_tristate(val1,val2):
    if val1 == val2:
        below_state=-1
        above_state =-1
    else:
        below_state = val1 > val2
        above_state = int(~below_state)
        below_state = int(below_state)
    return below_state,above_state

# def set_diffstate(val1,val2):
#     below_state = val1-val2
#     above_state = -1 * below_state
#     return below_state,above_state
def compute_least_condition(bin_counts):
    mask = bin_counts.index < 5
    val_below = bin_counts[mask].min()
    val_above = bin_counts[~mask].min()
    #NOTE: here val_above and val_below are swapped in arguments
    #since least value in below state should be lesser than above state
    below_state,above_state=set_tristate(val_above,val_below)
    return below_state,above_state
def compute_mostlow_condition(bin_counts,low_values_cutoff):
    mask = bin_counts < low_values_cutoff
    mask2 = bin_counts.index < 5
    val_below = bin_counts[mask & mask2].count()
    val_above = bin_counts[mask & ~mask2].count()
    
#     print(f'val_below:{val_below} val_above:{val_above}')
    below_state,above_state=set_tristate(val_below,val_above)
    return below_state,above_state

def compute_conditions(rolls_arr,size,initial_value=None):
    
    rolls_arr = rolls_arr[:size]
    low_values_cutoff = size * 0.1
    
    below_conditions = np.full(5,-100)
    above_conditions = np.full(5,-100)
    #Condition 1: 
    #Total number of low values on the given category is higher than that of other category 
    bins = [0,4750,5250,10000]
    labels = [0,-1,1]
    bin_counts_cond1 = pd.cut(rolls_arr, bins,labels=labels).value_counts()
    debugprint('Sum bincounts')
    debugprint(bin_counts_cond1)
#     print(bin_counts.loc[0],bin_counts.loc[1])
    
    below_conditions[0],above_conditions[0]=set_tristate(bin_counts_cond1.loc[1],bin_counts_cond1.loc[0])
    
    bins = list(range(0,10001,1000))
    labels = list(range(0,10))
    bin_counts = pd.cut(rolls_arr, bins,labels=labels).value_counts()
    debugprint('Full bincounts')
    debugprint(bin_counts)
    #Condition 2: 
    #This category has the least value
    below_conditions[1],above_conditions[1]=compute_least_condition(bin_counts)
    
    #Condition 3: 
    #This category has the most number of low values
    below_conditions[2],above_conditions[2]=compute_mostlow_condition(bin_counts,low_values_cutoff)
    
#     print(f'INITIAL roll:  {rolls_arr[0]}')g
    if initial_value is None:
        initial_value=rolls_arr[0]
    initial_bin = get_bin(labels,int(initial_value/1000))
    
    if initial_bin<5:
        mask_filt = bin_counts.index > initial_bin
    else:
        mask_filt = bin_counts.index < initial_bin
    debugprint(f'filtered bincounts with initial bin:{initial_bin}')
    debugprint(bin_counts[mask_filt])
    #Condition 4: 
    #This category has the least value which are less than or equal to / greater than or equal to matching hash value bin
    below_conditions[3],above_conditions[3]=compute_least_condition(bin_counts[mask_filt])
    
    #Condition 5: 
    #This category has the most number of low values
    below_conditions[4],above_conditions[4]=compute_mostlow_condition(bin_counts[mask_filt],
                                                                      low_values_cutoff)
   
    return below_conditions,above_conditions,bin_counts_cond1,bin_counts
    
def predict_seed_output(server_hash,nonce,client_seed,
                        hash_list_server=None,hash_list_nonce=None,
                        initial_value=None):
    chain_length=2000
    if (hash_list_server is None) and (hash_list_nonce is None):
        hash_list_server = generate_hash_chain(server_hash,chain_length)
#     print('Hash chain completed')
    if hash_list_nonce is not None:
        rolls_list  = compute_multirolls_nonce(server_hash,client_seed,hash_list_nonce)
    else:
        rolls_list  = compute_multirolls(hash_list_server,client_seed,nonce)
#     print('Roll computation completed')
    #100 values conditions
    rolls_arr=np.array(rolls_list)
    cond_100_below,cond_100_above,\
    bin_counts_cond1_100,bin_counts_100 = compute_conditions(rolls_arr,100,initial_value)
    cond_1000_below,cond_1000_above,\
    bin_counts_cond1_1000,bin_counts_1000 = compute_conditions(rolls_arr,1000,initial_value)
    cond_2000_below,cond_2000_above,\
    bin_counts_cond1_2000,bin_counts_2000 = compute_conditions(rolls_arr,2000,initial_value)
#     cond_4000_below,cond_4000_above = compute_conditions(rolls_arr,4000)
    
    cond_below_all = np.concatenate((cond_100_below,cond_1000_below,
                                     cond_2000_below,
#                                      cond_4000_below
                                    ))
    cond_above_all = np.concatenate((cond_100_above,cond_1000_above,
                                     cond_2000_above,
#                                      cond_4000_above
                                    ))
    
    debugprint(cond_below_all)
    debugprint(cond_above_all)
    
    below_passes = (cond_below_all == 1).sum()
    above_passes = (cond_above_all == 1).sum()
    
    if below_passes > above_passes:
        cond_all =  cond_below_all
        passed_state = BELOW
        total_passes = below_passes
    else:
        cond_all = cond_above_all
        passed_state = ABOVE
        total_passes = above_passes
  
    diff = abs(below_passes-above_passes)
    bin_counts_list = [bin_counts_cond1_100,bin_counts_100,
                      bin_counts_cond1_1000,bin_counts_1000,
                      bin_counts_cond1_2000,bin_counts_2000]
    
    if (bin_counts_cond1_100.loc[-1] < 5) & \
    (bin_counts_cond1_1000.loc[-1] < 50) & (bin_counts_cond1_2000.loc[-1] < 100):
        passed_state = INTER
    

    return cond_all,diff,total_passes,passed_state,bin_counts_list



In [32]:
def compare_prev(server_seed_array,
                client_seed,
                nonce_array,value_to_match=None):
#     print('compare_prev STARTED')
    vectorized_calculate_roll = np.vectorize(calculate_roll)

    # Compute the roll values for the input arrays
    roll_array = vectorized_calculate_roll(server_seed_array,
                                       client_seed,
                                       nonce_array)
    if value_to_match is None:
        value_to_match = roll_array[0]
    indices =np.where(roll_array==value_to_match)[0]
    indices_prev = indices -1 
#     print(len(indices_prev))
#     print(np.sort(roll_array[indices_prev]))
    prev_roll_match = roll_array[indices_prev]
    bins = [0,4750,5250,10000]
    labels = [0,-1,1]
    bin_counts = pd.cut(roll_array, bins,labels=labels).value_counts()
#     print('Sum bincounts')
#     print(bin_counts)
    
    return bin_counts

def validate_test_topk(test,client_seed_data,pass_exp1,pass_exp2,
                       pass_state_reqd=None,last_index=None,result_df=None,
                       client_char_count=None):
    i = len(client_seed_data)-1 if last_index is None else last_index - 1
    last_success_index =i
    # client_seed_data = client_seed_data[::-1]
    client_size = len(client_seed_data)
    client_scan_fresh=True 
    bincountcols=[]
    bincountprevcols=[]
    sumbincols = ['lt4750','inter','gt5250']
    
    for col in sumbincols:
        bincountprevcols.append('prev_'+str(col))
    bincountprevcols_nonce =['nonce_'+col for col in bincountprevcols]
    for hash_size in [100,1000,2000]:
        for col in sumbincols:
            bincountcols.append(str(hash_size)+'_'+str(col))
        for col in range(10):
            bincountcols.append(str(hash_size)+'_'+str(col))
#     print(bincountcols)

    bincountcols_nonce = ['nonce_'+col for col in bincountcols]
    

    if result_df is None:
        result_df = pd.DataFrame(columns=['match','seed','hash','nonce',
                                    'client_seed','client_index',
                                    'total_passes','passed_state',
                                    'total_passes_1','total_passes_2',
                                    'roll_actual','roll_hash','roll_blank_server'
                                         ] + bincountprevcols+bincountprevcols_nonce +\
                                             bincountcols+bincountcols_nonce)
    result_success=True
    for server_count,(server_index,row) in enumerate(test.iterrows()):
#         if server_count % 10==0:
#             i = len(client_seed_data)-1
#             print(f'server_count:{server_count}')
        
#         if not result_success:
#             i= last_success_index
#             i = len(client_seed_data)-1
        cur_seed = row['seed']
        cur_nonce = row['nonce']
        cur_hash= row['hash']
        
        chain_length=2000
        cur_hash_list = generate_hash_chain(cur_hash,chain_length)
        cur_hash_list_nonce = generate_hash_chain(str(cur_nonce),chain_length)
        
#         cur_hash_list_rev = generate_hash_chain(cur_hash[::-1],chain_length)
        
#         client_seed_data=np.array(cur_hash_list[1:])
#         client_seed_data=(str(cur_nonce) + pd.Series(client_seed_data)).values
        
##         cur_hash_list=list((cur_hash + pd.Series(np.array(cur_hash_list))).values)
      
        fullclientscan=False
        match = False
        prev_match=""
        while not match and not fullclientscan:
            
            if i % 1000==0:
                print(f'current scan : {i}')
            
            if client_char_count is None:
                client_seed = client_seed_data[i]
            else:
                client_seed = client_seed_data[i][:client_char_count]

#             client_seed= generate_hash((client_seed_data[i] + prev_match).encode())
#             client_seed= generate_hash((str(cur_nonce)+client_seed_data[i]).encode())
            roll_seed_actual=calculate_roll(cur_seed,client_seed,cur_nonce)
            roll_hash=calculate_roll(cur_hash,client_seed,cur_nonce)
            
            if (3000<=roll_hash<=7000) or ((i+1) in last_indices):
                total_passes=-1
                passed_state=ABOVE
                passed_state_2=BELOW
 
            else:
                cond_all,diff,total_passes_1,passed_state,bin_counts_list = predict_seed_output(cur_hash,
                                                                              cur_nonce,
                                                                              client_seed,
                                                                             hash_list_server=cur_hash_list)
                #This condition is to reduce time execution
                total_passes_2=0
                passed_state_2=passed_state
                if (pass_exp2>=total_passes_1):
                    cond_all_2,diff_2,total_passes_2,passed_state_2,bin_counts_list_nonce = predict_seed_output(cur_hash,
                                                                                  cur_nonce,
                                                                                  client_seed,
                                                                                 hash_list_nonce=cur_hash_list_nonce)
#                 cond_all_3,diff_3,total_passes_3,passed_state_3,bin_counts_list_nonce_2 = predict_seed_output(cur_hash,
#                                                                               cur_nonce,
#                                                                               client_seed,
#                                                                              hash_list_nonce=cur_hash_list_nonce)
# #                 cond_all_3,diff_3,total_passes_3,passed_state_3,bin_counts_list_rev = predict_seed_output(cur_hash,
#                                                                               cur_nonce,
#                                                                               client_seed,
#                                                                              cur_hash_list_rev)
                total_passes = total_passes_1 + total_passes_2  

            i-=1
#             if (total_passes_1>=pass_exp1) & (total_passes_2>=pass_exp2) & (passed_state==passed_state_2):
#             if (pass_exp2>=total_passes>=pass_exp1) & (passed_state==passed_state_2) \
#                 & (last_index!=i+1):
            if (pass_exp2>=total_passes>=pass_exp1) & (passed_state==passed_state_2) \
                    & (passed_state!=INTER) & ((pass_state_reqd==None) | (pass_state_reqd==passed_state) ):
#                 print(f'Matching client at {i+1} for seed:{cur_seed} nonce:{cur_nonce}')
                match=True
#                 rand = random.randint(1,2)
#                 print(f'random number:{rand}')
#                 if rand==1:
#                     match=True
            elif (i<0):
                i = len(client_seed_data)-1
                if client_scan_fresh:
                    fullclientscan=True
                    print(f'No matching client for seed:{cur_seed} nonce:{cur_nonce}')
        
        if match:
#             print('Hash Chain starts')
            server_hash_list = generate_hash_chain(cur_hash,10)
            bincount_prev = compare_prev(server_hash_list,client_seed,cur_nonce,value_to_match=roll_hash)
#             print('Nonce Hash Chain starts')
            nonce_hash_list = generate_hash_chain(str(cur_nonce),10)
            bincount_prev_nonce = compare_prev(cur_hash,client_seed,nonce_hash_list,value_to_match=roll_hash)
            roll_blank_server=calculate_roll('',client_seed,cur_nonce)
            
            bincount_row =[]
            for bincount_df in bin_counts_list:
                bincount_row.extend(list(bincount_df.values))
#             print(bincount_row)
            bincount_row_nonce =[]
            for bincount_df in bin_counts_list_nonce:
                bincount_row_nonce.extend(list(bincount_df.values))
#             print(bincount_row_nonce)
            result_success = ((passed_state==ABOVE) & (roll_seed_actual>5250))  \
                            | ((passed_state==BELOW) & (roll_seed_actual<4750))
            if result_success:
                last_success_index= i+1
            result = [result_success,cur_seed, cur_hash, cur_nonce,
                                           client_seed,i+1,total_passes,passed_state,
                                           total_passes_1,total_passes_2,
                                           roll_seed_actual,roll_hash,roll_blank_server] + \
                                            list(bincount_prev.values) + list(bincount_prev_nonce.values) +\
                                            bincount_row + bincount_row_nonce
            result_df.loc[len(result_df.index)] = result
            print(f'server_count: {server_count} current result: {result_success, cur_nonce,total_passes,passed_state,roll_seed_actual,roll_hash}')
          
    return result_df
            
    

In [33]:
def predict_digit_pattern(server_hash,roll_hash,nonce,client_seed,
                        match_digit_arr,match_digit_indices,
                        mismatch_digit_arr=None,mismatch_digit_indices=None):

#     rolls_list  = compute_multirolls(hash_list_server,client_seed,nonce)
    match = True
    
#     roll_first = rolls_list[0]
    factors = np.array([10000,1000,100,10])
    factors_filt = factors[match_digit_indices]
    for i,val in enumerate(factors_filt):
        match = match & (int((roll_hash % val) / (val / 10))==match_digit_arr[i])
        if not match: 
            break

    if mismatch_digit_arr is not None:
        factors_filt = factors[mismatch_digit_indices]
        for i,val in enumerate(factors_filt):
            match = match & (int((roll_hash % val) / (val / 10))!=mismatch_digit_arr[i])
            if not match: 
                break            

    return match
 

def predict_digit_output(server_hash,nonce,client_seed,
                         match_digit_arr,match_digit_count_arr,
                        hash_list_server,match_digit_indices=None):

    rolls_list  = compute_multirolls(hash_list_server,client_seed,nonce)
    match = True
    
    #eliminate zero from all roll_hash
#     for roll in rolls_list:
#         str_roll = f'{roll:04d}'
#         cur_count = str_roll.count('0')
#         if cur_count!=0:
#             match=False
            
    for roll,match_digit,match_count,match_indices \
            in zip(rolls_list,match_digit_arr,match_digit_count_arr,match_digit_indices):
        str_roll = f'{roll:04d}'
        
        if match_digit_indices is None:
            str_roll_indexed = str_roll
        else:
            str_roll_indexed = ''
            for index in match_indices:
                str_roll_indexed = str_roll_indexed + str_roll[int(index)]
                
        cur_count = str_roll_indexed.count(match_digit)
        if cur_count!=match_count:
            match=False
            
    return match, rolls_list

def validate_digits(test,client_seed_data,pass_state_reqd,
                    match_digit_arr,match_digit_count_arr,
                    feature_chain_length=20,
                    match_digit_indices=None,
                    last_index=None,result_df=None):
    i = len(client_seed_data)-1 if last_index is None else last_index - 1
    last_success_index =i
    client_size = len(client_seed_data)
    client_scan_fresh=True 
    

    if result_df is None:
        result_df = pd.DataFrame(columns=['match','seed','hash','nonce',
                                    'client_seed','client_index',
                                    'passed_state',
                                    'roll_actual','roll_hash_list',
                                         ])
    result_success=True
    for server_count,(server_index,row) in enumerate(test.iterrows()):
        
        i = len(client_seed_data)-1 if last_index is None else last_index - 1
        
        
        cur_seed = row['seed']
        cur_nonce = row['nonce']
        cur_hash= row['hash']
        
        if is_data_hash:
            cur_hash_list = list(test[server_count+1:server_count+1+feature_chain_length]['seed'])
        else:
            cur_hash_list = generate_hash_chain(cur_hash,feature_chain_length)
        
      
        fullclientscan=False
        match = False
        prev_match=""
        while not match and not fullclientscan:
            
            if i % 10000==0:
                print(f'current scan : {i}')
            
            client_seed = client_seed_data[i]

            roll_seed_actual=calculate_roll(cur_seed,client_seed,cur_nonce)
            roll_hash=calculate_roll(cur_hash,client_seed,cur_nonce)
        
            match, rolls_list = predict_digit_output(cur_hash,cur_nonce,client_seed,
                         match_digit_arr,match_digit_count_arr,
                        cur_hash_list,match_digit_indices=match_digit_indices)
            i-=1
            
            if (i<0):
                i = len(client_seed_data)-1
                if client_scan_fresh:
                    fullclientscan=True
                    if not match:
                        print(f'No matching client for seed:{cur_seed} nonce:{cur_nonce}')
        
        if match:
            result_success = ((pass_state_reqd==ABOVE) & (roll_seed_actual>5250))  \
                            | ((pass_state_reqd==BELOW) & (roll_seed_actual<4750))
            if result_success:
                last_success_index= i+1
            result = [result_success,cur_seed, cur_hash, cur_nonce,
                                           client_seed,i+1,pass_state_reqd,
                                           roll_seed_actual,rolls_list]
            result_df.loc[len(result_df.index)] = result
            print(f'server_count: {server_count} current result: {result_success, cur_nonce,pass_state_reqd,roll_seed_actual,roll_hash}')
          
    return result_df
  

In [34]:

def validate_test_topk_2(test,client_seed_data,hash_count_to_check,pass_state_reqd=BELOW,
                       last_index=None,result_df=None):
    i = len(client_seed_data)-1 #if last_index is None else last_index - 1
    last_success_index =i
    # client_seed_data = client_seed_data[::-1]
    client_size = len(client_seed_data)
    client_scan_fresh=True 

    if result_df is None:
        result_df = pd.DataFrame(columns=['match','seed','hash','nonce',
                                    'client_seed','client_index','passed_state',
                                    'roll_actual','roll_hash','roll_blank_server'
                                         ] )
    result_success=True
    for server_count,(server_index,row) in enumerate(test.iterrows()):
        cur_seed = row['seed']
        cur_nonce = row['nonce']
        cur_hash= row['hash']
        
        cur_hash_list = generate_hash_chain(cur_hash,hash_count_to_check+1)
#         cur_hash_list_nonce = generate_hash_chain(str(cur_nonce),chain_length)
        
        fullclientscan=False
        match = False
        prev_match=""
        while not match and not fullclientscan:
            
            if i % 10000==0:
                print(f'current scan : {i}')
            
            client_seed = client_seed_data[i] 
            roll_seed_actual=calculate_roll(cur_seed,client_seed,cur_nonce)
            roll_hashes=[]
            for hash_chain_val in cur_hash_list:
                roll_hashes.append(calculate_roll(hash_chain_val,client_seed,cur_nonce))
            roll_hashes_arr = np.array(roll_hashes)
            
            roll_hashes_arr_2 = roll_hashes_arr[:len(roll_hashes_arr)-1]

            if pass_state_reqd==BELOW:
                last_rev = (roll_hashes_arr[len(roll_hashes_arr)-1]<4750)
                mask = roll_hashes_arr_2<5250
#                 roll_hashes_arr_1stdigit = (roll_hashes_arr_2  / 1000).astype('int')
#                 flag_check_all_1000 = True
#                 for val1000 in range(6,10):
#                     flag_check_all_1000 = flag_check_all_1000 & len(roll_hashes_arr_1stdigit[roll_hashes_arr_1stdigit==val1000])>0
                    
            else:
                last_rev = (roll_hashes_arr[len(roll_hashes_arr)-1]>5250)
                mask = roll_hashes_arr_2>4750
            
            match = (len(roll_hashes_arr_2[mask])==0) #& last_rev # & flag_check_all_1000
            roll_hash = roll_hashes_arr[0]
            i-=1
            if (i<0):
                i = len(client_seed_data)-1
                if client_scan_fresh:
                    fullclientscan=True
                    print(f'No matching client for seed:{cur_seed} nonce:{cur_nonce}')
        
        if match:
            roll_blank_server=calculate_roll('',client_seed,cur_nonce)
            
            result_success = ((pass_state_reqd==ABOVE) & (roll_seed_actual>5250))  \
                            | ((pass_state_reqd==BELOW) & (roll_seed_actual<4750))
            result = [result_success,cur_seed, cur_hash, cur_nonce,
                                           client_seed,i+1,BELOW,
                                           roll_seed_actual,roll_hash,roll_blank_server] 
            result_df.loc[len(result_df.index)] = result
            print(f'server_count: {server_count} current result: {result_success,cur_nonce,roll_seed_actual,roll_hashes_arr}')
          
    return result_df
            
    

In [35]:
# test_file_random ='/kaggle/input/hash-generate-random-seeds/test_data_random.csv'
# test_seeds_2=pd.read_csv(test_file_random)
# print(test_seeds_2.shape)
# test_seeds_2.head()

In [36]:
# test_file ='/kaggle/input/hashdata/server seed test data 4.csv'
# test_seeds_4=pd.read_csv(test_file)
# print(test_seeds_4.shape)
# test_seeds_4.head()

In [37]:
# test_file ='/kaggle/input/hashdata/server seed test data 5.csv'
# test_seeds_5=pd.read_csv(test_file)
# print(test_seeds_5.shape)
# test_seeds_5.head()

In [38]:
# #create special client seed pattern array

# def get_hexval(val):
#     hexdata='abcdef'
#     if val >=10:
#         return hexdata[val-10]
#     else:
#         return str(val)

# str_spl_client = 'woxpwoxpwoxpwoxpwoxp066103c1b2a6ebe01cf30afd49a6b931278793fc457dee84510f03e11779d5be'
# spl_client_list=[]
# for index in range(20,20+64):
#     for val in range(16):
#         spl_client_list.append(str_spl_client[:index]+get_hexval(val)+str_spl_client[index+1:])
# spl_client_array = np.array(spl_client_list)
# print(len(spl_client_array))
# spl_client_array[1020:]

##### Machine Learning

Generate Data

In [39]:
def generate_data(test,client_seed_data,pass_state_reqd,
                    match_digit_arr,match_digit_count_arr,
                    mismatch_digit_arr=None,mismatch_digit_indices=None,
                    feature_chain_length=20,
                    is_data_hash=False,
                    match_digit_indices=None,
                    match_count_exp=1,
                    test_limit = None,
                    last_index=None,result_df=None,
                    print_client_scan=False,
                    hash_list_nonce=None):
    i = len(client_seed_data)-1 if last_index is None else last_index - 1
    last_success_index =i
    client_size = len(client_seed_data)
    client_scan_fresh=True 
    

    if result_df is None:
        result_df = pd.DataFrame(columns=['match','seed','hash','nonce',
                                    'client_seed','client_index',
                                    'passed_state',
                                    'roll_actual','roll_hash_list','roll_hash_list_nonce',
                                         ])
    result_success=True
    for server_count,(server_index,row) in enumerate(test.iterrows()):
        
        if (server_count % 10)==0:
            print(f'server_count:{server_count}')
        
        if (test_limit is not None) and (server_count >= test_limit):
            break
        
        #for large volume of data, it is recommended to set i to entire lengthof client seeds
        #for every server seed
#         i = len(client_seed_data)-1 if last_index is None else last_index - 1
        
        cur_seed = row['seed']
        cur_nonce = row['nonce']
        cur_hash= row['hash']
        
       
        if is_data_hash:
            cur_hash_list = list(test[server_count+1:server_count+1+feature_chain_length+1]['seed'])
        else:
            cur_hash_list = generate_hash_chain(cur_hash,feature_chain_length)
        
          
        fullclientscan=False
        match = False
        prev_match=""
        match_count=0
        
        client_start = i
        while match_count < match_count_exp and not fullclientscan:
            
            if print_client_scan  & (i % 1000==0) :
                print(f'current scan : {i}')
            
            client_seed = client_seed_data[i]
            client_index = i

            roll_seed_actual=calculate_roll(cur_seed,client_seed,cur_nonce)
            roll_hash=calculate_roll(cur_hash,client_seed,cur_nonce)
            
            match = predict_digit_pattern(cur_hash,roll_hash,cur_nonce,client_seed,
                        match_digit_arr,match_digit_indices,
                        mismatch_digit_arr=mismatch_digit_arr,
                        mismatch_digit_indices=mismatch_digit_indices)
        
            # match, rolls_list = predict_digit_output(cur_hash,cur_nonce,client_seed,
            #              match_digit_arr,match_digit_count_arr,
            #             cur_hash_list,match_digit_indices=match_digit_indices)
            i-=1
            
            if i==client_start and not match:
                print()
                print(i,client_start)
                print(f'No further match for seed:{cur_seed} nonce:{cur_nonce} match count: {match_count}')
                print()
                fullclientscan=True
            if (i<0):
                i = len(client_seed_data)-1
       
            if match:
                
#                 print(f'Matched for seed:{cur_seed} i:{i} match count: {match_count}')
                rolls_list  = compute_multirolls(cur_hash_list,client_seed,cur_nonce)
                if hash_list_nonce is None:
                    hash_list_nonce = generate_hash_chain(str(cur_nonce),feature_chain_length)

                rolls_list_nonce  = compute_multirolls_nonce(cur_hash,client_seed,hash_list_nonce)

                match_count += 1
                
                result_success = ((pass_state_reqd==ABOVE) & (roll_seed_actual>5250))  \
                                | ((pass_state_reqd==BELOW) & (roll_seed_actual<4750))
                if result_success:
                    last_success_index= i+1
                result = [result_success,cur_seed, cur_hash, cur_nonce,
                                               client_seed,client_index,pass_state_reqd,
                                               roll_seed_actual,rolls_list,rolls_list_nonce]
                
                result_df.loc[len(result_df.index)] = result
#                 print(f'server_count: {server_count} current result: {result_success, cur_nonce,pass_state_reqd,roll_seed_actual,roll_hash}')
          
    return result_df


In [40]:
hashval = '416e56f7cc46f549f3f9946a5264475854a53d5c363487535a06fcc066d7120e'#'3e24f29a3ae70e54aedd86b0c68640050be7dace8ae679f1fb85d1325a56ddae'
seed = hashval #'c77b3e783f094e255b0693f89094bed95ad9738d86f44fa4c09c8b0d58e2c73e'
nonce = 2653 #2390

feature_chain_length=60
train_hash_size = 2158#121
train_client_size = 2000 #20

In [41]:
analysis_df=None

In [42]:
cur_hash_list = generate_hash_chain(hashval,train_hash_size)

train_data = pd.DataFrame(columns=['seed','hash','nonce'])
train_data['seed']=np.array(cur_hash_list[:-1])
train_data['hash']=np.array(cur_hash_list)[1:]
train_data['nonce'] = nonce

print(len(train_data))
train_data.head()

2158


,seed,hash,nonce
0,416e56f7cc46f549f3f9946a5264475854a53d5c363487535a06fcc066d7120e,37852b2aec85dc4ef278a4f9eb6e83cff933d03f334d39bf3e4f189b0d364774,2653
1,37852b2aec85dc4ef278a4f9eb6e83cff933d03f334d39bf3e4f189b0d364774,4e03d0263e7efef0f77e974064249054dbbd7ea8feb489997284e56eb2e59d98,2653
2,4e03d0263e7efef0f77e974064249054dbbd7ea8feb489997284e56eb2e59d98,c29cc874c133ce94ef1316ba0f6cb2cd54846dcca4a9a7793b5d821c8ce27d41,2653
3,c29cc874c133ce94ef1316ba0f6cb2cd54846dcca4a9a7793b5d821c8ce27d41,1e4cf26c69ee29e0752ac0dd36ab894a2cb6fa0ca98f49dbe601df9e5f6cda1b,2653
4,1e4cf26c69ee29e0752ac0dd36ab894a2cb6fa0ca98f49dbe601df9e5f6cda1b,ed0bdaa811adab3127d410057c3e6ed431630a9e60254cedb1fecbcad8cc6110,2653


In [43]:
PATTERN_99=False
HIGH_ANALYSIS=False

In [44]:
READ_FROM_FILE=False

In [45]:
%%time
# match_digit_arr = np.array(['999',])
# match_digit_count_arr=np.array([1])
# match_digit_indices = np.array(['012'])

if PATTERN_99:
    match_digit_arr=np.array([9,9])
    mismatch_digit_arr=np.array([9,9])
else:
    match_digit_arr=np.array([0,0])
    mismatch_digit_arr=np.array([0,0])
    
match_digit_indices = np.array([1,2])
match_digit_count_arr=np.array([1])
mismatch_digit_indices = np.array([0,3])

CPU times: user 10 µs, sys: 2 µs, total: 12 µs
Wall time: 14.8 µs


In [46]:
%%time

if not READ_FROM_FILE:

    cur_client_seed_array = client_seed_array_2[:1000001]

    results_df = None

    last_index =  len(cur_client_seed_array)-1 #1024
    start = 0

    hash_list_nonce = generate_hash_chain(str(nonce),feature_chain_length)

    results_df = generate_data(train_data,
                                cur_client_seed_array,
                                 ABOVE,
                        match_digit_arr,match_digit_count_arr,match_digit_indices=match_digit_indices,
                        mismatch_digit_arr=mismatch_digit_arr,mismatch_digit_indices=mismatch_digit_indices,
                        match_count_exp=train_client_size,
                        feature_chain_length=feature_chain_length,
                        is_data_hash = True,
                        test_limit=62,
                        last_index=last_index,result_df=results_df,
                        hash_list_nonce=hash_list_nonce)
    row = results_df.iloc[len(results_df)-1]
    last_index = row['client_index']

server_count:0
server_count:10
server_count:20
server_count:30
server_count:40
server_count:50
server_count:60
CPU times: user 22min 12s, sys: 972 ms, total: 22min 13s
Wall time: 22min 13s


In [47]:
# last_index
# results_df['client_index'].min()

In [48]:
# results_df.head()

In [49]:
# print(len(results_df))
# results_df['client_seed'].nunique()

In [50]:
%%time
if PATTERN_99:
    file_pattern_str = 'x99x'
else:
    file_pattern_str = 'x00x'
filename = f'data/results_df_{nonce}_Large_{file_pattern_str}_pattern.csv'
if READ_FROM_FILE:
    results_df = pd.read_csv(filename, converters={'roll_hash_list': pd.eval,
                                                 'roll_hash_list_nonce': pd.eval})
else:
    results_df.to_csv(filename,index=False)

CPU times: user 3.42 s, sys: 104 ms, total: 3.52 s
Wall time: 3.52 s


In [51]:
results_df.head()

,match,seed,hash,nonce,client_seed,client_index,passed_state,roll_actual,roll_hash_list,roll_hash_list_nonce
0,True,416e56f7cc46f549f3f9946a5264475854a53d5c363487535a06fcc066d7120e,37852b2aec85dc4ef278a4f9eb6e83cff933d03f334d39bf3e4f189b0d364774,2653,woxpwoxpwoxpwoxpwoxpadf10d5d3a6a0b9b4ce5423d09c20ccae2fd7f1af5d6dbd68bd48ff9626a9449,999966,Above,6731,"[1001, 6163, 9140, 3222, 5167, 7611, 9058, 5191, 4677, 3246, 5666, 3200, 4990, 9474, 8081, 1823, 3807, 23, 9435, 7679, 2523, 2000, 917, 4401, 4313, 8411, 4190, 1208, 1701, 2082, 8587, 1754, 4159, 5837, 6092, 9574, 8123, 1574, 8550, 9330, 1469, 9149, 3605, 2981, 1, 4401, 2918, 4533, 2532, 2021, 4827, 8463, 6310, 6910, 3187, 7318, 6973, 493, 8244, 4875, 8448]","[1001, 2252, 5994, 1612, 8359, 5693, 1078, 1717, 1477, 9114, 6706, 1143, 3007, 4521, 307, 8626, 1895, 3309, 4935, 9613, 4326, 5906, 1218, 3781, 8490, 3349, 2835, 2210, 2265, 9411, 268, 7471, 4282, 1390, 9875, 7848, 7440, 3405, 3208, 7240, 7402, 273, 9010, 4474, 1892, 9372, 5356, 1250, 3269, 3809, 5228, 7773, 5112, 8766, 4361, 5740, 7310, 7620, 4168, 3759, 5414]"
1,False,416e56f7cc46f549f3f9946a5264475854a53d5c363487535a06fcc066d7120e,37852b2aec85dc4ef278a4f9eb6e83cff933d03f334d39bf3e4f189b0d364774,2653,woxpwoxpwoxpwoxpwoxpd4ebb77a8b670e5888d3b61cd2eba7ae91299eaab7eee0d1d8e949e53977848a,999901,Above,2758,"[6008, 9968, 5706, 6859, 5268, 8393, 5775, 8439, 5264, 7902, 8783, 2350, 2441, 1534, 9838, 994, 9100, 341, 8139, 5364, 9423, 1766, 7200, 10, 6660, 6470, 5947, 5323, 8261, 5813, 7776, 1614, 6655, 1232, 6104, 329, 9484, 7288, 7505, 5949, 785, 1547, 248, 7522, 1476, 6175, 1575, 4841, 8377, 1939, 6520, 385, 7222, 7487, 388, 4744, 1275, 808, 6004, 1999, 7120]","[6008, 6780, 6372, 5496, 4304, 5501, 2613, 973, 566, 7239, 2260, 6774, 4871, 78, 5342, 4997, 3045, 9948, 5751, 3574, 4489, 9387, 3467, 5459, 4086, 7378, 7220, 1682, 2923, 3639, 6926, 5454, 8742, 8169, 6224, 9525, 7796, 635, 4971, 7999, 5408, 7082, 2633, 1567, 1275, 3780, 1742, 4924, 1815, 9048, 8101, 2001, 336, 7458, 9515, 2280, 152, 8615, 1723, 1574, 302]"
2,False,416e56f7cc46f549f3f9946a5264475854a53d5c363487535a06fcc066d7120e,37852b2aec85dc4ef278a4f9eb6e83cff933d03f334d39bf3e4f189b0d364774,2653,woxpwoxpwoxpwoxpwoxp270812fe08ed514d0f01ea5dc929e1534b852af245c6a3f17fd6a174ef9e0304,999779,Above,4875,"[2009, 7787, 7855, 7946, 1819, 3973, 6996, 2976, 9108, 1368, 8664, 3003, 9129, 6931, 565, 3371, 7186, 1952, 629, 8265, 6568, 3938, 3292, 6444, 3510, 3908, 6234, 2576, 6285, 7480, 7718, 611, 728, 8189, 5852, 7335, 6307, 76, 2578, 6770, 642, 4377, 2900, 5786, 2218, 1246, 7657, 1788, 9016, 1298, 1528, 3276, 559, 3170, 654, 3668, 1960, 8317, 3444, 7632, 4124]","[2009, 1713, 8547, 2566, 6850, 265, 3104, 5212, 3125, 1283, 3220, 984, 6234, 2069, 627, 3967, 7449, 5666, 1404, 8750, 4884, 3117, 8059, 2999, 4598, 5905, 305, 2055, 7811, 963, 2146, 159, 3747, 2799, 7306, 2742, 5694, 4871, 2931, 6273, 7222, 2119, 7696, 7105, 5232, 3151, 983, 9620, 1261, 9654, 3721, 2509, 290, 2980, 9569, 6529, 3047, 1171, 7511, 7244, 183]"
3,True,416e56f7cc46f549f3f9946a5264475854a53d5c363487535a06fcc066d7120e,37852b2aec85dc4ef278a4f9eb6e83cff933d03f334d39bf3e4f189b0d364774,2653,woxpwoxpwoxpwoxpwoxpaa00cf4f9b89f30d74296bd3ef6d53afc7ffb0a1b6e4b8406392bc88aec63f1d,999762,Above,9450,"[3006, 3008, 7418, 6677, 5100, 7534, 5483, 3288, 8459, 8180, 7290, 8876, 4418, 4016, 6779, 9314, 529, 4185, 3952, 5009, 1133, 139, 4759, 4005, 4089, 6767, 3666, 6661, 3706, 5171, 2817, 4746, 7511, 5349, 111, 6543, 6157, 7283, 7643, 791, 3494, 2593, 3090, 8635, 6624, 5892, 3146, 1679, 9924, 1299, 1855, 747, 1353, 2572, 8939, 1512, 39, 2443, 3658, 7941, 7027]","[3006, 7556, 548, 1866, 6146, 8182, 3619, 9396, 6428, 7032, 5056, 6573, 7421, 6276, 23, 8163, 7930, 7210, 2803, 6301, 3244, 4877, 6665, 8890, 1592, 1074, 69, 3254, 773, 431, 768, 4890, 4853, 2403, 9751, 2928, 7599, 5564, 6728, 5989, 2806, 8141, 5455, 1196, 7809, 7310, 411, 8771, 7912, 3972, 1854, 9953, 32, 5199, 1834, 9231, 2743, 477, 1737, 6478, 9963]"
4,True,416e56f7cc46f54

In [52]:
HIGH_TARGET= 9000 #8944
LOW_TARGET = 1000 #1056
ZERO_COL_CUTOFF = 4

In [53]:
def create_target_mask(data,bHighCheck=HIGH_ANALYSIS):
    if bHighCheck:
        mask = data['roll_actual'] > HIGH_TARGET
    else:
        mask = data['roll_actual'] < LOW_TARGET   
    return mask

In [54]:
def generate_features(initial_df,istrain):
    rolls_df = initial_df['roll_hash_list'].apply(pd.Series)
    rolls_df.columns = [f'roll_{i}' for i in range(rolls_df.shape[1])]
    
#     for col in list(rolls_df.columns)[1:]:
#         rolls_df[f'{col}_dig2'] = ((rolls_df[col] % 1000) / 100).astype('int')
    rolls_df['count_lt_1000'] = (rolls_df < 1000).sum(axis=1)
    rolls_df['count_gt_5000'] = (rolls_df > 5000).sum(axis=1)-1
    
    if istrain:
        features_df = pd.concat([initial_df[['roll_actual']],rolls_df],axis=1)
        mask = create_target_mask(features_df)
        features_df['target'] =0
        features_df.loc[mask,'target'] = 1
#         features_df=features_df.drop(['roll_actual'],axis=1)

        features_df['roll_hash_list_nonce']=initial_df['roll_hash_list_nonce']
        features_df['roll_actual']=features_df['roll_actual'].astype('int')
        print(features_df['target'].value_counts())
    else:
        features_df = rolls_df
    return features_df


In [55]:
train_manual = generate_features(results_df[25*train_client_size:50*train_client_size],True)
# train_manual = generate_features(results_df,True)
train_manual.tail()

0    45015
1     4985
Name: target, dtype: int64


,roll_actual,roll_0,roll_1,roll_2,roll_3,roll_4,roll_5,roll_6,roll_7,roll_8,roll_9,roll_10,roll_11,roll_12,roll_13,roll_14,roll_15,roll_16,roll_17,roll_18,roll_19,roll_20,roll_21,roll_22,roll_23,roll_24,roll_25,roll_26,roll_27,roll_28,roll_29,roll_30,roll_31,roll_32,roll_33,roll_34,roll_35,roll_36,roll_37,roll_38,roll_39,roll_40,roll_41,roll_42,roll_43,roll_44,roll_45,roll_46,roll_47,roll_48,roll_49,roll_50,roll_51,roll_52,roll_53,roll_54,roll_55,roll_56,roll_57,roll_58,roll_59,roll_60,count_lt_1000,count_gt_5000,target,roll_hash_list_nonce
99995,6935,8004,3863,4148,7803,7323,7606,3273,2022,7557,9542,6302,4639,2502,1632,8449,4376,7421,8870,5257,8328,3321,2175,7594,9614,5104,9983,9563,2257,4835,4839,7703,2516,8620,4477,189,8717,151,6082,479,2251,3345,6488,7601,3738,8485,941,8944,8364,2376,7498,2365,1992,5469,3459,3258,5278,9287,2676,8280,2568,9669,4,31,0,"[8004, 9558, 4270, 2056, 2415, 9772, 7992, 6930, 6675, 4146, 9041, 6451, 3830, 3624, 4010, 5734, 1377, 8824, 2112, 1880, 225, 7955, 2652, 1695, 2764, 5288, 738, 5471, 3901, 8720, 1567, 7678, 4263, 9945, 140, 1169, 6891, 8009, 6198, 5048, 6727, 5089, 3884, 9365, 9317, 2150, 5171, 7254, 2959, 4749, 9278, 870, 7606, 6566, 6528, 3637, 3123, 2410, 4915, 3298, 3965]"
99996,4968,6003,9730,5349,4834,870,2851,872,183,1658,8479,6993,2737,6800,4290,6123,2390,1269,1140,6238,2107,7142,2618,7833,7369,8210,6583,6204,2728,8110,2830,1908,4615,8429,2796,8951,6416,6521,6852,5040,593,9459,9321,2739,8648,7438,18,4551,8984,7436,9746,8232,8370,1293,786,2718,2261,7269,7595,7975,5053,5656,6,34,0,"[6003, 6240, 3224, 6255, 9722, 743, 7059, 9505, 2030, 4339, 6078, 5792, 8214, 3968, 3221, 1071, 3971, 268, 2483, 9607, 188, 9194, 39, 4965, 7611, 9665, 4912, 1346, 7987, 2785, 4312, 4922, 7294, 4259, 1028, 857, 5182, 6662, 7484, 7001, 2906, 6399, 232, 9007, 9473, 9908, 1829, 8659, 6934, 6772, 7607, 9255, 5304, 2997, 5286, 6257, 5480, 277, 1088, 420, 5002]"
99997,2606,2005,2006,8027,9934,6748,3199,8575,5370,772,6693,875,5171,3132,4700,606,6274,220,6416,2877,4071,3519,8966,9064,5760,1507,1551,6308,9163,1301,2089,1441,6285,5304,3690,597,9228,3764,5664,9448,3022,1993,613,9924,9683,3055,2721,2686,593,3935,3947,5823,4399,4759,6314,2334,6020,2573,966,623,3750,5625,9,24,0,"[2005, 4329, 4850, 2211, 4878, 6884, 3891, 6900, 7682, 7265, 6868, 3708, 4695, 5771, 6065, 8252, 6610, 5082, 21, 4153, 8295, 5979, 1764, 1127, 8743, 6865, 5352, 3746, 4923, 8313, 198, 9702, 2691, 7837, 6026, 752, 827, 9387, 7348, 4818, 2875, 2595, 9497, 2033, 8913, 5179, 3212, 9765, 5526, 870, 6579, 1800, 9826, 1349, 3955, 301, 4273, 5055, 7815, 1242, 101]"
99998,7038,5002,6083,1452,6210,7218,8928,8462,7586,7442,7199,1432,6571,1943,7060,1483,2401,1283,8979,9280,844,8266,9602,7689,6209,2424,2030,6692,7344,7411,1630,1233,6276,1570,6172,2605,5024,794,1720,7523,6638,8882,589,8467,4703,2931,1881,5533,6528,828,8475,7310,2290,4724,8700,7512,7904,1324,3510,3101,6432,5453,4,35,0,"[5002, 4511, 6175, 6237, 7031, 5760, 3406, 551, 680, 6555, 42, 9454, 3948, 4738, 3170, 8718, 834, 5440, 5524, 54, 4308, 2156, 9475, 6654, 9251, 4069, 331, 9280, 1283, 4005, 7063, 8281, 2943, 7744, 6611, 2732, 188, 3880, 7823, 9341, 4571, 7024, 459, 1442, 4915, 4534, 4591, 9598, 9781, 3539, 1071, 7566, 4612, 7924, 8427, 1164, 2190, 6090, 3249, 4972, 6265]"
99999,5550,5009,9524,8385,3473,19,1680,906,2849,2442,8124,7159,2744,3529,741,7364,6013,5595,2665,6112,6181,7894,1778,5076,421,7752,8219,2372,9634,3752,726,8099,5102,7492,3567,2159,1300,9685,3699,9065,42,3664,3864,7182,4183,1149,4446,9087,9848,9545,606,7352,535,3050,3147,2911,6492,38,2876,8279,6473,6241,9,28,0,"[5009, 7255, 1021, 8320, 1509, 5871, 3730, 8691, 2737, 1879, 9062, 2957, 4306, 6811, 1289, 2100, 1446, 8750, 5741, 3060, 7096, 5457, 6765, 5979, 6255, 5127, 7974, 6399, 9697, 8962, 2055, 1198, 7098, 3924, 2834, 3613, 2001, 4189, 4277, 9515, 6454, 6899, 3687, 7901, 3860, 9300, 2833, 15, 7409, 6748, 9572, 6032, 9560, 4877, 1079, 3799, 7263, 4571, 6678, 8131, 2121]"


In [56]:
cols_roll = [col for col in train_manual.columns if col.startswith('roll_') and col not in ['roll_0','roll_actual','roll_hash_list_nonce']]
print(cols_roll)

['roll_1', 'roll_2', 'roll_3', 'roll_4', 'roll_5', 'roll_6', 'roll_7', 'roll_8', 'roll_9', 'roll_10', 'roll_11', 'roll_12', 'roll_13', 'roll_14', 'roll_15', 'roll_16', 'roll_17', 'roll_18', 'roll_19', 'roll_20', 'roll_21', 'roll_22', 'roll_23', 'roll_24', 'roll_25', 'roll_26', 'roll_27', 'roll_28', 'roll_29', 'roll_30', 'roll_31', 'roll_32', 'roll_33', 'roll_34', 'roll_35', 'roll_36', 'roll_37', 'roll_38', 'roll_39', 'roll_40', 'roll_41', 'roll_42', 'roll_43', 'roll_44', 'roll_45', 'roll_46', 'roll_47', 'roll_48', 'roll_49', 'roll_50', 'roll_51', 'roll_52', 'roll_53', 'roll_54', 'roll_55', 'roll_56', 'roll_57', 'roll_58', 'roll_59', 'roll_60']


In [57]:
def digit_9_stats(data,bHighCheck):
    
    after_vals = [8,10]
    if bHighCheck:
        cutoff_col = f'count_dig_{cur_digit}'
        zero_col = 'count_dig_0'
        after_col = 'count_hi'
        cutoff_range = range(0,7)
#         mask_actual = (data['roll_actual']>HIGH_TARGET)
        
    else:
        cutoff_col = 'count_dig_0'
        zero_col = f'count_dig_{cur_digit}'
        after_col = 'count_hi'
        cutoff_range = range(0,7)
#         mask_actual =(data['roll_actual']<LOW_TARGET)
        
    mask_actual = create_target_mask(data,bHighCheck)

        
    for cutoff in cutoff_range:
        
        print(f'********** cutoff:{cutoff}')
        mask = (data[cutoff_col]==cutoff) & (data[zero_col]==ZERO_COL_CUTOFF) 
        total = len(data[mask])
        if total ==0:
            success_count=0
            prob=0
        else:
            success_count = len(data[mask & mask_actual])
            prob =round(100 * success_count/ total ,0)
        print(f'before values - probability: {prob} total : {total}  success: {success_count}')
        for val in after_vals:
            mask2 = mask   & (data[after_col]<=val) #& (data['count_hi']+data['count_lo']<=18)
            total = len(data[mask2])
            if total ==0:
                success_count=0
                prob=0
            else:
                success_count = len(data[mask2 & mask_actual])
                prob =round(100 * success_count/ total ,0)
            print(f'AFTER - factor:{val} - probability: {prob} total : {total}  success: {success_count}')


In [58]:
def add_one_aft(nonce,zero_col_cutoff,pattern,bHighCheck,digit,cutoff,afterval,ratio_min,
                aft_cur,tr_total,tr_success,
               aft_all,aft_0,aft_1,
               analysis_df):
    if aft_cur >=ratio_min: 
        result=[nonce,zero_col_cutoff,pattern,bHighCheck,digit,cutoff,afterval,aft_cur,np.nan,
               aft_all,aft_0,aft_1,
               np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,
               tr_total,tr_success,
               np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan
               ]
        analysis_df.loc[len(analysis_df.index)]=result  
    return analysis_df

def save_train_stats(data,nonce,zero_col_cutoff,pattern,bHighCheck,ratio_min,
                    analysis_df=None):
    
    digits = [9,8,7,6,5,4,3,2,1]
    for digit in digits:
#         print(f'****** digit : {digit}')
        if analysis_df is None:
            columns = ['nonce','zero_col_cutoff','pattern_99','target_high','digit','cutoff','afterval','tr_ratio','te_ratio',
                      'aft_all','aft_1','aft_2',
                       'pr_mean','pr_med','pr_1','pr_2','pr_3','pr_4','pr_5','pr_tr_cutoff','pr_val_cutoff',
                      'tr_total','tr_success','te_total','te_success',
                      'te_pr_1_count','te_pr_2_count','te_pr_3_count',
                       'te_pr_4_count','te_pr_5_count',
                      'te_mean_count','te_med_count','te_tr_count','te_val_count']
            analysis_df = pd.DataFrame(columns=columns)

        after_vals = [8,10]
        if bHighCheck:
            cutoff_col = f'count_dig_{digit}'
            zero_col = 'count_dig_0'
            after_col = 'count_hi'
            cutoff_range = range(0,6)

        else:
            cutoff_col = 'count_dig_0'
            zero_col = f'count_dig_{digit}'
            after_col = 'count_hi'
            cutoff_range = range(0,6)

        mask_actual=create_target_mask(data,bHighCheck)
        for cutoff in cutoff_range:

#             print(f'********** cutoff:{cutoff}')
            mask = (data[cutoff_col]==cutoff) & (data[zero_col]==zero_col_cutoff) 
            total_all = len(data[mask])
            if total_all ==0:
                success_all=0
                aft_all=0
            else:
                success_all = len(data[mask & mask_actual])
                aft_all =round(100 * success_all/ total_all ,0)
#             print(f'before values - probability: {aft_all} total : {total_all}  success: {success_all}')
            aft = np.zeros(len(after_vals))
            success = np.zeros(len(after_vals))
            total = np.zeros(len(after_vals))

            for i,val in enumerate(after_vals):
                mask2 = mask   & (data[after_col]<=val) #& (data['count_hi']+data['count_lo']<=18)
                total[i] = len(data[mask2])
                if total[i] ==0:
                    success[i]=0
                    aft[i]=0
                else:
                    success[i] = len(data[mask2 & mask_actual])
                    aft[i] =round(100 * success[i]/ total[i] ,0)
#                 print(f'AFTER - factor:{val} - probability: {aft[i]} total : {total[i]}  success: {success[i]}')

            analysis_df= add_one_aft(nonce,zero_col_cutoff,pattern,bHighCheck,
                                     digit,cutoff,25,ratio_min,
                                    aft_all,total_all,success_all,
                                   aft_all,aft[0],aft[1],
                                   analysis_df)
            analysis_df= add_one_aft(nonce,zero_col_cutoff,pattern,bHighCheck,
                                     digit,cutoff,after_vals[0],ratio_min,
                                    aft[0],total[0],success[0],
                                   aft_all,aft[0],aft[1],
                                   analysis_df)
            analysis_df= add_one_aft(nonce,zero_col_cutoff,pattern,bHighCheck,
                                     digit,cutoff,after_vals[1],ratio_min,
                                    aft[1],total[1],success[1],
                                   aft_all,aft[0],aft[1],
                                   analysis_df)
    return analysis_df


In [59]:
cur_digit = 9
cur_zero_digit = 0

In [60]:
# train_manual.head()

In [61]:
# results_df.iloc[1072]

In [62]:
train_manual['roll_actual_dig_1'] = (train_manual['roll_actual'] / 1000).astype('int')
print(train_manual['roll_actual_dig_1'].value_counts(normalize=True))

train_manual['count_hi'] = (train_manual[cols_roll] > 5250).sum(axis=1)
train_manual['count_lo'] = (train_manual[cols_roll] < 4750).sum(axis=1)

train_manual['count_dig_1'] = ( (train_manual[cols_roll] / 1000).astype('int')==1).sum(axis=1)
train_manual['count_dig_2'] = ( (train_manual[cols_roll] / 1000).astype('int')==2).sum(axis=1)
train_manual['count_dig_3'] = ( (train_manual[cols_roll] / 1000).astype('int')==3).sum(axis=1)
train_manual['count_dig_4'] = ( (train_manual[cols_roll] / 1000).astype('int')==4).sum(axis=1)
train_manual['count_dig_5'] = ( (train_manual[cols_roll] / 1000).astype('int')==5).sum(axis=1)
train_manual['count_dig_6'] = ( (train_manual[cols_roll] / 1000).astype('int')==6).sum(axis=1)
train_manual['count_dig_9'] = ( (train_manual[cols_roll] / 1000).astype('int')==9).sum(axis=1)
train_manual['count_dig_8'] = ( (train_manual[cols_roll] / 1000).astype('int')==8).sum(axis=1)
train_manual['count_dig_7'] = ( (train_manual[cols_roll] / 1000).astype('int')==7).sum(axis=1)
train_manual['count_dig_0'] = ( (train_manual[cols_roll] / 1000).astype('int')==cur_zero_digit).sum(axis=1)
cols = ['roll_actual','roll_actual_dig_1',
        'count_dig_9','count_dig_0',
        'count_lt_1000','count_gt_5000','roll_0'] + cols_roll

7     0.10144
9     0.10140
2     0.10116
5     0.10074
1     0.10002
4     0.09990
0     0.09970
8     0.09934
3     0.09814
6     0.09810
10    0.00006
Name: roll_actual_dig_1, dtype: float64


In [63]:
# for cur_digit in [9,8,7,5]:
#     print()
#     print(f'************************** cur digit {cur_digit} ************************************************')
#     print('************************** HIGH ************************************************')
#     digit_9_stats(train_manual,True)
#     print()
#     print('************************** LOW ************************************************')
#     digit_9_stats(train_manual,False)

In [64]:
# %%time
# temp_df=None
# for zero_cutoff in range(0,6):
#     temp_df=save_train_stats(train_manual,nonce,zero_cutoff,PATTERN_99,HIGH_ANALYSIS,0,
#                         analysis_df=temp_df)
# temp_df

Prediction

In [65]:
# # hashval = '393cc340566014a83e78d302f079c98f5cd4e4bc4c34321231cf479559556038'#'3e24f29a3ae70e54aedd86b0c68640050be7dace8ae679f1fb85d1325a56ddae'
# seed = 'dummy' # hashval #'c77b3e783f094e255b0693f89094bed95ad9738d86f44fa4c09c8b0d58e2c73e'
# # nonce = 2548 #2390

In [66]:
# test_data = pd.DataFrame(columns=['seed','hash','nonce'])
# test_data.loc[0,'seed']=seed
# test_data.loc[0,'hash']=hashval
# test_data.loc[0,'nonce'] = nonce

# print(len(test_data))
# test_data.head()

In [67]:
# if PATTERN_99:
#     match_digit_arr=np.array([9,9])
#     mismatch_digit_arr=np.array([9,9])
# else:
#     match_digit_arr=np.array([0,0])
#     mismatch_digit_arr=np.array([0,0])
    
# match_digit_indices = np.array([1,2])
# match_digit_count_arr=np.array([1])
# mismatch_digit_indices = np.array([0,3])


In [68]:
# %%time
# # match_digit_arr = np.array(['999',])
# # match_digit_count_arr=np.array([1])
# # match_digit_indices = np.array(['012'])

# # match_digit_arr=np.array([9,9])
# # match_digit_indices = np.array([1,2])
# # match_digit_count_arr=np.array([1])

# if not READ_FROM_FILE_TEST:

#     cur_client_seed_array = client_seed_array_2[:1000001]

#     results_df_test = None

#     last_index_test =  len(cur_client_seed_array)-1 #1024
#     start = 0

#     results_df_test = generate_data(test_data,
#                                 cur_client_seed_array,
#                                  ABOVE,
#                         match_digit_arr,match_digit_count_arr,match_digit_indices=match_digit_indices,
#                         mismatch_digit_arr=mismatch_digit_arr,mismatch_digit_indices=mismatch_digit_indices,
#                         match_count_exp=10000, #train_client_size,
#                         feature_chain_length=20,
#                         is_data_hash = False,
#                         test_limit=1,
#                         last_index=last_index_test,result_df=results_df_test,
#                         print_client_scan=False)
#     row = results_df_test.iloc[len(results_df_test)-1]
#     last_index_test = row['client_index']

In [69]:
# print(results_df_test['client_index'].min())
# len(results_df_test)

In [70]:
def generate_test_features(initial_df):
    rolls_df = initial_df['roll_hash_list'].apply(pd.Series)
    rolls_df.columns = [f'roll_{i}' for i in range(rolls_df.shape[1])]
    cols_roll = [col for col in rolls_df.columns if col.startswith('roll_') and col not in ['roll_0']]

    print(cols_roll)
    rolls_df['count_hi'] = (rolls_df[cols_roll] > 5250).sum(axis=1)
    rolls_df['count_lo'] = (rolls_df[cols_roll] < 4750).sum(axis=1)

    rolls_df['count_dig_1'] = ( (rolls_df[cols_roll] / 1000).astype('int')==1).sum(axis=1)
    rolls_df['count_dig_2'] = ( (rolls_df[cols_roll] / 1000).astype('int')==2).sum(axis=1)
    rolls_df['count_dig_3'] = ( (rolls_df[cols_roll] / 1000).astype('int')==3).sum(axis=1)
    rolls_df['count_dig_4'] = ( (rolls_df[cols_roll] / 1000).astype('int')==4).sum(axis=1)
    rolls_df['count_dig_5'] = ( (rolls_df[cols_roll] / 1000).astype('int')==5).sum(axis=1)
    rolls_df['count_dig_6'] = ( (rolls_df[cols_roll] / 1000).astype('int')==6).sum(axis=1)
    rolls_df['count_dig_9'] = ( (rolls_df[cols_roll] / 1000).astype('int')==9).sum(axis=1)
    rolls_df['count_dig_8'] = ( (rolls_df[cols_roll] / 1000).astype('int')==8).sum(axis=1)
    rolls_df['count_dig_7'] = ( (rolls_df[cols_roll] / 1000).astype('int')==7).sum(axis=1)
    rolls_df['count_dig_0'] = ( (rolls_df[cols_roll] / 1000).astype('int')==cur_zero_digit).sum(axis=1)
    
    
    rolls_df['roll_actual'] = initial_df['roll_actual']
    rolls_df['client_seed'] = initial_df['client_seed']
    rolls_df['client_index'] = initial_df['client_index']

    return rolls_df


In [71]:
def gen_test_data(cur_nonce,cur_hash,cur_seed,file_suffix=""):
    test_data = pd.DataFrame(columns=['seed','hash','nonce'])
    test_data.loc[0,'seed']=cur_seed
    test_data.loc[0,'hash']=cur_hash
    test_data.loc[0,'nonce'] = cur_nonce

    print(len(test_data))
    print(test_data.head())
    
    if PATTERN_99:
        match_digit_arr=np.array([9,9])
        mismatch_digit_arr=np.array([9,9])
    else:
        match_digit_arr=np.array([0,0])
        mismatch_digit_arr=np.array([0,0])

    match_digit_indices = np.array([1,2])
    match_digit_count_arr=np.array([1])
    mismatch_digit_indices = np.array([0,3]) 
    
    if PATTERN_99:
        file_pattern_str = 'x99x'
    else:
        file_pattern_str = 'x00x'
        
    filename = f'data/results_df_test_{nonce}_{file_pattern_str}_pattern{file_suffix}.csv'
    
    if  READ_FROM_FILE_TEST:
        results_df_test = pd.read_csv(filename, converters={'roll_hash_list': pd.eval,
                                                        'roll_hash_list_nonce': pd.eval})
    else:

        cur_client_seed_array = client_seed_array_2[:1000001]
        results_df_test = None

        last_index_test =  len(cur_client_seed_array)-1 #1024
        start = 0

        results_df_test = generate_data(test_data,
                                    cur_client_seed_array,
                                     ABOVE,
                            match_digit_arr,match_digit_count_arr,match_digit_indices=match_digit_indices,
                            mismatch_digit_arr=mismatch_digit_arr,mismatch_digit_indices=mismatch_digit_indices,
                            match_count_exp=10000, #train_client_size,
                            feature_chain_length=feature_chain_length,
                            is_data_hash = False,
                            test_limit=1,
                            last_index=last_index_test,result_df=results_df_test,
                            print_client_scan=False)
        #Remove client seed duplicates from test data
        results_df_test= results_df_test.drop_duplicates(subset='client_seed')
        
        results_df_test.to_csv(filename,index=False)
        
    print('Test Data Generation Completed')    
    test_manual = generate_test_features(results_df_test)
    print('Test Features Completed')    
        
#         row = results_df_test.iloc[len(results_df_test)-1]
#         last_index_test = row['client_index']   

    return results_df_test,test_manual
 

In [72]:
%%time
READ_FROM_FILE_TEST=False
# seed='dummy'
seed = generate_hash_chain(hashval,26)[25]
print('seed:',seed)
results_df_test,test_manual=gen_test_data(nonce,hashval,seed,file_suffix="")
print()
print(results_df_test[['seed','hash','nonce']].head(1))
test_manual.head()

seed: 809d6a154ad7c7dcb33d1389bad8f809197e7b78ecb9b1c859fae02150615ecf
1
                                                               seed  \
0  809d6a154ad7c7dcb33d1389bad8f809197e7b78ecb9b1c859fae02150615ecf   

                                                               hash nonce  
0  416e56f7cc46f549f3f9946a5264475854a53d5c363487535a06fcc066d7120e  2653  
server_count:0

999999 999999
No further match for seed:809d6a154ad7c7dcb33d1389bad8f809197e7b78ecb9b1c859fae02150615ecf nonce:2653 match count: 8115

Test Data Generation Completed
['roll_1', 'roll_2', 'roll_3', 'roll_4', 'roll_5', 'roll_6', 'roll_7', 'roll_8', 'roll_9', 'roll_10', 'roll_11', 'roll_12', 'roll_13', 'roll_14', 'roll_15', 'roll_16', 'roll_17', 'roll_18', 'roll_19', 'roll_20', 'roll_21', 'roll_22', 'roll_23', 'roll_24', 'roll_25', 'roll_26', 'roll_27', 'roll_28', 'roll_29', 'roll_30', 'roll_31', 'roll_32', 'roll_33', 'roll_34', 'roll_35', 'roll_36', 'roll_37', 'roll_38', 'roll_39', 'roll_40', 'roll_41', 'roll_4

,roll_0,roll_1,roll_2,roll_3,roll_4,roll_5,roll_6,roll_7,roll_8,roll_9,roll_10,roll_11,roll_12,roll_13,roll_14,roll_15,roll_16,roll_17,roll_18,roll_19,roll_20,roll_21,roll_22,roll_23,roll_24,roll_25,roll_26,roll_27,roll_28,roll_29,roll_30,roll_31,roll_32,roll_33,roll_34,roll_35,roll_36,roll_37,roll_38,roll_39,roll_40,roll_41,roll_42,roll_43,roll_44,roll_45,roll_46,roll_47,roll_48,roll_49,roll_50,roll_51,roll_52,roll_53,roll_54,roll_55,roll_56,roll_57,roll_58,roll_59,roll_60,count_hi,count_lo,count_dig_1,count_dig_2,count_dig_3,count_dig_4,count_dig_5,count_dig_6,count_dig_9,count_dig_8,count_dig_7,count_dig_0,roll_actual,client_seed,client_index
0,8004,3529,7199,8036,618,5991,9163,2888,8359,3603,1858,2901,9426,7593,1660,2058,9350,390,5340,633,6368,2912,1827,3928,4330,4908,9928,4835,9599,326,1507,8578,3773,165,9107,6209,6788,7771,9973,7347,1972,2564,8596,1981,375,7528,1757,63,6122,1907,9304,2735,6261,1038,5830,5555,6103,9300,5021,2441,6333,29,28,9,7,4,3,5,7,9,4,5,7,4908,woxpwoxpwoxpwoxpwoxp0bbe6de4a229ac3e15d00974678283561c5e60e77afb77ecec3c2323117db51b,999920
1,7007,9474,7245,6893,2485,6050,5309,5688,7855,4371,9806,8376,5274,4255,8803,279,6595,1023,3999,1632,2136,11,8063,2647,4664,9427,8190,9761,3164,4066,6750,9275,9874,7150,7042,3165,3406,4631,5355,3994,7001,4342,4022,9324,852,3223,6138,2274,565,7718,3418,814,9354,6580,8378,7637,5062,8232,5579,3693,7757,33,26,2,4,8,7,6,6,8,6,8,5,9427,woxpwoxpwoxpwoxpwoxpe96b82cb6801d2a3645779e55c914e87230a1b1273dd4908d362912b4878f387,999526
2,2003,5468,76,1326,816,2272,6813,4660,9574,7999,6948,9975,2414,455,9237,311,813,5254,1902,8226,7407,4992,18,9996,2585,1113,999,3285,7274,7947,6121,5313,1116,9485,1053,4467,138,1458,46,1974,4226,4526,3996,8578,7538,565,8915,1412,4764,2242,4390,3111,6702,7896,7859,7390,7019,7873,2672,1891,3741,25,33,9,5,4,7,3,4,5,3,10,10,1113,woxpwoxpwoxpwoxpwoxpb514f92b44049f629e07b040a4065dfd6a55fe193fb64b5a287be96797f9eaf4,999477
3,4003,7815,7649,3607,1821,5274,2039,5479,6589,8805,4219,3600,3194,5613,3581,1041,7958,4468,2104,1043,8291,4120,8864,1831,5373,2464,6046,2627,1105,1379,5134,8149,5455,1881,4540,5552,6058,2337,8316,588,8670,6067,7584,9871,3375,298,5859,2432,4564,5587,9918,1571,716,6348,5740,4361,5800,7106,1568,5976,2895,29,30,9,7,5,6,12,5,2,6,5,3,2464,woxpwoxpwoxpwoxpwoxpce9ec986ce1d78b27e01bd16c32bf2a2d2b2a6f240c7cb279e832c39c9c45023,999349
4,9008,6405,303,5697,5202,829,2678,9671,3446,2854,7916,7798,4042,3121,1097,1978,6640,4906,3228,4437,3826,3843,9253,4258,4081,7090,6311,6866,6606,2741,179,6806,29,1951,2662,7189,3568,6053,1791,6936,9603,3723,4816,7199,8199,2924,7113,2336,3486,8712,1429,648,2989,1204,9173,4659,8741,9638,9471,1441,7275,25,32,7,7,8,7,2,8,6,3,7,5,7090,woxpwoxpwoxpwoxpwoxp18ba950788686fb86182c64f4fca2224fba2eae5ad04b86a7437f4df5364807c,999305


In [73]:
%%time
READ_FROM_FILE_TEST=False
#Generate hash test data
print(f'{hashval=}')
hash_l2=generate_hash(hashval.encode())
print(f'{hash_l2=}')
results_df_test_hash,test_manual_hash=gen_test_data(nonce,hash_l2,hashval,file_suffix="_hash")
print()
print(results_df_test_hash.head(1))
test_manual_hash.head()

hashval='416e56f7cc46f549f3f9946a5264475854a53d5c363487535a06fcc066d7120e'
hash_l2='37852b2aec85dc4ef278a4f9eb6e83cff933d03f334d39bf3e4f189b0d364774'
1
                                                               seed  \
0  416e56f7cc46f549f3f9946a5264475854a53d5c363487535a06fcc066d7120e   

                                                               hash nonce  
0  37852b2aec85dc4ef278a4f9eb6e83cff933d03f334d39bf3e4f189b0d364774  2653  
server_count:0

999999 999999
No further match for seed:416e56f7cc46f549f3f9946a5264475854a53d5c363487535a06fcc066d7120e nonce:2653 match count: 8205

Test Data Generation Completed
['roll_1', 'roll_2', 'roll_3', 'roll_4', 'roll_5', 'roll_6', 'roll_7', 'roll_8', 'roll_9', 'roll_10', 'roll_11', 'roll_12', 'roll_13', 'roll_14', 'roll_15', 'roll_16', 'roll_17', 'roll_18', 'roll_19', 'roll_20', 'roll_21', 'roll_22', 'roll_23', 'roll_24', 'roll_25', 'roll_26', 'roll_27', 'roll_28', 'roll_29', 'roll_30', 'roll_31', 'roll_32', 'roll_33', 'roll_34', 'roll

,roll_0,roll_1,roll_2,roll_3,roll_4,roll_5,roll_6,roll_7,roll_8,roll_9,roll_10,roll_11,roll_12,roll_13,roll_14,roll_15,roll_16,roll_17,roll_18,roll_19,roll_20,roll_21,roll_22,roll_23,roll_24,roll_25,roll_26,roll_27,roll_28,roll_29,roll_30,roll_31,roll_32,roll_33,roll_34,roll_35,roll_36,roll_37,roll_38,roll_39,roll_40,roll_41,roll_42,roll_43,roll_44,roll_45,roll_46,roll_47,roll_48,roll_49,roll_50,roll_51,roll_52,roll_53,roll_54,roll_55,roll_56,roll_57,roll_58,roll_59,roll_60,count_hi,count_lo,count_dig_1,count_dig_2,count_dig_3,count_dig_4,count_dig_5,count_dig_6,count_dig_9,count_dig_8,count_dig_7,count_dig_0,roll_actual,client_seed,client_index
0,1001,6163,9140,3222,5167,7611,9058,5191,4677,3246,5666,3200,4990,9474,8081,1823,3807,23,9435,7679,2523,2000,917,4401,4313,8411,4190,1208,1701,2082,8587,1754,4159,5837,6092,9574,8123,1574,8550,9330,1469,9149,3605,2981,1,4401,2918,4533,2532,2021,4827,8463,6310,6910,3187,7318,6973,493,8244,4875,8448,25,30,6,7,6,10,4,5,7,8,3,4,6731,woxpwoxpwoxpwoxpwoxpadf10d5d3a6a0b9b4ce5423d09c20ccae2fd7f1af5d6dbd68bd48ff9626a9449,999966
1,6008,9968,5706,6859,5268,8393,5775,8439,5264,7902,8783,2350,2441,1534,9838,994,9100,341,8139,5364,9423,1766,7200,10,6660,6470,5947,5323,8261,5813,7776,1614,6655,1232,6104,329,9484,7288,7505,5949,785,1547,248,7522,1476,6175,1575,4841,8377,1939,6520,385,7222,7487,388,4744,1275,808,6004,1999,7120,37,22,10,2,0,2,9,8,5,6,9,9,2758,woxpwoxpwoxpwoxpwoxpd4ebb77a8b670e5888d3b61cd2eba7ae91299eaab7eee0d1d8e949e53977848a,999901
2,2009,7787,7855,7946,1819,3973,6996,2976,9108,1368,8664,3003,9129,6931,565,3371,7186,1952,629,8265,6568,3938,3292,6444,3510,3908,6234,2576,6285,7480,7718,611,728,8189,5852,7335,6307,76,2578,6770,642,4377,2900,5786,2218,1246,7657,1788,9016,1298,1528,3276,559,3170,654,3668,1960,8317,3444,7632,4124,26,34,8,5,11,2,2,8,3,4,9,8,4875,woxpwoxpwoxpwoxpwoxp270812fe08ed514d0f01ea5dc929e1534b852af245c6a3f17fd6a174ef9e0304,999779
3,3006,3008,7418,6677,5100,7534,5483,3288,8459,8180,7290,8876,4418,4016,6779,9314,529,4185,3952,5009,1133,139,4759,4005,4089,6767,3666,6661,3706,5171,2817,4746,7511,5349,111,6543,6157,7283,7643,791,3494,2593,3090,8635,6624,5892,3146,1679,9924,1299,1855,747,1353,2572,8939,1512,39,2443,3658,7941,7027,25,31,6,4,9,7,6,7,2,5,8,6,9450,woxpwoxpwoxpwoxpwoxpaa00cf4f9b89f30d74296bd3ef6d53afc7ffb0a1b6e4b8406392bc88aec63f1d,999762
4,8007,2792,2296,7424,2787,4156,5825,905,2592,844,1154,6889,462,1602,6276,4510,5644,5148,8110,7026,1970,3187,597,5474,1758,4396,6277,7809,2714,6257,2709,5850,742,8321,651,1465,9476,4600,1356,8459,8855,5339,6224,1230,6330,2624,8330,4343,6586,7990,7556,9766,7636,5342,4693,2095,7737,8401,5885,2722,8470,30,29,7,9,1,6,8,7,2,7,7,6,9516,woxpwoxpwoxpwoxpwoxpccdeaee9711fb94af47578fe0de4696aedeae4755f36e34afc734240c8bb0bf6,999506


In [74]:
%%time
READ_FROM_FILE_TEST=False
cur_seed = cur_hash_list[25]
cur_hash = cur_hash_list[26]
print(f'{cur_seed=}')
print(f'{cur_hash=}')

results_df_test_hash25,test_manual_hash25=gen_test_data(nonce,cur_hash,cur_seed,file_suffix="_hash25")
print()
print(results_df_test_hash25.head(1))
test_manual_hash25.head()

cur_seed='809d6a154ad7c7dcb33d1389bad8f809197e7b78ecb9b1c859fae02150615ecf'
cur_hash='b28bd5ec59ccfabecd327dc2e0fa79a5a0a1d79eebd3a8f4e3c7b9c194f6b7c6'
1
                                                               seed  \
0  809d6a154ad7c7dcb33d1389bad8f809197e7b78ecb9b1c859fae02150615ecf   

                                                               hash nonce  
0  b28bd5ec59ccfabecd327dc2e0fa79a5a0a1d79eebd3a8f4e3c7b9c194f6b7c6  2653  
server_count:0

999999 999999
No further match for seed:809d6a154ad7c7dcb33d1389bad8f809197e7b78ecb9b1c859fae02150615ecf nonce:2653 match count: 7935

Test Data Generation Completed
['roll_1', 'roll_2', 'roll_3', 'roll_4', 'roll_5', 'roll_6', 'roll_7', 'roll_8', 'roll_9', 'roll_10', 'roll_11', 'roll_12', 'roll_13', 'roll_14', 'roll_15', 'roll_16', 'roll_17', 'roll_18', 'roll_19', 'roll_20', 'roll_21', 'roll_22', 'roll_23', 'roll_24', 'roll_25', 'roll_26', 'roll_27', 'roll_28', 'roll_29', 'roll_30', 'roll_31', 'roll_32', 'roll_33', 'roll_34', 'ro

,roll_0,roll_1,roll_2,roll_3,roll_4,roll_5,roll_6,roll_7,roll_8,roll_9,roll_10,roll_11,roll_12,roll_13,roll_14,roll_15,roll_16,roll_17,roll_18,roll_19,roll_20,roll_21,roll_22,roll_23,roll_24,roll_25,roll_26,roll_27,roll_28,roll_29,roll_30,roll_31,roll_32,roll_33,roll_34,roll_35,roll_36,roll_37,roll_38,roll_39,roll_40,roll_41,roll_42,roll_43,roll_44,roll_45,roll_46,roll_47,roll_48,roll_49,roll_50,roll_51,roll_52,roll_53,roll_54,roll_55,roll_56,roll_57,roll_58,roll_59,roll_60,count_hi,count_lo,count_dig_1,count_dig_2,count_dig_3,count_dig_4,count_dig_5,count_dig_6,count_dig_9,count_dig_8,count_dig_7,count_dig_0,roll_actual,client_seed,client_index
0,5002,3974,7743,3495,8884,4530,6279,9716,6015,2760,9353,5382,7668,5661,6358,5394,5095,5471,2557,2498,6127,6753,545,9245,201,3379,492,3664,1750,7478,1162,7882,5865,4003,4714,1096,615,1137,724,7094,7156,3135,8073,8840,8082,329,4321,1604,6663,3456,6255,7858,9003,996,5206,6990,5312,1496,4559,8431,9199,31,27,6,3,6,5,8,8,5,5,7,7,1457,woxpwoxpwoxpwoxpwoxp9c14e1c25909be397ba22150fff4fb7f4c3b13a75cb0f6f93c1a71e041b6678d,999904
1,2005,6534,5127,4506,5648,6986,2359,3465,3283,653,3847,6453,1920,5890,7872,9720,3929,1123,5226,5161,5212,4280,2828,194,6681,9770,7999,7465,8221,6828,571,9394,6484,9417,5691,5469,3965,8250,493,8547,7460,9314,9819,4781,1752,6745,6739,147,6513,3367,6796,4341,2874,7849,6206,3023,1436,6796,9412,9755,2671,32,23,4,4,7,4,8,12,8,3,5,5,5766,woxpwoxpwoxpwoxpwoxpe3c057ab2a1b78f430287e038139bbe17b28f213d02549190a3e1d8c9d21544f,999893
2,6009,1883,5766,2139,1622,1610,6027,4249,2301,9367,6376,5960,1613,4721,624,8587,6692,3929,2223,6476,6642,8587,1594,279,5717,8340,6246,2122,8843,5908,564,6277,1924,3010,5136,5325,9467,1882,9893,7622,1941,6675,8572,1964,3499,445,5336,5932,518,1898,428,1497,6209,513,3494,3494,1388,3292,7755,1617,4870,26,32,13,4,6,3,8,9,3,5,2,7,6055,woxpwoxpwoxpwoxpwoxp2a53bc81f79dac2d6caeb183cc8b2e5ce50e8c5a4b538a63cbf421ca945d5678,999796
3,5005,2599,6442,3309,1442,3873,2710,2440,3699,8308,111,1844,4293,9201,1718,5114,4930,227,8175,4923,7857,9675,8005,3539,237,1834,6499,6557,8689,7597,185,1571,3912,4568,6570,7201,2268,4091,5056,375,4731,321,583,5932,3869,4327,2437,1610,3230,3606,3762,7555,1501,9465,5928,6171,8153,8936,368,7650,7329,22,34,7,5,9,7,4,5,3,6,6,8,2896,woxpwoxpwoxpwoxpwoxp0d5ebad028dfad020927079a9040cdd85ee16ac78a9729e99f75a8eb41a76c4b,999298
4,1002,4900,1148,2783,6418,4275,2310,9286,6407,5411,6078,997,5717,1458,8131,8612,4859,3979,6864,6199,6531,8930,1907,6260,7557,2885,9452,1476,5750,2777,3949,4530,4062,7014,8399,696,9415,7940,2320,7729,1085,7737,964,1083,4970,6481,8158,2221,5701,6337,6358,2723,6708,9936,5640,4327,424,2608,823,7653,4450,31,26,6,8,2,8,5,11,4,5,6,5,8317,woxpwoxpwoxpwoxpwoxpe4eba1a9447bbf1fe37fd807e8c16240c1e46a6ac86d097a85fb0f6f2b000f5e,999095


In [75]:
%%time
READ_FROM_FILE_TEST=False
cur_seed = cur_hash_list[24]
cur_hash = cur_hash_list[25]
print(f'{cur_seed=}')
print(f'{cur_hash=}')

results_df_test_hash24,test_manual_hash24=gen_test_data(nonce,cur_hash,cur_seed,file_suffix="_hash24")
print()
print(results_df_test_hash24.head(1))
test_manual_hash24.head()

cur_seed='d8917c50fcf883feab02fb44ab74fe42b67e7c36b08aae593766810853bc36f5'
cur_hash='809d6a154ad7c7dcb33d1389bad8f809197e7b78ecb9b1c859fae02150615ecf'
1
                                                               seed  \
0  d8917c50fcf883feab02fb44ab74fe42b67e7c36b08aae593766810853bc36f5   

                                                               hash nonce  
0  809d6a154ad7c7dcb33d1389bad8f809197e7b78ecb9b1c859fae02150615ecf  2653  
server_count:0

999999 999999
No further match for seed:d8917c50fcf883feab02fb44ab74fe42b67e7c36b08aae593766810853bc36f5 nonce:2653 match count: 8111

Test Data Generation Completed
['roll_1', 'roll_2', 'roll_3', 'roll_4', 'roll_5', 'roll_6', 'roll_7', 'roll_8', 'roll_9', 'roll_10', 'roll_11', 'roll_12', 'roll_13', 'roll_14', 'roll_15', 'roll_16', 'roll_17', 'roll_18', 'roll_19', 'roll_20', 'roll_21', 'roll_22', 'roll_23', 'roll_24', 'roll_25', 'roll_26', 'roll_27', 'roll_28', 'roll_29', 'roll_30', 'roll_31', 'roll_32', 'roll_33', 'roll_34', 'ro

,roll_0,roll_1,roll_2,roll_3,roll_4,roll_5,roll_6,roll_7,roll_8,roll_9,roll_10,roll_11,roll_12,roll_13,roll_14,roll_15,roll_16,roll_17,roll_18,roll_19,roll_20,roll_21,roll_22,roll_23,roll_24,roll_25,roll_26,roll_27,roll_28,roll_29,roll_30,roll_31,roll_32,roll_33,roll_34,roll_35,roll_36,roll_37,roll_38,roll_39,roll_40,roll_41,roll_42,roll_43,roll_44,roll_45,roll_46,roll_47,roll_48,roll_49,roll_50,roll_51,roll_52,roll_53,roll_54,roll_55,roll_56,roll_57,roll_58,roll_59,roll_60,count_hi,count_lo,count_dig_1,count_dig_2,count_dig_3,count_dig_4,count_dig_5,count_dig_6,count_dig_9,count_dig_8,count_dig_7,count_dig_0,roll_actual,client_seed,client_index
0,8007,879,7864,1505,8271,4401,9005,551,5720,8213,5823,1412,3267,3421,8982,6122,4718,6979,4628,4212,4010,2454,5390,9086,62,8399,9059,9431,4357,4408,302,1342,7548,605,4743,5626,2846,1512,9693,4419,1231,2696,9872,1614,3659,777,4261,1601,8192,569,652,8455,2288,1300,2701,5887,5623,1541,2605,1345,8269,23,37,10,6,3,10,6,2,6,7,2,8,8703,woxpwoxpwoxpwoxpwoxpfbb64e954735b397895248c505651f17750f3beaed39a17ad29454c89360e9d4,999918
1,2009,9290,8811,218,9460,4374,87,465,1629,2004,3009,9162,8821,8515,9248,5255,4949,2936,6374,3274,3000,3546,8402,2321,1708,3086,8841,83,4187,7504,8007,1482,2055,9292,9444,5716,9372,1742,7732,7002,6603,7032,3223,9768,3130,1678,8924,5811,9561,6319,8712,7874,4792,8242,2690,9424,4398,4079,1870,165,2121,30,28,6,6,7,6,3,3,10,9,5,5,1488,woxpwoxpwoxpwoxpwoxpd3eefb654e1f5c2f337b61d41150149e3c5faf1bdd7bb908418e5889afacc89a,999638
2,5003,2869,7542,5486,4836,7283,4165,5917,2034,702,4107,4601,2857,4207,3754,4195,9919,787,8822,3855,6895,1123,5068,843,3352,2720,550,4935,8893,3618,7244,4384,1251,3587,8673,4497,5297,6059,9631,9463,7909,6720,720,8392,3041,226,5373,3337,7077,8410,7752,6608,5253,7122,9459,3081,434,7539,4422,2447,4655,26,31,2,5,8,11,6,4,4,5,8,7,723,woxpwoxpwoxpwoxpwoxp6fd304138b7ba507b6701a71d2ed21255fceb84b1ecf2c17c3051cf135005d07,999631
3,3007,5243,4147,6136,8650,9267,7120,3435,1072,8298,4601,5135,9782,4953,2720,2485,6126,4752,2512,47,5121,9709,8708,1305,403,4558,3069,8796,8166,680,5586,2516,5517,9684,2609,3241,5961,2115,445,5130,4854,5457,4856,3618,8102,3937,9642,5669,2992,6171,9261,5367,5100,2503,1979,4579,3378,2882,6094,9082,2723,24,27,3,10,6,8,11,4,7,6,1,4,3608,woxpwoxpwoxpwoxpwoxpeff453da8642eacd230b6cdbb08d935f54534cff5c3e01a514039b727a12f14d,999363
4,3007,937,7009,6472,2131,9628,6396,2313,866,7070,6668,693,1577,5623,4201,1286,8585,6666,3558,6024,9818,3321,4175,4057,2654,1945,1742,7970,9580,4130,729,3587,7116,7824,977,6387,279,4200,6141,4666,5504,6792,4608,2953,8614,497,5063,1621,7979,7610,2525,8473,9259,8585,8505,7547,5249,6194,8079,8737,4176,30,28,5,5,3,8,4,9,4,7,8,7,9505,woxpwoxpwoxpwoxpwoxpc2f06ec7f561be0b18f2355c5eb30e2a6b6615c9942c4e87588c56aa1bba5c71,999302


In [76]:
# %%time
# if PATTERN_99:
#     file_pattern_str = 'x99x'
# else:
#     file_pattern_str = 'x00x'
# filename = f'data/results_df_test_{nonce}_{file_pattern_str}_pattern.csv'

# if READ_FROM_FILE_TEST:
#     results_df_test = pd.read_csv(filename, converters={'roll_hash_list': pd.eval,
#                                                         'roll_hash_list_nonce': pd.eval})
# else:
#     results_df_test.to_csv(filename,index=False)

In [77]:
# s=70
# e=80
# l=90
# client_seed_min_1=results_df[s*train_client_size:e*train_client_size]['client_index'].min()
# client_seed_max_1=results_df[s*train_client_size:e*train_client_size]['client_index'].max()
# print(client_seed_min_1,client_seed_max_1)
# results_1 = results_df[s*train_client_size:e*train_client_size]
# print(results_1['client_index'].describe(percentiles=[0.09,0.1,0.15,0.2,0.25]))

# client_seed_min=results_df[e*train_client_size:l*train_client_size]['client_index'].min()
# client_seed_max=results_df[e*train_client_size:l*train_client_size]['client_index'].max()
# print(client_seed_min,client_seed_max)
# results_2 = results_df[e*train_client_size:l*train_client_size]
# print(results_2['client_index'].describe(percentiles=[0.05,0.06,0.07,0.08,0.09,0.25]))
# client_seed_min_2=results_2[results_2['client_index']>client_seed_max_1]['client_index'].min()
# client_seed_max_2=results_2['client_index'].max()
# print(client_seed_min_2,client_seed_max_2)


In [78]:
# client_seed_min_1=results_df[25*train_client_size:39*train_client_size]['client_index'].min()
# client_seed_max_1=results_df[25*train_client_size:39*train_client_size]['client_index'].max()
# print(client_seed_min_1,client_seed_max_1)

# client_seed_min=results_df[39*train_client_size:50*train_client_size]['client_index'].min()
# client_seed_max=results_df[39*train_client_size:50*train_client_size]['client_index'].max()
# print(client_seed_min,client_seed_max)
# print(results_df[39*train_client_size:50*train_client_size]['client_index'].describe(percentiles=[0.05,0.06,0.07,0.08,0.09,0.25]))
# results_2 = results_df[39*train_client_size:50*train_client_size]
# client_seed_min_2=results_2[results_2['client_index']>client_seed_max_1]['client_index'].min()
# client_seed_max_2=results_2['client_index'].max()
# print(client_seed_min_2,client_seed_max_2)


In [79]:
# mask = (results_df_test['client_index']<=client_seed_max_1) | (results_df_test['client_index']>=client_seed_min_2)
# results_df_test = results_df_test[mask]
# print(len(results_df_test))
# print(results_df_test['client_index'].describe(percentiles=[0.25,0.5,0.52,0.55,0.57,0.6,0.7,0.8,0.9]))

In [80]:
# test_manual = generate_test_features(results_df_test)

In [81]:
# # rand=random.randint(0,len(rolls_df)-1)
# rand=np.random.choice(range(0,len(rolls_df)))
# print('random location:',rand)
# rolls_df.iloc[rand]['client_seed']

In [82]:
# res = []
# for i in range(100):
#     res.append(np.random.choice(range(0,len(rolls_df))) )
    
# pd.Series(np.array(res)).value_counts()

In [83]:

def check_special_pattern_match(data,bHighCheck,cutoff,after_val):
     
#     rolls_df = initial_df['roll_hash_list'].apply(pd.Series)
#     rolls_df.columns = [f'roll_{i}' for i in range(rolls_df.shape[1])]
#     cols_roll = [col for col in rolls_df.columns if col.startswith('roll_') and col not in ['roll_0']]
    
    if bHighCheck:
        cutoff_col = f'count_dig_{cur_digit}'
        zero_col = 'count_dig_0'
        after_col = 'count_hi'
    else:
        cutoff_col = 'count_dig_0'
        zero_col = f'count_dig_{cur_digit}'
        after_col = 'count_hi'

#     rolls_df['count_hi'] = (rolls_df[cols_roll] > 5250).sum(axis=1)
#     rolls_df['count_lo'] = (rolls_df[cols_roll] < 4750).sum(axis=1)

#     rolls_df['count_dig_1'] = ( (rolls_df[cols_roll] / 1000).astype('int')==1).sum(axis=1)
#     rolls_df['count_dig_2'] = ( (rolls_df[cols_roll] / 1000).astype('int')==2).sum(axis=1)
#     rolls_df['count_dig_3'] = ( (rolls_df[cols_roll] / 1000).astype('int')==3).sum(axis=1)
#     rolls_df['count_dig_4'] = ( (rolls_df[cols_roll] / 1000).astype('int')==4).sum(axis=1)
#     rolls_df['count_dig_5'] = ( (rolls_df[cols_roll] / 1000).astype('int')==5).sum(axis=1)
#     rolls_df['count_dig_6'] = ( (rolls_df[cols_roll] / 1000).astype('int')==6).sum(axis=1)
#     rolls_df['count_dig_9'] = ( (rolls_df[cols_roll] / 1000).astype('int')==9).sum(axis=1)
#     rolls_df['count_dig_8'] = ( (rolls_df[cols_roll] / 1000).astype('int')==8).sum(axis=1)
#     rolls_df['count_dig_7'] = ( (rolls_df[cols_roll] / 1000).astype('int')==7).sum(axis=1)
#     rolls_df['count_dig_0'] = ( (rolls_df[cols_roll] / 1000).astype('int')==cur_zero_digit).sum(axis=1)
    
    
#     rolls_df['roll_actual'] = initial_df['roll_actual']
#     rolls_df['client_seed'] = initial_df['client_seed']
#     rolls_df['client_index'] = initial_df['client_index']

    
    mask = (data[cutoff_col]==cutoff) & (data[zero_col]==ZERO_COL_CUTOFF)   \
                            & (data[after_col]<=after_val)  \
                            #& (data['count_hi']+data['count_lo']<=18)
    
    return data[mask]

def compareactual(rolls_df,bHighCheck):
    
    mask_actual=create_target_mask(rolls_df,bHighCheck)
    rolls_filt = rolls_df[mask_actual]
    return len(rolls_filt)


def save_test_stats(analysis_df, data,pattern,bHighCheck,is_prod=False):
    
    mask = (analysis_df['pattern_99']==pattern) & (analysis_df['target_high']==bHighCheck) 
    analysis_df_filt = analysis_df[mask]
    for row in analysis_df_filt.itertuples():
        digit = row.digit
        cutoff = row.cutoff
        after_val = row.afterval
        bHighCheck = row.target_high
        zero_col_cutoff = row.zero_col_cutoff
        
        if bHighCheck:
            cutoff_col = f'count_dig_{digit}'
            zero_col = 'count_dig_0'
            after_col = 'count_hi'
        else:
            cutoff_col = 'count_dig_0'
            zero_col = f'count_dig_{digit}'
            after_col = 'count_hi'
            
        mask = (data[cutoff_col]==cutoff) & (data[zero_col]==zero_col_cutoff)   \
                            & (data[after_col]<=after_val)  \
                            #& (data['count_hi']+data['count_lo']<=18)
    
        data_filt = data[mask]
        total = len(data_filt)
        analysis_df.at[row.Index,'te_total']  = total  
        
        if not is_prod:
            success  = compareactual(data_filt,bHighCheck)
            if total ==0:
                prob=0
            else:
                prob = round(100 * success/total,0)

            analysis_df.at[row.Index,'te_ratio']  = prob  
            analysis_df.at[row.Index,'te_success']  = success  
        
    return analysis_df

        

In [84]:

# bHighCheck = False
# for cur_digit in [9,8,7,5]:
#     for bHighCheck in [True,False]:
#         if bHighCheck:
#             high_low_text = 'HIGH'
#         else:
#             high_low_text = 'LOW'
#         print()
#         print(f'****** cur_digit {cur_digit} ******  {high_low_text} ')

#         for cutoff in range(0,7):
#             rolls_df = check_special_pattern_match(test_manual,bHighCheck,cutoff,25)
#             len_match  = compareactual(rolls_df,bHighCheck)
#             if len(rolls_df) ==0:
#                 success_count=0
#             else:
#                 success_count = round(100 * len_match/len(rolls_df),0)
#             print()
#             print(f'cutoff:{cutoff} General --- total:{len(rolls_df)} matched:{len_match} success:{success_count}')
#             for afterval in [8,10]:
#                 rolls_df = check_special_pattern_match(test_manual,bHighCheck,cutoff,afterval)
#                 len_match  = compareactual(rolls_df,bHighCheck)
#                 if len(rolls_df)==0:
#                     success_count=0
#                 else:
#                     success_count = round(100 * len_match/len(rolls_df),0)
#                 print(f'cutoff:{cutoff} afterval:{afterval} --- total:{len(rolls_df)} matched:{len_match} success:{success_count}')



In [85]:
# analysis_df_test = save_test_stats(temp_df, test_manual)
# analysis_df_test

In [86]:
# mask = (analysis_df_test['digit'].isin([1,2,3,4,6])) & (analysis_df_test['tr_ratio']>=13)
# mask2 = mask & (analysis_df_test['te_ratio']>=13)
# print(len(analysis_df_test[mask]))
# print(len(analysis_df_test[mask2]))
# analysis_df_test[mask2]

##### Feature Generation

In [87]:
def generate_roll_features_k(rolls_df,roll_start,roll_end,suffix="",stat_suffix=""):
    first_k_roll_cols = [f'roll_{i}{suffix}' for i in range(roll_start,roll_end+1)]
    rolls_df[f'count_gt_9000_k{stat_suffix}'] = (rolls_df[first_k_roll_cols] >= 9000).sum(axis=1)
    rolls_df[f'count_lt_1000_k{stat_suffix}'] = (rolls_df[first_k_roll_cols] < 1000).sum(axis=1)
    
    return rolls_df
    
def generate_roll_features(rolls_df,
                           roll_start=1, roll_end=20,k=4,
                          suffix="",stat_suffix=""):
    cols_roll = [f'roll_{i}{suffix}' for i in range(roll_start,roll_end+1)]
    print(cols_roll)
    
    if (roll_start==1) & (roll_end==20):
        stat_suffix = suffix
    else:
        stat_suffix = f'_{roll_start}_{roll_end}{suffix}'
    
    rolls_df[f'count_lt_1000{stat_suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==0).sum(axis=1)
    rolls_df[f'count_gt_9000{stat_suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==9).sum(axis=1)
    rolls_df[f'count_gt_8000{stat_suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==8).sum(axis=1)
    rolls_df[f'count_gt_7000{stat_suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==7).sum(axis=1)
    rolls_df[f'count_gt_6000{stat_suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==6).sum(axis=1)
    rolls_df[f'count_gt_5000{stat_suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==5).sum(axis=1)
    rolls_df[f'count_gt_4000{stat_suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==4).sum(axis=1)
    rolls_df[f'count_gt_3000{stat_suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==3).sum(axis=1)
    rolls_df[f'count_gt_2000{stat_suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==2).sum(axis=1)
    rolls_df[f'count_gt_1000{stat_suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==1).sum(axis=1)
    
    rolls_df[f'count_hi{stat_suffix}'] = (rolls_df[cols_roll] > 5250).sum(axis=1)
    rolls_df[f'count_lo{stat_suffix}'] = (rolls_df[cols_roll] < 4750).sum(axis=1)    
    rolls_df[f'count_lo_hi{stat_suffix}'] = rolls_df[f'count_hi{stat_suffix}'] \
                                            + rolls_df[f'count_lo{stat_suffix}']   


    rolls_df=generate_roll_features_k(rolls_df,roll_start,roll_start+k,
                                      suffix=suffix,stat_suffix=stat_suffix)
    
    roll_list_strs = np.full(len(rolls_df),"")
    for col in cols_roll:
        roll_list_strs = rolls_df[col].astype('str') + roll_list_strs
    
    rolls_df[f'total_dig_9{stat_suffix}']= roll_list_strs.str.count('9')
    rolls_df[f'total_dig_0{stat_suffix}']= roll_list_strs.str.count('0')
    
    rolls_df[f'roll_mean{stat_suffix}'] = (rolls_df[cols_roll]).mean(axis=1)
    rolls_df[f'roll_std{stat_suffix}'] = (rolls_df[cols_roll]).std(axis=1)
     
    print(f'Roll Features from {roll_start} to {roll_end} {suffix} completed' )
    
    return rolls_df

def generate_features_full(initial_df,istrain,feature_chain_length):
    
    rolls_df = initial_df['roll_hash_list'].apply(pd.Series)
    rolls_df.columns = [f'roll_{i}' for i in range(rolls_df.shape[1])]    
    rolls_df_nonce = initial_df['roll_hash_list_nonce'].apply(pd.Series)
    nonce_suffix = '_nonce'
    rolls_df_nonce.columns = [f'roll_{i}{nonce_suffix}' for i in range(rolls_df_nonce.shape[1])] 
    
    #generate top 20 hash features
    rolls_df=generate_roll_features(rolls_df,roll_start=1,
                                    roll_end=20, k=4)
    rolls_df_nonce=generate_roll_features(rolls_df_nonce,
                                    suffix=nonce_suffix,roll_start=1,
                                    roll_end=20 , k=4)
    
    #generate 25's hash features
    roll_start =25
    roll_end = 50
    rolls_df=generate_roll_features(rolls_df,
                                    roll_start=roll_start,
                                    roll_end=roll_end, k=5)
    rolls_df_nonce=generate_roll_features(rolls_df_nonce,
                                    suffix=nonce_suffix ,
                                    roll_start=roll_start,
                                    roll_end=roll_end, k=5)  

    #generate 50's hash features
    roll_start =50
    roll_end = 60
    rolls_df=generate_roll_features(rolls_df,
                                    roll_start=roll_start,
                                    roll_end=roll_end, k=5)
    rolls_df_nonce=generate_roll_features(rolls_df_nonce,
                                    suffix=nonce_suffix ,roll_start=roll_start,
                                    roll_end=roll_end, k=5)      
    #generate for full hash features
    roll_start =1
    roll_end = feature_chain_length
    rolls_df=generate_roll_features(rolls_df,
                                    roll_start=roll_start,
                                    roll_end=roll_end, k=5)
    rolls_df_nonce=generate_roll_features(rolls_df_nonce,
                                    suffix=nonce_suffix ,roll_start=roll_start,
                                    roll_end=roll_end, k=5)        
    if istrain:
        features_df = pd.concat([initial_df[['roll_actual','seed','hash','client_seed']],
                                 rolls_df,
                                 rolls_df_nonce],axis=1)
        mask = create_target_mask(features_df)
            
        features_df['target'] =0
        features_df.loc[mask,'target'] = 1
        features_df['roll_actual']=features_df['roll_actual'].astype('int')
        print(features_df['target'].value_counts())
    else:
        features_df = pd.concat([initial_df[['roll_actual','seed','hash','client_seed']],
                                             rolls_df,rolls_df_nonce],axis=1)
    return features_df


In [88]:
# def generate_roll_features(initial_df,colname,feature_chain_length,
#                            k=4,
#                           suffix=""):
#     rolls_df = initial_df[colname].apply(pd.Series)
#     rolls_df.columns = [f'roll_{i}{suffix}' for i in range(rolls_df.shape[1])]
    
#     # roll_list_strs = initial_df[colname].apply(lambda x: ', '.join([str(i) for i in x]))
    
#     cols_roll = [f'roll_{i}{suffix}' for i in range(1,feature_chain_length+1)]
#     print(cols_roll)
    
# #     rolls_df[f'count_lt_1000{suffix}'] = (rolls_df < 1000).sum(axis=1)
# #     rolls_df[f'count_gt_9000{suffix}'] = (rolls_df >= 9000).sum(axis=1)
# #     rolls_df[f'count_gt_5000{suffix}'] = (rolls_df >= 5000).sum(axis=1)
# #     rolls_df[f'count_gt_7000{suffix}'] = (rolls_df >= 7000).sum(axis=1)
# #     rolls_df[f'count_gt_8000{suffix}'] = (rolls_df >= 8000).sum(axis=1)
    
#     rolls_df[f'count_lt_1000{suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==0).sum(axis=1)
#     rolls_df[f'count_gt_9000{suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==9).sum(axis=1)
#     rolls_df[f'count_gt_8000{suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==8).sum(axis=1)
#     rolls_df[f'count_gt_7000{suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==7).sum(axis=1)
#     rolls_df[f'count_gt_6000{suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==6).sum(axis=1)
#     rolls_df[f'count_gt_5000{suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==5).sum(axis=1)
#     rolls_df[f'count_gt_4000{suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==4).sum(axis=1)
#     rolls_df[f'count_gt_3000{suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==3).sum(axis=1)
#     rolls_df[f'count_gt_2000{suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==2).sum(axis=1)
#     rolls_df[f'count_gt_1000{suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==1).sum(axis=1)
    
#     rolls_df[f'count_hi{suffix}'] = (rolls_df[cols_roll] > 5250).sum(axis=1)
#     rolls_df[f'count_lo{suffix}'] = (rolls_df[cols_roll] < 4750).sum(axis=1)    
#     rolls_df[f'count_lo_hi{suffix}'] = rolls_df[f'count_hi{suffix}'] \
#                                             + rolls_df[f'count_lo{suffix}']   

#     first_k_roll_cols = [f'roll_{i}{suffix}' for i in range(1,k+1)]
#     rolls_df[f'count_gt_9000_k{suffix}'] = (rolls_df[first_k_roll_cols] >= 9000).sum(axis=1)
    
#     roll_list_strs = initial_df[colname].apply(lambda x: ', '.join([f'{i:04d}' for i in x]))
    
#     rolls_df[f'total_dig_9{suffix}']= roll_list_strs.str.count('9')
#     rolls_df[f'total_dig_0{suffix}']= roll_list_strs.str.count('0')
    
#     rolls_df[f'roll_mean{suffix}'] = (rolls_df[cols_roll]).mean(axis=1)
#     rolls_df[f'roll_std{suffix}'] = (rolls_df[cols_roll]).std(axis=1)
   

# #     if (suffix==""):
# #         rolls_df[f'roll_0_1st_digit'] = (rolls_df['roll_0'] / 1000).astype('int')
# #         rolls_df[f'roll_0_last_digit'] = (rolls_df['roll_0'] % 10)
    
    
#     return rolls_df

# def generate_features_full(initial_df,istrain,feature_chain_length):
    
#     rolls_df=generate_roll_features(initial_df,'roll_hash_list',feature_chain_length,k=4)
#     rolls_df_nonce=generate_roll_features(initial_df,'roll_hash_list_nonce',
#                                     feature_chain_length,k=4,suffix='_nonce' )
    
#     if istrain:
#         features_df = pd.concat([initial_df[['roll_actual','seed','hash','client_seed']],
#                                  rolls_df,
#                                  rolls_df_nonce],axis=1)
#         mask = create_target_mask(features_df)
            
#         features_df['target'] =0
#         features_df.loc[mask,'target'] = 1
#         features_df['roll_actual']=features_df['roll_actual'].astype('int')
#         print(features_df['target'].value_counts())
#     else:
#         features_df = pd.concat([initial_df[['roll_actual','seed','hash','client_seed']],
#                                              rolls_df,rolls_df_nonce],axis=1)
#     return features_df


In [89]:
# HIGH_ANALYSIS=True

In [90]:
%%time
# train = generate_features_full(results_df[train_client_size:],True,feature_chain_length)
train = generate_features_full(results_df,True,feature_chain_length)
print(len(train))
train.head()

['roll_1', 'roll_2', 'roll_3', 'roll_4', 'roll_5', 'roll_6', 'roll_7', 'roll_8', 'roll_9', 'roll_10', 'roll_11', 'roll_12', 'roll_13', 'roll_14', 'roll_15', 'roll_16', 'roll_17', 'roll_18', 'roll_19', 'roll_20']
Roll Features from 1 to 20  completed
['roll_1_nonce', 'roll_2_nonce', 'roll_3_nonce', 'roll_4_nonce', 'roll_5_nonce', 'roll_6_nonce', 'roll_7_nonce', 'roll_8_nonce', 'roll_9_nonce', 'roll_10_nonce', 'roll_11_nonce', 'roll_12_nonce', 'roll_13_nonce', 'roll_14_nonce', 'roll_15_nonce', 'roll_16_nonce', 'roll_17_nonce', 'roll_18_nonce', 'roll_19_nonce', 'roll_20_nonce']
Roll Features from 1 to 20 _nonce completed
['roll_25', 'roll_26', 'roll_27', 'roll_28', 'roll_29', 'roll_30', 'roll_31', 'roll_32', 'roll_33', 'roll_34', 'roll_35', 'roll_36', 'roll_37', 'roll_38', 'roll_39', 'roll_40', 'roll_41', 'roll_42', 'roll_43', 'roll_44', 'roll_45', 'roll_46', 'roll_47', 'roll_48', 'roll_49', 'roll_50']
Roll Features from 25 to 50  completed
['roll_25_nonce', 'roll_26_nonce', 'roll_27_nonc

,roll_actual,seed,hash,client_seed,roll_0,roll_1,roll_2,roll_3,roll_4,roll_5,roll_6,roll_7,roll_8,roll_9,roll_10,roll_11,roll_12,roll_13,roll_14,roll_15,roll_16,roll_17,roll_18,roll_19,roll_20,roll_21,roll_22,roll_23,roll_24,roll_25,roll_26,roll_27,roll_28,roll_29,roll_30,roll_31,roll_32,roll_33,roll_34,roll_35,roll_36,roll_37,roll_38,roll_39,roll_40,roll_41,roll_42,roll_43,roll_44,roll_45,roll_46,roll_47,roll_48,roll_49,roll_50,roll_51,roll_52,roll_53,roll_54,roll_55,roll_56,roll_57,roll_58,roll_59,roll_60,count_lt_1000,count_gt_9000,count_gt_8000,count_gt_7000,count_gt_6000,count_gt_5000,count_gt_4000,count_gt_3000,count_gt_2000,count_gt_1000,count_hi,count_lo,count_lo_hi,count_gt_9000_k,count_lt_1000_k,total_dig_9,total_dig_0,roll_mean,roll_std,count_lt_1000_25_50,count_gt_9000_25_50,count_gt_8000_25_50,count_gt_7000_25_50,count_gt_6000_25_50,count_gt_5000_25_50,count_gt_4000_25_50,count_gt_3000_25_50,count_gt_2000_25_50,count_gt_1000_25_50,count_hi_25_50,count_lo_25_50,count_lo_hi_25_50,count_gt_9000_k_25_50,count_lt_1000_k_25_50,total_dig_9_25_50,total_dig_0_25_50,roll_mean_25_50,roll_std_25_50,count_lt_1000_50_60,count_gt_9000_50_60,count_gt_8000_50_60,count_gt_7000_50_60,count_gt_6000_50_60,count_gt_5000_50_60,count_gt_4000_50_60,count_gt_3000_50_60,count_gt_2000_50_60,count_gt_1000_50_60,count_hi_50_60,count_lo_50_60,count_lo_hi_50_60,count_gt_9000_k_50_60,count_lt_1000_k_50_60,total_dig_9_50_60,total_dig_0_50_60,roll_mean_50_60,roll_std_50_60,count_lt_1000_1_60,count_gt_9000_1_60,count_gt_8000_1_60,count_gt_7000_1_60,count_gt_6000_1_60,count_gt_5000_1_60,count_gt_4000_1_60,count_gt_3000_1_60,count_gt_2000_1_60,count_gt_1000_1_60,count_hi_1_60,count_lo_1_60,count_lo_hi_1_60,count_gt_9000_k_1_60,count_lt_1000_k_1_60,total_dig_9_1_60,total_dig_0_1_60,roll_mean_1_60,roll_std_1_60,roll_0_nonce,roll_1_nonce,roll_2_nonce,roll_3_nonce,roll_4_nonce,roll_5_nonce,roll_6_nonce,roll_7_nonce,roll_8_nonce,roll_9_nonce,roll_10_nonce,roll_11_nonce,roll_12_nonce,roll_13_nonce,roll_14_nonce,roll_15_nonce,roll_16_nonce,roll_17_nonce,roll_18_nonce,roll_19_nonce,roll_20_nonce,roll_21_nonce,roll_22_nonce,roll_23_nonce,roll_24_nonce,roll_25_nonce,roll_26_nonce,roll_27_nonce,roll_28_nonce,roll_29_nonce,roll_30_nonce,roll_31_nonce,roll_32_nonce,roll_33_nonce,roll_34_nonce,roll_35_nonce,roll_36_nonce,roll_37_nonce,roll_38_nonce,roll_39_nonce,roll_40_nonce,roll_41_nonce,roll_42_nonce,roll_43_nonce,roll_44_nonce,roll_45_nonce,roll_46_nonce,roll_47_nonce,roll_48_nonce,roll_49_nonce,roll_50_nonce,roll_51_nonce,roll_52_nonce,roll_53_nonce,roll_54_nonce,roll_55_nonce,roll_56_nonce,roll_57_nonce,roll_58_nonce,roll_59_nonce,roll_60_nonce,count_lt_1000_nonce,count_gt_9000_nonce,count_gt_8000_nonce,count_gt_7000_nonce,count_gt_6000_nonce,count_gt_5000_nonce,count_gt_4000_nonce,count_gt_3000_nonce,count_gt_2000_nonce,count_gt_1000_nonce,count_hi_nonce,count_lo_nonce,count_lo_hi_nonce,count_gt_9000_k_nonce,count_lt_1000_k_nonce,total_dig_9_nonce,total_dig_0_nonce,roll_mean_nonce,roll_std_nonce,count_lt_1000_25_50_nonce,count_gt_9000_25_50_nonce,count_gt_8000_25_50_nonce,count_gt_7000_25_50_nonce,count_gt_6000_25_50_nonce,count_gt_5000_25_50_nonce,count_gt_4000_25_50_nonce,count_gt_3000_25_50_nonce,count_gt_2000_25_50_nonce,count_gt_1000_25_50_nonce,count_hi_25_50_nonce,count_lo_25_50_nonce,count_lo_hi_25_50_nonce,count_gt_9000_k_25_50_nonce,count_lt_1000_k_25_50_nonce,total_dig_9_25_50_nonce,total_dig_0_25_50_nonce,roll_mean_25_50_nonce,roll_std_25_50_nonce,count_lt_1000_50_60_nonce,count_gt_9000_50_60_nonce,count_gt_8000_50_60_nonce,count_gt_7000_50_60_nonce,count_gt_6000_50_60_nonce,count_gt_5000_50_60_nonce,count_gt_4000_50_60_nonce,count_gt_3000_50_60_nonce,count_gt_2000_50_60_nonce,count_gt_1000_50_60_nonce,count_hi_50_60_nonce,count_lo_50_60_nonce,count_lo_hi_50_60_nonce,count_gt_9000_k_50_60_nonce,count_lt_1000_k_50_60_nonce,total_dig_9_50_60_nonce,total_dig_0_50_60_nonce,roll_mean_50_60_nonce,roll_std_50_60_nonce,count_lt_1000_1_60_nonce,count_gt_9

In [91]:
%%time
test = generate_features_full(results_df_test,False,feature_chain_length)
print(len(test))
test.head()

['roll_1', 'roll_2', 'roll_3', 'roll_4', 'roll_5', 'roll_6', 'roll_7', 'roll_8', 'roll_9', 'roll_10', 'roll_11', 'roll_12', 'roll_13', 'roll_14', 'roll_15', 'roll_16', 'roll_17', 'roll_18', 'roll_19', 'roll_20']
Roll Features from 1 to 20  completed
['roll_1_nonce', 'roll_2_nonce', 'roll_3_nonce', 'roll_4_nonce', 'roll_5_nonce', 'roll_6_nonce', 'roll_7_nonce', 'roll_8_nonce', 'roll_9_nonce', 'roll_10_nonce', 'roll_11_nonce', 'roll_12_nonce', 'roll_13_nonce', 'roll_14_nonce', 'roll_15_nonce', 'roll_16_nonce', 'roll_17_nonce', 'roll_18_nonce', 'roll_19_nonce', 'roll_20_nonce']
Roll Features from 1 to 20 _nonce completed
['roll_25', 'roll_26', 'roll_27', 'roll_28', 'roll_29', 'roll_30', 'roll_31', 'roll_32', 'roll_33', 'roll_34', 'roll_35', 'roll_36', 'roll_37', 'roll_38', 'roll_39', 'roll_40', 'roll_41', 'roll_42', 'roll_43', 'roll_44', 'roll_45', 'roll_46', 'roll_47', 'roll_48', 'roll_49', 'roll_50']
Roll Features from 25 to 50  completed
['roll_25_nonce', 'roll_26_nonce', 'roll_27_nonc

,roll_actual,seed,hash,client_seed,roll_0,roll_1,roll_2,roll_3,roll_4,roll_5,roll_6,roll_7,roll_8,roll_9,roll_10,roll_11,roll_12,roll_13,roll_14,roll_15,roll_16,roll_17,roll_18,roll_19,roll_20,roll_21,roll_22,roll_23,roll_24,roll_25,roll_26,roll_27,roll_28,roll_29,roll_30,roll_31,roll_32,roll_33,roll_34,roll_35,roll_36,roll_37,roll_38,roll_39,roll_40,roll_41,roll_42,roll_43,roll_44,roll_45,roll_46,roll_47,roll_48,roll_49,roll_50,roll_51,roll_52,roll_53,roll_54,roll_55,roll_56,roll_57,roll_58,roll_59,roll_60,count_lt_1000,count_gt_9000,count_gt_8000,count_gt_7000,count_gt_6000,count_gt_5000,count_gt_4000,count_gt_3000,count_gt_2000,count_gt_1000,count_hi,count_lo,count_lo_hi,count_gt_9000_k,count_lt_1000_k,total_dig_9,total_dig_0,roll_mean,roll_std,count_lt_1000_25_50,count_gt_9000_25_50,count_gt_8000_25_50,count_gt_7000_25_50,count_gt_6000_25_50,count_gt_5000_25_50,count_gt_4000_25_50,count_gt_3000_25_50,count_gt_2000_25_50,count_gt_1000_25_50,count_hi_25_50,count_lo_25_50,count_lo_hi_25_50,count_gt_9000_k_25_50,count_lt_1000_k_25_50,total_dig_9_25_50,total_dig_0_25_50,roll_mean_25_50,roll_std_25_50,count_lt_1000_50_60,count_gt_9000_50_60,count_gt_8000_50_60,count_gt_7000_50_60,count_gt_6000_50_60,count_gt_5000_50_60,count_gt_4000_50_60,count_gt_3000_50_60,count_gt_2000_50_60,count_gt_1000_50_60,count_hi_50_60,count_lo_50_60,count_lo_hi_50_60,count_gt_9000_k_50_60,count_lt_1000_k_50_60,total_dig_9_50_60,total_dig_0_50_60,roll_mean_50_60,roll_std_50_60,count_lt_1000_1_60,count_gt_9000_1_60,count_gt_8000_1_60,count_gt_7000_1_60,count_gt_6000_1_60,count_gt_5000_1_60,count_gt_4000_1_60,count_gt_3000_1_60,count_gt_2000_1_60,count_gt_1000_1_60,count_hi_1_60,count_lo_1_60,count_lo_hi_1_60,count_gt_9000_k_1_60,count_lt_1000_k_1_60,total_dig_9_1_60,total_dig_0_1_60,roll_mean_1_60,roll_std_1_60,roll_0_nonce,roll_1_nonce,roll_2_nonce,roll_3_nonce,roll_4_nonce,roll_5_nonce,roll_6_nonce,roll_7_nonce,roll_8_nonce,roll_9_nonce,roll_10_nonce,roll_11_nonce,roll_12_nonce,roll_13_nonce,roll_14_nonce,roll_15_nonce,roll_16_nonce,roll_17_nonce,roll_18_nonce,roll_19_nonce,roll_20_nonce,roll_21_nonce,roll_22_nonce,roll_23_nonce,roll_24_nonce,roll_25_nonce,roll_26_nonce,roll_27_nonce,roll_28_nonce,roll_29_nonce,roll_30_nonce,roll_31_nonce,roll_32_nonce,roll_33_nonce,roll_34_nonce,roll_35_nonce,roll_36_nonce,roll_37_nonce,roll_38_nonce,roll_39_nonce,roll_40_nonce,roll_41_nonce,roll_42_nonce,roll_43_nonce,roll_44_nonce,roll_45_nonce,roll_46_nonce,roll_47_nonce,roll_48_nonce,roll_49_nonce,roll_50_nonce,roll_51_nonce,roll_52_nonce,roll_53_nonce,roll_54_nonce,roll_55_nonce,roll_56_nonce,roll_57_nonce,roll_58_nonce,roll_59_nonce,roll_60_nonce,count_lt_1000_nonce,count_gt_9000_nonce,count_gt_8000_nonce,count_gt_7000_nonce,count_gt_6000_nonce,count_gt_5000_nonce,count_gt_4000_nonce,count_gt_3000_nonce,count_gt_2000_nonce,count_gt_1000_nonce,count_hi_nonce,count_lo_nonce,count_lo_hi_nonce,count_gt_9000_k_nonce,count_lt_1000_k_nonce,total_dig_9_nonce,total_dig_0_nonce,roll_mean_nonce,roll_std_nonce,count_lt_1000_25_50_nonce,count_gt_9000_25_50_nonce,count_gt_8000_25_50_nonce,count_gt_7000_25_50_nonce,count_gt_6000_25_50_nonce,count_gt_5000_25_50_nonce,count_gt_4000_25_50_nonce,count_gt_3000_25_50_nonce,count_gt_2000_25_50_nonce,count_gt_1000_25_50_nonce,count_hi_25_50_nonce,count_lo_25_50_nonce,count_lo_hi_25_50_nonce,count_gt_9000_k_25_50_nonce,count_lt_1000_k_25_50_nonce,total_dig_9_25_50_nonce,total_dig_0_25_50_nonce,roll_mean_25_50_nonce,roll_std_25_50_nonce,count_lt_1000_50_60_nonce,count_gt_9000_50_60_nonce,count_gt_8000_50_60_nonce,count_gt_7000_50_60_nonce,count_gt_6000_50_60_nonce,count_gt_5000_50_60_nonce,count_gt_4000_50_60_nonce,count_gt_3000_50_60_nonce,count_gt_2000_50_60_nonce,count_gt_1000_50_60_nonce,count_hi_50_60_nonce,count_lo_50_60_nonce,count_lo_hi_50_60_nonce,count_gt_9000_k_50_60_nonce,count_lt_1000_k_50_60_nonce,total_dig_9_50_60_nonce,total_dig_0_50_60_nonce,roll_mean_50_60_nonce,roll_std_50_60_nonce,count_lt_1000_1_60_nonce,count_gt_9

In [92]:
%%time
test_hash = generate_features_full(results_df_test_hash,False,feature_chain_length)
print(len(test))
test_hash.head()

['roll_1', 'roll_2', 'roll_3', 'roll_4', 'roll_5', 'roll_6', 'roll_7', 'roll_8', 'roll_9', 'roll_10', 'roll_11', 'roll_12', 'roll_13', 'roll_14', 'roll_15', 'roll_16', 'roll_17', 'roll_18', 'roll_19', 'roll_20']
Roll Features from 1 to 20  completed
['roll_1_nonce', 'roll_2_nonce', 'roll_3_nonce', 'roll_4_nonce', 'roll_5_nonce', 'roll_6_nonce', 'roll_7_nonce', 'roll_8_nonce', 'roll_9_nonce', 'roll_10_nonce', 'roll_11_nonce', 'roll_12_nonce', 'roll_13_nonce', 'roll_14_nonce', 'roll_15_nonce', 'roll_16_nonce', 'roll_17_nonce', 'roll_18_nonce', 'roll_19_nonce', 'roll_20_nonce']
Roll Features from 1 to 20 _nonce completed
['roll_25', 'roll_26', 'roll_27', 'roll_28', 'roll_29', 'roll_30', 'roll_31', 'roll_32', 'roll_33', 'roll_34', 'roll_35', 'roll_36', 'roll_37', 'roll_38', 'roll_39', 'roll_40', 'roll_41', 'roll_42', 'roll_43', 'roll_44', 'roll_45', 'roll_46', 'roll_47', 'roll_48', 'roll_49', 'roll_50']
Roll Features from 25 to 50  completed
['roll_25_nonce', 'roll_26_nonce', 'roll_27_nonc

,roll_actual,seed,hash,client_seed,roll_0,roll_1,roll_2,roll_3,roll_4,roll_5,roll_6,roll_7,roll_8,roll_9,roll_10,roll_11,roll_12,roll_13,roll_14,roll_15,roll_16,roll_17,roll_18,roll_19,roll_20,roll_21,roll_22,roll_23,roll_24,roll_25,roll_26,roll_27,roll_28,roll_29,roll_30,roll_31,roll_32,roll_33,roll_34,roll_35,roll_36,roll_37,roll_38,roll_39,roll_40,roll_41,roll_42,roll_43,roll_44,roll_45,roll_46,roll_47,roll_48,roll_49,roll_50,roll_51,roll_52,roll_53,roll_54,roll_55,roll_56,roll_57,roll_58,roll_59,roll_60,count_lt_1000,count_gt_9000,count_gt_8000,count_gt_7000,count_gt_6000,count_gt_5000,count_gt_4000,count_gt_3000,count_gt_2000,count_gt_1000,count_hi,count_lo,count_lo_hi,count_gt_9000_k,count_lt_1000_k,total_dig_9,total_dig_0,roll_mean,roll_std,count_lt_1000_25_50,count_gt_9000_25_50,count_gt_8000_25_50,count_gt_7000_25_50,count_gt_6000_25_50,count_gt_5000_25_50,count_gt_4000_25_50,count_gt_3000_25_50,count_gt_2000_25_50,count_gt_1000_25_50,count_hi_25_50,count_lo_25_50,count_lo_hi_25_50,count_gt_9000_k_25_50,count_lt_1000_k_25_50,total_dig_9_25_50,total_dig_0_25_50,roll_mean_25_50,roll_std_25_50,count_lt_1000_50_60,count_gt_9000_50_60,count_gt_8000_50_60,count_gt_7000_50_60,count_gt_6000_50_60,count_gt_5000_50_60,count_gt_4000_50_60,count_gt_3000_50_60,count_gt_2000_50_60,count_gt_1000_50_60,count_hi_50_60,count_lo_50_60,count_lo_hi_50_60,count_gt_9000_k_50_60,count_lt_1000_k_50_60,total_dig_9_50_60,total_dig_0_50_60,roll_mean_50_60,roll_std_50_60,count_lt_1000_1_60,count_gt_9000_1_60,count_gt_8000_1_60,count_gt_7000_1_60,count_gt_6000_1_60,count_gt_5000_1_60,count_gt_4000_1_60,count_gt_3000_1_60,count_gt_2000_1_60,count_gt_1000_1_60,count_hi_1_60,count_lo_1_60,count_lo_hi_1_60,count_gt_9000_k_1_60,count_lt_1000_k_1_60,total_dig_9_1_60,total_dig_0_1_60,roll_mean_1_60,roll_std_1_60,roll_0_nonce,roll_1_nonce,roll_2_nonce,roll_3_nonce,roll_4_nonce,roll_5_nonce,roll_6_nonce,roll_7_nonce,roll_8_nonce,roll_9_nonce,roll_10_nonce,roll_11_nonce,roll_12_nonce,roll_13_nonce,roll_14_nonce,roll_15_nonce,roll_16_nonce,roll_17_nonce,roll_18_nonce,roll_19_nonce,roll_20_nonce,roll_21_nonce,roll_22_nonce,roll_23_nonce,roll_24_nonce,roll_25_nonce,roll_26_nonce,roll_27_nonce,roll_28_nonce,roll_29_nonce,roll_30_nonce,roll_31_nonce,roll_32_nonce,roll_33_nonce,roll_34_nonce,roll_35_nonce,roll_36_nonce,roll_37_nonce,roll_38_nonce,roll_39_nonce,roll_40_nonce,roll_41_nonce,roll_42_nonce,roll_43_nonce,roll_44_nonce,roll_45_nonce,roll_46_nonce,roll_47_nonce,roll_48_nonce,roll_49_nonce,roll_50_nonce,roll_51_nonce,roll_52_nonce,roll_53_nonce,roll_54_nonce,roll_55_nonce,roll_56_nonce,roll_57_nonce,roll_58_nonce,roll_59_nonce,roll_60_nonce,count_lt_1000_nonce,count_gt_9000_nonce,count_gt_8000_nonce,count_gt_7000_nonce,count_gt_6000_nonce,count_gt_5000_nonce,count_gt_4000_nonce,count_gt_3000_nonce,count_gt_2000_nonce,count_gt_1000_nonce,count_hi_nonce,count_lo_nonce,count_lo_hi_nonce,count_gt_9000_k_nonce,count_lt_1000_k_nonce,total_dig_9_nonce,total_dig_0_nonce,roll_mean_nonce,roll_std_nonce,count_lt_1000_25_50_nonce,count_gt_9000_25_50_nonce,count_gt_8000_25_50_nonce,count_gt_7000_25_50_nonce,count_gt_6000_25_50_nonce,count_gt_5000_25_50_nonce,count_gt_4000_25_50_nonce,count_gt_3000_25_50_nonce,count_gt_2000_25_50_nonce,count_gt_1000_25_50_nonce,count_hi_25_50_nonce,count_lo_25_50_nonce,count_lo_hi_25_50_nonce,count_gt_9000_k_25_50_nonce,count_lt_1000_k_25_50_nonce,total_dig_9_25_50_nonce,total_dig_0_25_50_nonce,roll_mean_25_50_nonce,roll_std_25_50_nonce,count_lt_1000_50_60_nonce,count_gt_9000_50_60_nonce,count_gt_8000_50_60_nonce,count_gt_7000_50_60_nonce,count_gt_6000_50_60_nonce,count_gt_5000_50_60_nonce,count_gt_4000_50_60_nonce,count_gt_3000_50_60_nonce,count_gt_2000_50_60_nonce,count_gt_1000_50_60_nonce,count_hi_50_60_nonce,count_lo_50_60_nonce,count_lo_hi_50_60_nonce,count_gt_9000_k_50_60_nonce,count_lt_1000_k_50_60_nonce,total_dig_9_50_60_nonce,total_dig_0_50_60_nonce,roll_mean_50_60_nonce,roll_std_50_60_nonce,count_lt_1000_1_60_nonce,count_gt_9

#### Machine Learning

In [93]:
USE_STAT_FEATS = True

In [94]:

if USE_STAT_FEATS:
    exclude_stat_cols=[]
else:
#     exclude_stat_cols = ['roll_mean','roll_std','roll_mean_nonce','roll_std_nonce']
    exclude_stat_col_starts = ['roll_mean','roll_std','roll_mean_nonce','roll_std_nonce']
    exclude_stat_cols=[]
    for stat_col in exclude_stat_col_starts:
        exclude_stat_cols += [col for col in train.columns if col.startswith(stat_col)]
    print(exclude_stat_cols)
exclude_cols = []#[f'roll_{i}' for i in range(11,feature_chain_length)]
exclude_cols_nonce =[]# [f'roll_{i}_nonce' for i in range(11,feature_chain_length+1)]
exclude_count_cols = [f'count_gt_{i}000' for i in [1,2,3,4,6]]
exclude_count_cols_nonce = [f'count_gt_{i}000_nonce' for i in [1,2,3,4,6]]
othercols = ['roll_actual','client_seed','seed','hash'] + exclude_stat_cols

all_exclude_cols = othercols+exclude_cols+exclude_cols_nonce+\
                    exclude_count_cols+exclude_count_cols_nonce
                  
features = [col for col in train.columns if col not in all_exclude_cols]
print(len(features))
print(features)

265
['roll_0', 'roll_1', 'roll_2', 'roll_3', 'roll_4', 'roll_5', 'roll_6', 'roll_7', 'roll_8', 'roll_9', 'roll_10', 'roll_11', 'roll_12', 'roll_13', 'roll_14', 'roll_15', 'roll_16', 'roll_17', 'roll_18', 'roll_19', 'roll_20', 'roll_21', 'roll_22', 'roll_23', 'roll_24', 'roll_25', 'roll_26', 'roll_27', 'roll_28', 'roll_29', 'roll_30', 'roll_31', 'roll_32', 'roll_33', 'roll_34', 'roll_35', 'roll_36', 'roll_37', 'roll_38', 'roll_39', 'roll_40', 'roll_41', 'roll_42', 'roll_43', 'roll_44', 'roll_45', 'roll_46', 'roll_47', 'roll_48', 'roll_49', 'roll_50', 'roll_51', 'roll_52', 'roll_53', 'roll_54', 'roll_55', 'roll_56', 'roll_57', 'roll_58', 'roll_59', 'roll_60', 'count_lt_1000', 'count_gt_9000', 'count_gt_8000', 'count_gt_7000', 'count_gt_5000', 'count_hi', 'count_lo', 'count_lo_hi', 'count_gt_9000_k', 'count_lt_1000_k', 'total_dig_9', 'total_dig_0', 'roll_mean', 'roll_std', 'count_lt_1000_25_50', 'count_gt_9000_25_50', 'count_gt_8000_25_50', 'count_gt_7000_25_50', 'count_gt_6000_25_50', 'c

In [95]:
X = train[features].drop('target',axis=1)
y = train['target']

In [96]:
X.columns

Index(['roll_0', 'roll_1', 'roll_2', 'roll_3', 'roll_4', 'roll_5', 'roll_6',
       'roll_7', 'roll_8', 'roll_9',
       ...
       'count_gt_1000_1_60_nonce', 'count_hi_1_60_nonce',
       'count_lo_1_60_nonce', 'count_lo_hi_1_60_nonce',
       'count_gt_9000_k_1_60_nonce', 'count_lt_1000_k_1_60_nonce',
       'total_dig_9_1_60_nonce', 'total_dig_0_1_60_nonce',
       'roll_mean_1_60_nonce', 'roll_std_1_60_nonce'],
      dtype='object', length=264)

In [97]:
# results_df#[['nonce','roll_actual','roll_hash_list','client_seed','client_index']]

In [98]:
import xgboost as xgb
import matplotlib.pyplot as plt # for plotting graphs
import seaborn as sns # for plotting graphs
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, f1_score,precision_recall_curve

Train Validation Split

In [99]:
n_folds=10
tr_indices_folds = []
val_indices_folds=[]
tr_indices_seeds=[]
for fold in range(n_folds):
#     tr_indices = train[(fold+2) * train_client_size:].index
#     val_indices = train[(fold) * train_client_size:(fold+1) * train_client_size].index
    tr_offset = 25#28
    tr_length = 25#25
    tr_start = (fold+tr_offset) * train_client_size
    tr_end = (fold+tr_offset+tr_length) * train_client_size
    tr_indices = train[tr_start:tr_end].index #fold + 3
#     tr_indices = list(tr_indices) + list(train[70:90].index) #fold + 3
    
    tr_indices_seed=[]
    for i in range(1,tr_offset):
        tr_start_seed = (fold+i) * train_client_size
        tr_end_seed = (fold+i+1) * train_client_size
        tr_indices_seed.append(train[tr_start_seed:tr_end_seed].index) #fold + 3 
        
    # tr_start_seed = (fold+1) * train_client_size
    # tr_end_seed = (fold+tr_offset) * train_client_size
    # tr_indices_seed = train[tr_start_seed:tr_end_seed].index #fold + 3
    
    val_indices = train[(fold+1) * train_client_size:(fold+2) * train_client_size].index
#     val_indices = train[(fold) * train_client_size:(fold+1) * train_client_size].index
    tr_indices_seeds.append(tr_indices_seed)
    tr_indices_folds.append(tr_indices)
    val_indices_folds.append(val_indices)
    

In [100]:
#This function is useful to convert the predicted probabilities into labels 
#so that F1 score is optimized
#It first determines the probability threshold using precision recall curve at which F1 score is optimized and 
#then generate the optimized labels based on the determined probability threshold
def get_opt_cutoff_prec(labels,preds):
    precision, recall, thresholds  = precision_recall_curve(labels, preds)
    f1_score= 2*((precision*recall)/(precision+recall))
#     optimal_idx = np.nanargmax(precision[:len(precision)-1])
#     print(f'precision borders:{precision[0],precision[len(precision)-2]}')
#     print(f'recall borders:{recall[0],recall[len(recall)-2]}')
#     print(f'f1_score size:{len(f1_score)} recall size:{len(recall)}')
#     print(f'optimal_idx:{optimal_idx} precision size:{len(precision)} thresholds size:{len(thresholds)}')
    optimal_idx = np.nanargmax(f1_score)
    optimal_threshold = thresholds[optimal_idx]
#     return optimal_threshold, precision[optimal_idx]
    return optimal_threshold, f1_score[optimal_idx]

def convert_probtolabels(preds,cutoff=0.5):
    y_bin= preds.copy()
    y_bin[preds>cutoff] = 1
    y_bin[preds<=cutoff] = 0
    y_bin=y_bin.astype(int)

    return y_bin

In [101]:
ZERO_COL_CUTOFF=4

In [102]:
def train_status(data,bHighCheck):
    
    after_vals = [8,10]
    if bHighCheck:
        cutoff_col = 'count_gt_9000'
        zero_col = 'count_lt_1000'
        after_col = 'count_hi'
        cutoff_range = range(2+2,7+2)
        
    else:
        cutoff_col = 'total_dig_0'
        zero_col = 'total_dig_9'
        after_col = 'count_hi'
        cutoff_range = range(2,7)

    mask_actual=create_target_mask(data,bHighCheck)    
    for cutoff in cutoff_range:
        
        print(f'********** cutoff:{cutoff}')
        mask = (data[cutoff_col]==cutoff) & (data[zero_col]==ZERO_COL_CUTOFF) 
        total = len(data[mask])
        # print(data[mask])
        if total ==0:
            success_count=0
            prob=0
        else:
            success_count = len(data[mask & mask_actual])
            prob =round(100 * success_count/ total ,0)
        print(f'before values - probability: {prob} total : {total}  success: {success_count}')
        for val in after_vals:
            mask2 = mask   & (data[after_col]<=val) #& (data['count_hi']+train['count_lo']<=18)
            total = len(data[mask2])
            if total ==0:
                success_count=0
                prob=0
            else:
                success_count = len(data[mask2 & mask_actual])
                prob =round(100 * success_count/ total ,0)
            print(f'AFTER - factor:{val} - probability: {prob} total : {total}  success: {success_count}')


In [103]:
def validate_pattern(initial_df,bHighCheck,zero_col_cutoff,digit,cutoff,after_val):
     
    # rolls_df = initial_df['roll_hash_list'].apply(pd.Series)
    # rolls_df.columns = [f'roll_{i}' for i in range(rolls_df.shape[1])]
    
    # cols_roll = [col for col in rolls_df.columns if col.startswith('roll_') and col not in ['roll_0']]
    rolls_df= initial_df
    
    if bHighCheck:
        cutoff_col = f'count_gt_{digit}000'
        zero_col = 'count_lt_1000'
        after_col = 'count_hi'
    else:
        cutoff_col = 'count_lt_1000'
        zero_col = f'count_gt_{digit}000'
        after_col = 'count_hi'

    
    # rolls_df['client_seed'] = initial_df['client_seed']
    # rolls_df['client_index'] = initial_df['client_index']

    
    mask = (rolls_df[cutoff_col]==cutoff) \
                            & (rolls_df[after_col]<=after_val)  \
                            & (rolls_df[zero_col]==zero_col_cutoff)   \
                            #& (rolls_df['count_hi']+rolls_df['count_lo']<=18)
    
    return rolls_df[mask]
    

In [104]:
# val_res_df = validate_pattern(X_val,True,4,10)
# print(y_val[val_res_df.index])
# print(len(val_res_df))
# val_res_df

In [105]:
# probs= xgb_model.predict_proba(val_res_df)[:,1]
# print(probs)
# np.where(probs>0.3)

In [106]:
# # roll_actual_df = pd.DataFrame(columns=['roll_actual'])
# # roll_actual_df['roll_actual']=train['roll_actual'].iloc[tr_index]
# # print(len(X_tr))
# # print(len(roll_actual_df))

# oversample = RandomOverSampler(sampling_strategy='minority',random_state=123)
# # fit and apply the transform
# roll_actual_df, y_temp = oversample.fit_resample(train[features+['roll_actual']].iloc[tr_index], 
#                                                  train['target'].iloc[tr_index])
# print(len(roll_actual_df))

# # tr_df = pd.concat([X_tr,roll_actual_df],axis=1)
# # print(len(tr_df))
# train_status(roll_actual_df,True)

In [107]:
%%time
READ_MODEL_FILE = False

if READ_MODEL_FILE:
    xgb_models = joblib.load(f'data/models/models_{nonce}_{file_pattern_str}_pattern.dump')
else:

    params = { 'n_estimators':100,
              'max_leaves':25,
                'subsample':0.8,
              'random_state':145,
              # 'scale_pos_weight': 5,
    #           'max_depth':6,
            'learning_rate':0.05,
             'colsample_bytree':0.6,#0.85,
             'lambda':0.05,
             'alpha':0.1}

    # xgb_model = xgb.XGBClassifier(**params)
    xgb_models = []
    scores  = []
    ratios =[]
    tr_cutoffs=[]
    val_cutoffs=[]
    tr_last_cutoffs=[]

    for fold,(tr_index, val_index, tr_index_seed) in enumerate(zip(tr_indices_folds,val_indices_folds,tr_indices_seeds)):
        X_tr,y_tr = X.iloc[tr_index],y.iloc[tr_index]
        X_val,y_val = X.iloc[val_index],y.iloc[val_index]

        oversample = RandomOverSampler(sampling_strategy='minority',random_state=123)
        # fit and apply the transform
        X_tr, y_tr = oversample.fit_resample(X_tr, y_tr)
        print(pd.Series(y_tr).value_counts())


        print(f'\n ******************* fold: {fold} ********')
        print(f'tr size: { len(X_tr)}  val size:  {len(X_val)}')

        xgb_model = xgb.XGBClassifier(**params)
        xgb_model.fit(X_tr,y_tr)
        xgb_models.append(xgb_model)

        mean_opt_cutoff_tr = 0
        for tr_index_seed_cur in tr_index_seed:
            X_tr_seed,y_tr_seed = X.iloc[tr_index_seed_cur],y.iloc[tr_index_seed_cur]

            tr_probs = xgb_model.predict_proba(X_tr_seed)[:,1]
            opt_cutoff_tr, f1score = get_opt_cutoff_prec(y_tr_seed,tr_probs)
            # print('tr cutoff:',opt_cutoff_tr)
            tr_labels = convert_probtolabels(tr_probs,cutoff=opt_cutoff_tr) 
            mask1 = (tr_labels==1)
            mask2 = (y_tr_seed==1)
            total = len(tr_labels[mask1])
            success = len(tr_labels[mask1 & mask2])
            ratio = success/total if total!=0 else 0
            # print(f'tr ratio: {ratio} total: {total} success: {success}')
            mean_opt_cutoff_tr += opt_cutoff_tr / len(tr_index_seed)

        print(f'last cutoff :{opt_cutoff_tr}')
        print(f'mean tr cutoff :{mean_opt_cutoff_tr}')
        tr_last_cutoffs.append(opt_cutoff_tr)
        tr_cutoffs.append(mean_opt_cutoff_tr)
        val_probs = xgb_model.predict_proba(X_val)[:,1]
        print(val_probs[0:5])
        labels = convert_probtolabels(val_probs)
        score_init = accuracy_score(y_val,labels)

        opt_cutoff, f1score = get_opt_cutoff_prec(y_val,val_probs)
        print('valid cutoff:',opt_cutoff)
        val_cutoffs.append(opt_cutoff)
        # print(pd.Series(val_probs).describe(percentiles=[0.6,0.7,0.75,0.8]))
        val_labels = convert_probtolabels(val_probs,cutoff=opt_cutoff_tr) 
        mask1 = (val_labels==1)
        mask2 = (y_val==1)
        total = len(val_labels[mask1])
        success = len(val_labels[mask1 & mask2])
        ratio = success/total if total!=0 else 0
        print(f'ratio: {ratio} total: {total} success: {success}')
        ratios.append(ratio)
        cur_f1 = f1_score(y_val, val_labels,average='macro')
        cur_acc = accuracy_score(y_val,val_labels)
        print(f'val accuracy score:{cur_acc} f1 score:{cur_f1:.4f} initial accuracy score:{score_init}')
        scores.append(cur_acc)

        val_labels = convert_probtolabels(val_probs,cutoff=opt_cutoff) 
        mask1 = (val_labels==1)
        mask2 = (y_val==1)
        total = len(val_labels[mask1])
        success = len(val_labels[mask1 & mask2])
        ratio = success/total if total!=0 else 0
        print(f'ratio @val_cutoff: {ratio} total: {total} success: {success}')

    #     train_status(X_tr,True)

    #     top_prob = np.sort(val_probs)[::-1][:1]
    #     top_label = y_val[val_probs==top_prob ]
    #     print(f'top_prob:{top_prob} top_label:{top_label}')

    print(f'mean score:{np.mean(scores)}')    
    print(f'mean ratio:{np.mean(ratios)}')   
    
    print(np.mean(np.array(val_cutoffs)))
    print(np.mean(np.array(tr_cutoffs)))
    print(np.mean(np.array(tr_last_cutoffs)))
    print(ratios)
    
    joblib.dump(xgb_models,f'data/models/models_{nonce}_{file_pattern_str}_pattern.dump')
    print('Models Save completed')   

0    45015
1    45015
Name: target, dtype: int64

 ******************* fold: 0 ********
tr size: 90030  val size:  2000
[09:23:19] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1645117766796/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
last cutoff :0.27505773305892944
mean tr cutoff :0.35395907859007525
[0.435612   0.43872786 0.44870028 0.42588866 0.3887988 ]
valid cutoff: 0.3887988
ratio: 0.09727822580645161 total: 1984 success: 193
val accuracy score:0.104 f1 score:0.0968 initial accuracy score:0.714
ratio @val_cutoff: 0.1 total: 1760 success: 176
0    45053
1    45053
Name: target, dtype: int64

 ******************* fold: 1 ********
tr size: 90106  val size:  2000
[09:23:44] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1645117766796/work/src/learner.cc:1115

##### Prediction of Test 

In [108]:
# results_df[:train_client_size].head()

In [109]:
# results_df_test.head()

In [110]:
# def save_test_ml_probs(analysis_df,test,model,prob_tr_cutoff,prob_val_cutoff,
#                       multi_models=False):
#     features_test = [col for col in features if col!='target']
#     for row in analysis_df.itertuples():
#         digit = row.digit
#         cutoff = row.cutoff
#         after_val = row.afterval
#         bHighCheck = row.target_high
#         zero_col_cutoff = row.zero_col_cutoff

#         test_filt_df = validate_pattern(test,bHighCheck,zero_col_cutoff,
#                                         digit,cutoff,after_val)
#         filt_size = len(test_filt_df)
#         if filt_size==0:
#             print(f'No Record for zero_col_cutoff:{zero_col_cutoff} digit:{digit} cutoff:{cutoff} after_val:{after_val}')
#             continue
# #         print(f'validate pattern return size: {filt_size}')
#         if multi_models:
#             probs= np.zeros(len(test_filt_df))
#             for model_ind in model:
#                 probs+= model_ind.predict_proba(test_filt_df[features_test])[:,1] / len(model)
#         else: 
#             probs= model.predict_proba(test_filt_df[features_test])[:,1]
#         prob_cutoffs = [np.mean(probs),np.median(probs),0.1,0.2,0.3,0.4, 0.5,prob_tr_cutoff,prob_val_cutoff]
# #         prob_cutoffs = [np.mean(probs),np.median(probs),0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4, 0.45,0.5]
#         ratios = np.zeros(len(prob_cutoffs))
#         total = np.zeros(len(prob_cutoffs))
#         for i,prob_cutoff in enumerate(prob_cutoffs):
#             indices= np.where(probs>prob_cutoff)[0]
#             roll_filt = test['roll_actual'].loc[test_filt_df.iloc[indices].index]
#             total[i] = len(roll_filt)
#             if HIGH_ANALYSIS:
#                 mask = roll_filt>=HIGH_TARGET
#                 mask_arr =roll_filt.values>=HIGH_TARGET
#             else:
#                 mask=roll_filt<LOW_TARGET
#                 mask_arr =roll_filt.values<LOW_TARGET
#             success = len(roll_filt[mask])
#             if total[i]==0:
#                 ratios[i]=0
#             else:
#                 ratios[i] = 100 * success / total[i]
            
#             if (i >= 2) & (i<=6):
#                 analysis_df.at[row.Index,f'pr_{i-1}']  = ratios[i] 
#                 analysis_df.at[row.Index,f'te_pr_{i-1}_count']  = total[i] 
                
# #             print(f'prob_cutoff: {prob_cutoff} ratio:{ratio} total:{total} success:{success}')
            
#         analysis_df.at[row.Index,'pr_mean']  = ratios[0]  
#         analysis_df.at[row.Index,'pr_med']  = ratios[1] 
#         analysis_df.at[row.Index,'pr_tr_cutoff']  = ratios[7]  
#         analysis_df.at[row.Index,'pr_val_cutoff']  = ratios[8]         
#     return analysis_df


In [111]:
# def save_test_ml_probs_internal(analysis_df_filt,
# #                                 features_test,
# #                                 test,model,
#                                 prob_tr_cutoff,prob_val_cutoff,
#                                 pattern,bHighCheck,multi_models):
    
#     start = time.time()
#     #get data from shared memory
#     features_test = p.value[0]
#     test = p.value[1]
#     model = p.value[2]
# #     print(len(test))
# #     print(test.head())
#     end = time.time()
#     print(f'Read Shared memory time:{end-start}')
    
#     start = time.time()
    
#     for row in analysis_df_filt.itertuples():
#         digit = row.digit
#         cutoff = row.cutoff
#         after_val = row.afterval
#         bHighCheck = row.target_high
#         zero_col_cutoff = row.zero_col_cutoff

#         test_filt_df = validate_pattern(test,bHighCheck,zero_col_cutoff,
#                                         digit,cutoff,after_val)
#         filt_size = len(test_filt_df)
#         if filt_size==0:
#             print(f'No Record for zero_col_cutoff:{zero_col_cutoff} digit:{digit} cutoff:{cutoff} after_val:{after_val}')
#             continue
# #         print(f'validate pattern return size: {filt_size}')
#         if multi_models:
#             probs= np.zeros(len(test_filt_df))
#             for i,model_ind in enumerate(model):
#                 cur_prob= model_ind.predict_proba(test_filt_df[features_test])[:,1] 
#                 probs+= cur_prob / len(model)
#             prob_cutoffs = [np.mean(probs),np.median(probs),0.1,0.2,0.3,0.4,0.5,prob_tr_cutoff,prob_val_cutoff]
# #                 probs+= convert_probtolabels(cur_prob,cutoff=val_cutoffs[i]) / len(model)
# #             prob_cutoffs = [np.mean(probs),np.median(probs),0.4,0.5,0.6,0.7,0.8,prob_tr_cutoff,prob_val_cutoff]
#         else: 
#             probs= model.predict_proba(test_filt_df[features_test])[:,1]
#             prob_cutoffs = [np.mean(probs),np.median(probs),0.1,0.2,0.3,0.4,0.5,prob_tr_cutoff,prob_val_cutoff]
# #         prob_cutoffs = [np.mean(probs),np.median(probs),0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4, 0.45,0.5]
#         ratios = np.zeros(len(prob_cutoffs))
#         total = np.zeros(len(prob_cutoffs))
#         for i,prob_cutoff in enumerate(prob_cutoffs):
#             indices= np.where(probs>prob_cutoff)[0]
#             roll_filt = test['roll_actual'].loc[test_filt_df.iloc[indices].index]
#             total[i] = len(roll_filt)
#             if HIGH_ANALYSIS:
#                 mask = roll_filt>=HIGH_TARGET
#                 mask_arr =roll_filt.values>=HIGH_TARGET
#             else:
#                 mask=roll_filt<LOW_TARGET
#                 mask_arr =roll_filt.values<LOW_TARGET
#             success = len(roll_filt[mask])
#             if total[i]==0:
#                 ratios[i]=0
#             else:
#                 ratios[i] = 100 * success / total[i]
            
#             if (i >= 2) & (i<=6):
#                 analysis_df_filt.at[row.Index,f'pr_{i-1}']  = ratios[i] 
#                 analysis_df_filt.at[row.Index,f'te_pr_{i-1}_count']  = total[i] 
                
# #             print(f'prob_cutoff: {prob_cutoff} ratio:{ratio} total:{total} success:{success}')
            
#         analysis_df_filt.at[row.Index,'pr_mean']  = ratios[0]  
#         analysis_df_filt.at[row.Index,'pr_med']  = ratios[1] 
#         analysis_df_filt.at[row.Index,'pr_tr_cutoff']  = ratios[7]  
#         analysis_df_filt.at[row.Index,'pr_val_cutoff']  = ratios[8]       

#     end = time.time()
#     print(f'Internal function time:{end-start}')
#     return analysis_df_filt  

# def save_test_ml_probs(analysis_df,test,model,prob_tr_cutoff,prob_val_cutoff,pattern,bHighCheck,
#                       multi_models=False):
#     features_test = [col for col in features if col!='target']
#     mask = (analysis_df['pattern_99']==pattern) & (analysis_df['target_high']==bHighCheck) 
#     analysis_df_filt = analysis_df[mask]
#     other_args=[#features_test,test,model,
#                 prob_tr_cutoff,prob_val_cutoff,pattern,bHighCheck,multi_models]
#     analysis_df_filt=parallelize_dataframe(analysis_df_filt, save_test_ml_probs_internal, 
#                                            data_to_share=[features_test,test,model],
#                                            other_args=other_args, n_cores= multiprocessing.cpu_count())
#     if len(analysis_df_filt)==len(analysis_df):
#         analysis_df=analysis_df_filt
#     else:
#         analysis_df=pd.concat([analysis_df[~mask],analysis_filt],axis=0)
#     return analysis_df

In [112]:
# temp_df = pd.DataFrame()
# temp_df['one'] = np.array([3,42,20,8,19,54,72,15])
# temp_df['two'] = np.array([0,1,3,1,1,2,2,3])

# temp_df_filt = temp_df[temp_df['one']>=42]
# for row in temp_df_filt.itertuples():
#     print(row.Index)
# #     row.loc[row.Index,'two']=5
#     temp_df.loc[row.Index,'two']=5
# print(temp_df_filt.head())    
# temp_df

In [113]:
# def save_test_ml_probs_old(analysis_df,test,model,prob_tr_cutoff,prob_val_cutoff,pattern,bHighCheck,
#                       multi_models=False):
#     features_test = [col for col in features if col!='target']
#     mask = (analysis_df['pattern_99']==pattern) & (analysis_df['target_high']==bHighCheck) 
#     analysis_df_filt = analysis_df[mask]
    
#     start = time.time()
    
#     for row in analysis_df_filt.itertuples():
#         digit = row.digit
#         cutoff = row.cutoff
#         after_val = row.afterval
#         bHighCheck = row.target_high
#         zero_col_cutoff = row.zero_col_cutoff

#         test_filt_df = validate_pattern(test,bHighCheck,zero_col_cutoff,
#                                         digit,cutoff,after_val)
#         filt_size = len(test_filt_df)
#         if filt_size==0:
#             print(f'No Record for zero_col_cutoff:{zero_col_cutoff} digit:{digit} cutoff:{cutoff} after_val:{after_val}')
#             continue
# #         print(f'validate pattern return size: {filt_size}')
#         if multi_models:
#             probs= np.zeros(len(test_filt_df))
#             for i,model_ind in enumerate(model):
#                 cur_prob= model_ind.predict_proba(test_filt_df[features_test])[:,1] 
#                 probs+= cur_prob / len(model)
#             prob_cutoffs = [np.mean(probs),np.median(probs),0.1,0.2,0.3,0.4,0.5,prob_tr_cutoff,prob_val_cutoff]
# #                 probs+= convert_probtolabels(cur_prob,cutoff=val_cutoffs[i]) / len(model)
# #             prob_cutoffs = [np.mean(probs),np.median(probs),0.4,0.5,0.6,0.7,0.8,prob_tr_cutoff,prob_val_cutoff]
#         else: 
#             probs= model.predict_proba(test_filt_df[features_test])[:,1]
#             prob_cutoffs = [np.mean(probs),np.median(probs),0.1,0.2,0.3,0.4,0.5,prob_tr_cutoff,prob_val_cutoff]
# #         prob_cutoffs = [np.mean(probs),np.median(probs),0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4, 0.45,0.5]
#         ratios = np.zeros(len(prob_cutoffs))
#         total = np.zeros(len(prob_cutoffs))
#         for i,prob_cutoff in enumerate(prob_cutoffs):
#             indices= np.where(probs>prob_cutoff)[0]
#             roll_filt = test['roll_actual'].loc[test_filt_df.iloc[indices].index]
#             total[i] = len(roll_filt)
#             if HIGH_ANALYSIS:
#                 mask = roll_filt>=HIGH_TARGET
#                 mask_arr =roll_filt.values>=HIGH_TARGET
#             else:
#                 mask=roll_filt<LOW_TARGET
#                 mask_arr =roll_filt.values<LOW_TARGET
#             success = len(roll_filt[mask])
#             if total[i]==0:
#                 ratios[i]=0
#             else:
#                 ratios[i] = 100 * success / total[i]
            
#             if (i >= 2) & (i<=6):
#                 analysis_df.at[row.Index,f'pr_{i-1}']  = ratios[i] 
#                 analysis_df.at[row.Index,f'te_pr_{i-1}_count']  = total[i] 
                
# #             print(f'prob_cutoff: {prob_cutoff} ratio:{ratio} total:{total} success:{success}')
            
#         analysis_df.at[row.Index,'pr_mean']  = ratios[0]  
#         analysis_df.at[row.Index,'pr_med']  = ratios[1] 
#         analysis_df.at[row.Index,'pr_tr_cutoff']  = ratios[7]  
#         analysis_df.at[row.Index,'pr_val_cutoff']  = ratios[8]  
        
#     end = time.time()
#     print(f'Function time:{end-start}')   

#     return analysis_df

In [114]:
MEAN_CUTOFF=-1
MEDIAN_CUTOFF=-2
TR_CUTOFF=-3
VAL_CUTOFF=-4


def save_prob_success(test_filt_df,prob_tr_cutoff,prob_val_cutoff,
                      analysis_df,row_index,
                      bHighCheck):
    probs = test_filt_df['probs']
    prob_cutoffs = [np.mean(probs),
#                     np.percentile(probs,0.75),
                    np.median(probs),
                    0.1,0.2,0.3,0.4,0.5,
                    prob_tr_cutoff,prob_val_cutoff]
    ratios = np.zeros(len(prob_cutoffs))
    total = np.zeros(len(prob_cutoffs))
    for i,prob_cutoff in enumerate(prob_cutoffs):
        test_filt_prob=test_filt_df.loc[probs>prob_cutoff]
#         indices= np.where(probs>prob_cutoff)[0]
#         roll_filt = test['roll_actual'].loc[test_filt_df.iloc[indices].index]
        total[i] = len(test_filt_prob)
        mask=create_target_mask(test_filt_prob,bHighCheck)
        success = len(test_filt_prob[mask])
        if total[i]==0:
            ratios[i]=0
        else:
            ratios[i] = 100 * success / total[i]

        if (i >= 2) & (i<=6):
            analysis_df.at[row_index,f'pr_{i-1}']  = ratios[i] 
            analysis_df.at[row_index,f'te_pr_{i-1}_count']  = total[i] 

#             print(f'prob_cutoff: {prob_cutoff} ratio:{ratio} total:{total} success:{success}')

    analysis_df.at[row_index,'pr_mean']  = ratios[0]  
    analysis_df.at[row_index,'pr_med']  = ratios[1] 
    analysis_df.at[row_index,'pr_tr_cutoff']  = ratios[7]  
    analysis_df.at[row_index,'pr_val_cutoff']  = ratios[8]  
    
    analysis_df.at[row_index,'te_mean_count']  = total[0]  
    analysis_df.at[row_index,'te_med_count']  = total[1] 
    analysis_df.at[row_index,'te_tr_count']  = total[7]  
    analysis_df.at[row_index,'te_val_count']  = total[8]
    
    return analysis_df


def map_prob_cutoff(prob_cutoff,probs,prob_tr_cutoff,prob_val_cutoff):
    
    if prob_cutoff==MEAN_CUTOFF:
        prob_cutoff=np.mean(probs)
    elif prob_cutoff==MEDIAN_CUTOFF:
        prob_cutoff=np.median(probs)
    elif prob_cutoff==TR_CUTOFF:
        prob_cutoff=prob_tr_cutoff
    elif prob_cutoff==VAL_CUTOFF:
        prob_cutoff=prob_val_cutoff
        
    return prob_cutoff

def generate_probs(multi_models,model,data,features):
    if multi_models:
        probs= np.zeros(len(data))
        for i,model_ind in enumerate(model):
            cur_prob= model_ind.predict_proba(data[features])[:,1] 
            probs+= cur_prob / len(model)
    else: 
        probs= model.predict_proba(test_filt_df[features_test])[:,1]
    data['probs']=probs
    return data

def save_test_ml_probs(analysis_df,test,model,
                       prob_tr_cutoff,prob_val_cutoff,
                       pattern,bHighCheck,
                       gen_probs=True,
                       is_prod=False,
                       prob_cutoff=-1,
                      multi_models=False):
    
    start = time.time()
    features_test = [col for col in features if col not in ['target','probs']]
    if  is_prod:
        analysis_df_filt= analysis_df
    else:
        mask = (analysis_df['pattern_99']==pattern) & (analysis_df['target_high']==bHighCheck) 
        analysis_df_filt = analysis_df[mask]
    
    test_filt_all = None
    for row in analysis_df_filt.itertuples():
        digit = row.digit
        cutoff = row.cutoff
        after_val = row.afterval
        bHighCheck = row.target_high
        zero_col_cutoff = row.zero_col_cutoff

        test_filt_df = validate_pattern(test,bHighCheck,zero_col_cutoff,
                                        digit,cutoff,after_val)
        filt_size = len(test_filt_df)
#         print(f'{filt_size=}')
        if filt_size==0:
            print(f'No Record for zero_col_cutoff:{zero_col_cutoff} digit:{digit} cutoff:{cutoff} after_val:{after_val}')
            continue
        if gen_probs:
            test_filt_df = generate_probs(multi_models,model,test_filt_df,features_test)
        probs = test_filt_df['probs'].values
#         if multi_models:
#             probs= np.zeros(len(test_filt_df))
#             for i,model_ind in enumerate(model):
#                 cur_prob= model_ind.predict_proba(test_filt_df[features_test])[:,1] 
#                 probs+= cur_prob / len(model)
#         else: 
#             probs= model.predict_proba(test_filt_df[features_test])[:,1]

        if is_prod:
            prob_cutoff=map_prob_cutoff(prob_cutoff,probs,prob_tr_cutoff,prob_val_cutoff)
            mask = (test_filt_df['probs'] > prob_cutoff)
            test_filt_df=test_filt_df[mask]
        
            if test_filt_all is None:
                test_filt_all=test_filt_df
            else:
                test_filt_all=pd.concat([test_filt_all,test_filt_df],axis=0)
            
        if not is_prod:
            analysis_df = save_prob_success(test_filt_df,prob_tr_cutoff,prob_val_cutoff,
                              analysis_df,row.Index,bHighCheck)
            
    end = time.time()
    print(f'Function time:{end-start}')   

    return test_filt_all,analysis_df

In [115]:
# features_test = [col for col in features if col!='target']
# test_filt_df = validate_pattern(test[features_test],True,4,9,0,8)

# roll_filt = test['roll_actual'].loc[test_filt_df.index]
# total = len(roll_filt)
# if HIGH_ANALYSIS:
#     success = len(roll_filt[roll_filt>=9000])
# else:
#     success = len(roll_filt[roll_filt<1000])
# if total==0:
#     ratio=0
# else:
#     ratio = success / total
# print(f'ratio:{ratio} total:{total} success:{success}')
# # test_filt_df

In [116]:
# probs= xgb_model.predict_proba(test_filt_df)[:,1]
# # print(probs)
# print(np.mean(probs))
# print(np.median(probs))
# prob_cutoffs = [np.mean(probs),np.median(probs),0.1,0.2,0.3,0.4, 0.5]

# for prob_cutoff in prob_cutoffs:
#     indices= np.where(probs>prob_cutoff)[0]
# #     print(len(indices))
#     # print(indices)
#     # print(test_filt_df.iloc[indices].index)
#     roll_filt = test['roll_actual'].loc[test_filt_df.iloc[indices].index]
#     total = len(roll_filt)
#     if HIGH_ANALYSIS:
#         mask = roll_filt>=9000
#         mask_arr =roll_filt.values>=9000
#     else:
#         mask=roll_filt<1000
#         mask_arr =roll_filt.values<1000
# #     print(roll_filt.tail())
#     success = len(roll_filt[mask])
# #     print(np.where(mask_arr))
#     if total==0:
#         ratio=0
#     else:
#         ratio = success / total
#     print(f'prob_cutoff: {prob_cutoff} ratio:{ratio} total:{total} success:{success}')


In [117]:
CONT_ANALYSIS=False
IS_PROD=False

In [118]:
if not IS_PROD:
    
    actual_seed="dcd76f0a8a0e957d278a0286dc2e316cce9b5d10b9864c360d0d828c78a1a952"
#     actual_seed = cur_hash_list[25]
    print(actual_seed)
    # Vectorize the function
    vectorized_calculate_roll = np.vectorize(calculate_roll)

    # Compute the roll values for the input arrays
    roll_array = vectorized_calculate_roll(actual_seed,
                                           results_df_test['client_seed'],
                                           nonce)

    results_df_test['roll_actual']=roll_array
    test_manual = generate_test_features(results_df_test)
    test = generate_features_full(results_df_test,False,feature_chain_length)

dcd76f0a8a0e957d278a0286dc2e316cce9b5d10b9864c360d0d828c78a1a952
['roll_1', 'roll_2', 'roll_3', 'roll_4', 'roll_5', 'roll_6', 'roll_7', 'roll_8', 'roll_9', 'roll_10', 'roll_11', 'roll_12', 'roll_13', 'roll_14', 'roll_15', 'roll_16', 'roll_17', 'roll_18', 'roll_19', 'roll_20', 'roll_21', 'roll_22', 'roll_23', 'roll_24', 'roll_25', 'roll_26', 'roll_27', 'roll_28', 'roll_29', 'roll_30', 'roll_31', 'roll_32', 'roll_33', 'roll_34', 'roll_35', 'roll_36', 'roll_37', 'roll_38', 'roll_39', 'roll_40', 'roll_41', 'roll_42', 'roll_43', 'roll_44', 'roll_45', 'roll_46', 'roll_47', 'roll_48', 'roll_49', 'roll_50', 'roll_51', 'roll_52', 'roll_53', 'roll_54', 'roll_55', 'roll_56', 'roll_57', 'roll_58', 'roll_59', 'roll_60']
['roll_1', 'roll_2', 'roll_3', 'roll_4', 'roll_5', 'roll_6', 'roll_7', 'roll_8', 'roll_9', 'roll_10', 'roll_11', 'roll_12', 'roll_13', 'roll_14', 'roll_15', 'roll_16', 'roll_17', 'roll_18', 'roll_19', 'roll_20']
Roll Features from 1 to 20  completed
['roll_1_nonce', 'roll_2_nonce', 

In [119]:
l1=list(range(1,261))
# l2=list(range(50,80))
for hash_idx in [-1]+l1:
    
    if hash_idx==-1:
        actual_seed="dcd76f0a8a0e957d278a0286dc2e316cce9b5d10b9864c360d0d828c78a1a952"
        suffix = 'orig'
    else:    
        actual_seed = cur_hash_list[hash_idx]
        suffix = hash_idx
    print(actual_seed)
    # Vectorize the function
    vectorized_calculate_roll = np.vectorize(calculate_roll)

    # Compute the roll values for the input arrays
    test[f'roll_actual_{suffix}']=vectorized_calculate_roll(actual_seed,
                                           test['client_seed'],
                                           nonce)

dcd76f0a8a0e957d278a0286dc2e316cce9b5d10b9864c360d0d828c78a1a952
37852b2aec85dc4ef278a4f9eb6e83cff933d03f334d39bf3e4f189b0d364774
4e03d0263e7efef0f77e974064249054dbbd7ea8feb489997284e56eb2e59d98
c29cc874c133ce94ef1316ba0f6cb2cd54846dcca4a9a7793b5d821c8ce27d41
1e4cf26c69ee29e0752ac0dd36ab894a2cb6fa0ca98f49dbe601df9e5f6cda1b
ed0bdaa811adab3127d410057c3e6ed431630a9e60254cedb1fecbcad8cc6110
ae06536d5c885b93e7b9fa7efd59fc1d113a76deaf874bce2976a246bd469aa2
e9cc3992da17a86ae892765c6ddf0b83531b0a0c0e97613dd3590335cad42d5c
8a6fd1eb0899e602a98c82dd14b2368af915da9390accc86edebc42792acf1ef
733e41263252c28b97a52367e62b013025325cd95171727acfea30847c717845
8b89cb15149b97396631c308588721220b4e5a571a00579624bb74b368bbf766
82f50f0290d184669c6d73e215f5b0e40edb411a6b2a09ca4a42c87bfbe1d0f8
1854d7dfa4ad4bfdc52e855e1a295db90d88f8b998a6f7300cd5bca6c4229b53
f858a86151f3efe1cba5f9517c4b3321a47da87a5a4ecf2add7574fc5a4c0039
5956ca4f2ad45476f62038c37c6efeb6fd9ed6956f8407cae6e4eba539ac7cb4
39cfb9001b9db9eb3701bc499

In [120]:
# analysis_df=pd.read_csv(f'data/analysis_df_{nonce}.csv')

In [121]:
mean_imp = np.zeros(len(features)-1)
imp_df = pd.DataFrame()
imp_df['feature'] = [col for col in features if col!='target']
for i,model in enumerate(xgb_models):
    cur_imp = model.feature_importances_
    imp_df[f'importance_m{i}'] = cur_imp
#     print(cur_imp)
    mean_imp += cur_imp / len(xgb_models)

# print(mean_imp)
imp_df['importance'] = mean_imp
imp_df = imp_df.sort_values('importance',ascending=False).reset_index(drop=True)
imp_df

,feature,importance_m0,importance_m1,importance_m2,importance_m3,importance_m4,importance_m5,importance_m6,importance_m7,importance_m8,importance_m9,importance
0,roll_mean_1_60,0.004494,0.004857,0.003521,0.005373,0.005081,0.004519,0.004545,0.004270,0.005192,0.004621,0.004647
1,roll_42_nonce,0.004480,0.004705,0.004480,0.005259,0.004716,0.004458,0.004773,0.004153,0.004809,0.004344,0.004618
2,roll_60,0.005091,0.004781,0.004612,0.004851,0.004514,0.004489,0.004435,0.004398,0.004865,0.004082,0.004612
3,roll_19_nonce,0.004369,0.004341,0.004783,0.004774,0.004650,0.004460,0.004540,0.005113,0.004181,0.004686,0.004590
4,roll_mean_nonce,0.004089,0.005111,0.004705,0.004212,0.004574,0.004654,0.005245,0.004499,0.004698,0.004033,0.004582
5,roll_mean,0.004637,0.004224,0.004970,0.005326,0.004733,0.004056,0.004346,0.004539,0.003748,0.003942,0.004452
6,roll_mean_25_50_nonce,0.004465,0.004844,0.004545,0.004229,0.004933,0.003840,0.004646,0.004380,0.004182,0.004371,0.004443
7,roll_mean_1_60_nonce,0.004465,0.004068,0.004436,0.004962,0.004592,0.004632,0.004151,0.004473,0.004470,0.004107,0.004436
8,roll_7_nonce,0.004307,0.004609,0.004903,0.004191,0.004831,0.004590,0.004363,0.004197,0.004261,0.003935,0.004419
9,roll_32_nonce,0.004889,0.004664,0.004217,0.004400,0.004131,0.004249,0.004336,0.004720,0.004050,0.004326,0.004398


In [122]:
%%time
test_hash25 = generate_features_full(results_df_test_hash25,False,feature_chain_length)
test_hash25.head()

['roll_1', 'roll_2', 'roll_3', 'roll_4', 'roll_5', 'roll_6', 'roll_7', 'roll_8', 'roll_9', 'roll_10', 'roll_11', 'roll_12', 'roll_13', 'roll_14', 'roll_15', 'roll_16', 'roll_17', 'roll_18', 'roll_19', 'roll_20']
Roll Features from 1 to 20  completed
['roll_1_nonce', 'roll_2_nonce', 'roll_3_nonce', 'roll_4_nonce', 'roll_5_nonce', 'roll_6_nonce', 'roll_7_nonce', 'roll_8_nonce', 'roll_9_nonce', 'roll_10_nonce', 'roll_11_nonce', 'roll_12_nonce', 'roll_13_nonce', 'roll_14_nonce', 'roll_15_nonce', 'roll_16_nonce', 'roll_17_nonce', 'roll_18_nonce', 'roll_19_nonce', 'roll_20_nonce']
Roll Features from 1 to 20 _nonce completed
['roll_25', 'roll_26', 'roll_27', 'roll_28', 'roll_29', 'roll_30', 'roll_31', 'roll_32', 'roll_33', 'roll_34', 'roll_35', 'roll_36', 'roll_37', 'roll_38', 'roll_39', 'roll_40', 'roll_41', 'roll_42', 'roll_43', 'roll_44', 'roll_45', 'roll_46', 'roll_47', 'roll_48', 'roll_49', 'roll_50']
Roll Features from 25 to 50  completed
['roll_25_nonce', 'roll_26_nonce', 'roll_27_nonc

,roll_actual,seed,hash,client_seed,roll_0,roll_1,roll_2,roll_3,roll_4,roll_5,roll_6,roll_7,roll_8,roll_9,roll_10,roll_11,roll_12,roll_13,roll_14,roll_15,roll_16,roll_17,roll_18,roll_19,roll_20,roll_21,roll_22,roll_23,roll_24,roll_25,roll_26,roll_27,roll_28,roll_29,roll_30,roll_31,roll_32,roll_33,roll_34,roll_35,roll_36,roll_37,roll_38,roll_39,roll_40,roll_41,roll_42,roll_43,roll_44,roll_45,roll_46,roll_47,roll_48,roll_49,roll_50,roll_51,roll_52,roll_53,roll_54,roll_55,roll_56,roll_57,roll_58,roll_59,roll_60,count_lt_1000,count_gt_9000,count_gt_8000,count_gt_7000,count_gt_6000,count_gt_5000,count_gt_4000,count_gt_3000,count_gt_2000,count_gt_1000,count_hi,count_lo,count_lo_hi,count_gt_9000_k,count_lt_1000_k,total_dig_9,total_dig_0,roll_mean,roll_std,count_lt_1000_25_50,count_gt_9000_25_50,count_gt_8000_25_50,count_gt_7000_25_50,count_gt_6000_25_50,count_gt_5000_25_50,count_gt_4000_25_50,count_gt_3000_25_50,count_gt_2000_25_50,count_gt_1000_25_50,count_hi_25_50,count_lo_25_50,count_lo_hi_25_50,count_gt_9000_k_25_50,count_lt_1000_k_25_50,total_dig_9_25_50,total_dig_0_25_50,roll_mean_25_50,roll_std_25_50,count_lt_1000_50_60,count_gt_9000_50_60,count_gt_8000_50_60,count_gt_7000_50_60,count_gt_6000_50_60,count_gt_5000_50_60,count_gt_4000_50_60,count_gt_3000_50_60,count_gt_2000_50_60,count_gt_1000_50_60,count_hi_50_60,count_lo_50_60,count_lo_hi_50_60,count_gt_9000_k_50_60,count_lt_1000_k_50_60,total_dig_9_50_60,total_dig_0_50_60,roll_mean_50_60,roll_std_50_60,count_lt_1000_1_60,count_gt_9000_1_60,count_gt_8000_1_60,count_gt_7000_1_60,count_gt_6000_1_60,count_gt_5000_1_60,count_gt_4000_1_60,count_gt_3000_1_60,count_gt_2000_1_60,count_gt_1000_1_60,count_hi_1_60,count_lo_1_60,count_lo_hi_1_60,count_gt_9000_k_1_60,count_lt_1000_k_1_60,total_dig_9_1_60,total_dig_0_1_60,roll_mean_1_60,roll_std_1_60,roll_0_nonce,roll_1_nonce,roll_2_nonce,roll_3_nonce,roll_4_nonce,roll_5_nonce,roll_6_nonce,roll_7_nonce,roll_8_nonce,roll_9_nonce,roll_10_nonce,roll_11_nonce,roll_12_nonce,roll_13_nonce,roll_14_nonce,roll_15_nonce,roll_16_nonce,roll_17_nonce,roll_18_nonce,roll_19_nonce,roll_20_nonce,roll_21_nonce,roll_22_nonce,roll_23_nonce,roll_24_nonce,roll_25_nonce,roll_26_nonce,roll_27_nonce,roll_28_nonce,roll_29_nonce,roll_30_nonce,roll_31_nonce,roll_32_nonce,roll_33_nonce,roll_34_nonce,roll_35_nonce,roll_36_nonce,roll_37_nonce,roll_38_nonce,roll_39_nonce,roll_40_nonce,roll_41_nonce,roll_42_nonce,roll_43_nonce,roll_44_nonce,roll_45_nonce,roll_46_nonce,roll_47_nonce,roll_48_nonce,roll_49_nonce,roll_50_nonce,roll_51_nonce,roll_52_nonce,roll_53_nonce,roll_54_nonce,roll_55_nonce,roll_56_nonce,roll_57_nonce,roll_58_nonce,roll_59_nonce,roll_60_nonce,count_lt_1000_nonce,count_gt_9000_nonce,count_gt_8000_nonce,count_gt_7000_nonce,count_gt_6000_nonce,count_gt_5000_nonce,count_gt_4000_nonce,count_gt_3000_nonce,count_gt_2000_nonce,count_gt_1000_nonce,count_hi_nonce,count_lo_nonce,count_lo_hi_nonce,count_gt_9000_k_nonce,count_lt_1000_k_nonce,total_dig_9_nonce,total_dig_0_nonce,roll_mean_nonce,roll_std_nonce,count_lt_1000_25_50_nonce,count_gt_9000_25_50_nonce,count_gt_8000_25_50_nonce,count_gt_7000_25_50_nonce,count_gt_6000_25_50_nonce,count_gt_5000_25_50_nonce,count_gt_4000_25_50_nonce,count_gt_3000_25_50_nonce,count_gt_2000_25_50_nonce,count_gt_1000_25_50_nonce,count_hi_25_50_nonce,count_lo_25_50_nonce,count_lo_hi_25_50_nonce,count_gt_9000_k_25_50_nonce,count_lt_1000_k_25_50_nonce,total_dig_9_25_50_nonce,total_dig_0_25_50_nonce,roll_mean_25_50_nonce,roll_std_25_50_nonce,count_lt_1000_50_60_nonce,count_gt_9000_50_60_nonce,count_gt_8000_50_60_nonce,count_gt_7000_50_60_nonce,count_gt_6000_50_60_nonce,count_gt_5000_50_60_nonce,count_gt_4000_50_60_nonce,count_gt_3000_50_60_nonce,count_gt_2000_50_60_nonce,count_gt_1000_50_60_nonce,count_hi_50_60_nonce,count_lo_50_60_nonce,count_lo_hi_50_60_nonce,count_gt_9000_k_50_60_nonce,count_lt_1000_k_50_60_nonce,total_dig_9_50_60_nonce,total_dig_0_50_60_nonce,roll_mean_50_60_nonce,roll_std_50_60_nonce,count_lt_1000_1_60_nonce,count_gt_9

In [123]:
%%time
test_hash24 = generate_features_full(results_df_test_hash24,False,feature_chain_length)
test_hash24.head()

['roll_1', 'roll_2', 'roll_3', 'roll_4', 'roll_5', 'roll_6', 'roll_7', 'roll_8', 'roll_9', 'roll_10', 'roll_11', 'roll_12', 'roll_13', 'roll_14', 'roll_15', 'roll_16', 'roll_17', 'roll_18', 'roll_19', 'roll_20']
Roll Features from 1 to 20  completed
['roll_1_nonce', 'roll_2_nonce', 'roll_3_nonce', 'roll_4_nonce', 'roll_5_nonce', 'roll_6_nonce', 'roll_7_nonce', 'roll_8_nonce', 'roll_9_nonce', 'roll_10_nonce', 'roll_11_nonce', 'roll_12_nonce', 'roll_13_nonce', 'roll_14_nonce', 'roll_15_nonce', 'roll_16_nonce', 'roll_17_nonce', 'roll_18_nonce', 'roll_19_nonce', 'roll_20_nonce']
Roll Features from 1 to 20 _nonce completed
['roll_25', 'roll_26', 'roll_27', 'roll_28', 'roll_29', 'roll_30', 'roll_31', 'roll_32', 'roll_33', 'roll_34', 'roll_35', 'roll_36', 'roll_37', 'roll_38', 'roll_39', 'roll_40', 'roll_41', 'roll_42', 'roll_43', 'roll_44', 'roll_45', 'roll_46', 'roll_47', 'roll_48', 'roll_49', 'roll_50']
Roll Features from 25 to 50  completed
['roll_25_nonce', 'roll_26_nonce', 'roll_27_nonc

,roll_actual,seed,hash,client_seed,roll_0,roll_1,roll_2,roll_3,roll_4,roll_5,roll_6,roll_7,roll_8,roll_9,roll_10,roll_11,roll_12,roll_13,roll_14,roll_15,roll_16,roll_17,roll_18,roll_19,roll_20,roll_21,roll_22,roll_23,roll_24,roll_25,roll_26,roll_27,roll_28,roll_29,roll_30,roll_31,roll_32,roll_33,roll_34,roll_35,roll_36,roll_37,roll_38,roll_39,roll_40,roll_41,roll_42,roll_43,roll_44,roll_45,roll_46,roll_47,roll_48,roll_49,roll_50,roll_51,roll_52,roll_53,roll_54,roll_55,roll_56,roll_57,roll_58,roll_59,roll_60,count_lt_1000,count_gt_9000,count_gt_8000,count_gt_7000,count_gt_6000,count_gt_5000,count_gt_4000,count_gt_3000,count_gt_2000,count_gt_1000,count_hi,count_lo,count_lo_hi,count_gt_9000_k,count_lt_1000_k,total_dig_9,total_dig_0,roll_mean,roll_std,count_lt_1000_25_50,count_gt_9000_25_50,count_gt_8000_25_50,count_gt_7000_25_50,count_gt_6000_25_50,count_gt_5000_25_50,count_gt_4000_25_50,count_gt_3000_25_50,count_gt_2000_25_50,count_gt_1000_25_50,count_hi_25_50,count_lo_25_50,count_lo_hi_25_50,count_gt_9000_k_25_50,count_lt_1000_k_25_50,total_dig_9_25_50,total_dig_0_25_50,roll_mean_25_50,roll_std_25_50,count_lt_1000_50_60,count_gt_9000_50_60,count_gt_8000_50_60,count_gt_7000_50_60,count_gt_6000_50_60,count_gt_5000_50_60,count_gt_4000_50_60,count_gt_3000_50_60,count_gt_2000_50_60,count_gt_1000_50_60,count_hi_50_60,count_lo_50_60,count_lo_hi_50_60,count_gt_9000_k_50_60,count_lt_1000_k_50_60,total_dig_9_50_60,total_dig_0_50_60,roll_mean_50_60,roll_std_50_60,count_lt_1000_1_60,count_gt_9000_1_60,count_gt_8000_1_60,count_gt_7000_1_60,count_gt_6000_1_60,count_gt_5000_1_60,count_gt_4000_1_60,count_gt_3000_1_60,count_gt_2000_1_60,count_gt_1000_1_60,count_hi_1_60,count_lo_1_60,count_lo_hi_1_60,count_gt_9000_k_1_60,count_lt_1000_k_1_60,total_dig_9_1_60,total_dig_0_1_60,roll_mean_1_60,roll_std_1_60,roll_0_nonce,roll_1_nonce,roll_2_nonce,roll_3_nonce,roll_4_nonce,roll_5_nonce,roll_6_nonce,roll_7_nonce,roll_8_nonce,roll_9_nonce,roll_10_nonce,roll_11_nonce,roll_12_nonce,roll_13_nonce,roll_14_nonce,roll_15_nonce,roll_16_nonce,roll_17_nonce,roll_18_nonce,roll_19_nonce,roll_20_nonce,roll_21_nonce,roll_22_nonce,roll_23_nonce,roll_24_nonce,roll_25_nonce,roll_26_nonce,roll_27_nonce,roll_28_nonce,roll_29_nonce,roll_30_nonce,roll_31_nonce,roll_32_nonce,roll_33_nonce,roll_34_nonce,roll_35_nonce,roll_36_nonce,roll_37_nonce,roll_38_nonce,roll_39_nonce,roll_40_nonce,roll_41_nonce,roll_42_nonce,roll_43_nonce,roll_44_nonce,roll_45_nonce,roll_46_nonce,roll_47_nonce,roll_48_nonce,roll_49_nonce,roll_50_nonce,roll_51_nonce,roll_52_nonce,roll_53_nonce,roll_54_nonce,roll_55_nonce,roll_56_nonce,roll_57_nonce,roll_58_nonce,roll_59_nonce,roll_60_nonce,count_lt_1000_nonce,count_gt_9000_nonce,count_gt_8000_nonce,count_gt_7000_nonce,count_gt_6000_nonce,count_gt_5000_nonce,count_gt_4000_nonce,count_gt_3000_nonce,count_gt_2000_nonce,count_gt_1000_nonce,count_hi_nonce,count_lo_nonce,count_lo_hi_nonce,count_gt_9000_k_nonce,count_lt_1000_k_nonce,total_dig_9_nonce,total_dig_0_nonce,roll_mean_nonce,roll_std_nonce,count_lt_1000_25_50_nonce,count_gt_9000_25_50_nonce,count_gt_8000_25_50_nonce,count_gt_7000_25_50_nonce,count_gt_6000_25_50_nonce,count_gt_5000_25_50_nonce,count_gt_4000_25_50_nonce,count_gt_3000_25_50_nonce,count_gt_2000_25_50_nonce,count_gt_1000_25_50_nonce,count_hi_25_50_nonce,count_lo_25_50_nonce,count_lo_hi_25_50_nonce,count_gt_9000_k_25_50_nonce,count_lt_1000_k_25_50_nonce,total_dig_9_25_50_nonce,total_dig_0_25_50_nonce,roll_mean_25_50_nonce,roll_std_25_50_nonce,count_lt_1000_50_60_nonce,count_gt_9000_50_60_nonce,count_gt_8000_50_60_nonce,count_gt_7000_50_60_nonce,count_gt_6000_50_60_nonce,count_gt_5000_50_60_nonce,count_gt_4000_50_60_nonce,count_gt_3000_50_60_nonce,count_gt_2000_50_60_nonce,count_gt_1000_50_60_nonce,count_hi_50_60_nonce,count_lo_50_60_nonce,count_lo_hi_50_60_nonce,count_gt_9000_k_50_60_nonce,count_lt_1000_k_50_60_nonce,total_dig_9_50_60_nonce,total_dig_0_50_60_nonce,roll_mean_50_60_nonce,roll_std_50_60_nonce,count_lt_1000_1_60_nonce,count_gt_9

In [124]:
test_hash1 = test_hash.copy()
# test_hash = test_hash24.copy()
# test_hash = test_hash1.copy()

In [125]:
test_filt=test.copy()
test_filt_hash=test_hash.copy()
features_test = [col for col in features if col not in ['target','probs']]

In [126]:
test_hash_probs = generate_probs(True,xgb_models,test_hash,features_test)
cutoffs = []
ratios =[]
totals =[]
print(test_hash_probs['probs'].describe())
cutoffs = np.linspace(test_hash_probs['probs'].min(),
                      test_hash_probs['probs'].quantile(0.95),200)
for i,cutoff in enumerate(cutoffs[:len(cutoffs)-1]):
#     cutoff=0.4
#     diff = 0.01
#     print(cutoff,cutoffs[i+1])
    mask = (test_hash_probs['probs']>=cutoff) & (test_hash_probs['probs']<=cutoffs[i+1])
    mask2=create_target_mask(test_hash_probs)
    
#     mask2 = test_hash_probs['roll_actual']>=9000
    total = len(test_hash_probs[mask])
    if total==0:
        ratio=0
        success=0
    else:
        success = len(test_hash_probs[mask & mask2])
        ratio = success / total
    ratios.append(ratio)
    totals.append(total)
#     print(ratio,success,total)

print(len(ratios),len(cutoffs))
df=pd.DataFrame()
df['cutoff']=cutoffs[:len(cutoffs)-1]
df['cutoff_2']=cutoffs[1:len(cutoffs)]
df['ratio'] =ratios
df['total'] =totals

df = df.sort_values('ratio',ascending=False).reset_index(drop=True)
print(df[:5].mean())
print(df[:10].mean())
df.head(10)

count    8205.000000
mean        0.455080
std         0.037694
min         0.234803
25%         0.435310
50%         0.460629
75%         0.480696
max         0.564209
Name: probs, dtype: float64
199 200
cutoff      0.327778
cutoff_2    0.329145
ratio       0.640000
total       5.400000
dtype: float64
cutoff      0.348150
cutoff_2    0.349518
ratio       0.467727
total       8.400000
dtype: float64


,cutoff,cutoff_2,ratio,total
0,0.281291,0.282658,1.000000,1
1,0.294963,0.296331,1.000000,1
2,0.341451,0.342818,0.400000,5
3,0.381102,0.382469,0.400000,15
4,0.340083,0.341451,0.400000,5
5,0.356491,0.357858,0.333333,3
6,0.342818,0.344185,0.333333,3
7,0.316840,0.318207,0.333333,3
8,0.322309,0.323676,0.250000,4
9,0.504156,0.505523,0.227273,44


In [127]:
df

,cutoff,cutoff_2,ratio,total
0,0.281291,0.282658,1.000000,1
1,0.294963,0.296331,1.000000,1
2,0.341451,0.342818,0.400000,5
3,0.381102,0.382469,0.400000,15
4,0.340083,0.341451,0.400000,5
5,0.356491,0.357858,0.333333,3
6,0.342818,0.344185,0.333333,3
7,0.316840,0.318207,0.333333,3
8,0.322309,0.323676,0.250000,4
9,0.504156,0.505523,0.227273,44


In [128]:
df.describe()

,cutoff,cutoff_2,ratio,total
count,199.000000,199.000000,199.000000,199.000000
mean,0.370163,0.371531,0.079611,39.165829
std,0.078742,0.078742,0.122629,48.435604
min,0.234803,0.236171,0.000000,0.000000
25%,0.302483,0.303851,0.000000,1.000000
50%,0.370163,0.371531,0.066667,12.000000
75%,0.437843,0.439211,0.115417,69.000000
max,0.505523,0.506891,1.000000,167.000000


In [129]:
print(len(df[df['total']==2]))
print(len(df[(df['total']==2) & (df['ratio']>0)]))
df[df['total']==2]

6
0


,cutoff,cutoff_2,ratio,total
139,0.256680,0.258047,0.0,2
161,0.320942,0.322309,0.0,2
164,0.325043,0.326411,0.0,2
166,0.327778,0.329145,0.0,2
171,0.333247,0.334614,0.0,2
181,0.286760,0.288127,0.0,2


In [130]:
df[df['ratio']<0.12].index[0]

46

In [131]:
test['roll_actual']=test[f'roll_actual_orig'].copy()

In [132]:
test_probs = generate_probs(True,xgb_models,test,features_test)
top_k=[]
all_k=[]
for k in range(len(df)):
    row = df.iloc[k]
#     print(row['ratio'],row['cutoff'],row['cutoff_2'])
    mask = (test_probs['probs']>=row['cutoff']) & (test_probs['probs']<=row['cutoff_2'])
    mask2 = create_target_mask(test_probs)    
    test_filt = test_probs[mask]

    total = len(test_filt)
    if total==0:
        success=0
        ratio=0
    else:
        success = len(test_probs[mask & mask2])
        ratio = success / total
    if total==2:
        all_k.append(k) 
    if ratio>=0.12:
        if total==2:
            top_k.append((k,ratio,total))
#     print(ratio,success,total,row['cutoff'],k)
print(len(all_k))
print(all_k)
print(len(top_k))
print(top_k)
mask = (test_hash_probs['probs']>=row['cutoff']) & (test_hash_probs['probs']<=row['cutoff_2'])
test_filt_hash  = test_hash_probs[mask]

11
[10, 130, 146, 161, 169, 174, 183, 186, 188, 192, 195]
4
[(10, 0.5, 2), (169, 1.0, 2), (188, 0.5, 2), (192, 0.5, 2)]


In [154]:
all_k_arr = np.array(all_k)
diff = np.diff(all_k_arr)
print(diff)
ind = np.where(diff>=15)
print(ind[0]+1)
trans_k = np.sort(np.concatenate((all_k_arr[ind[0]],all_k_arr[ind[0]+1])))
trans_k

[120  16  15   8   5   9   3   2   4   3]
[1 2 3]


array([ 10, 130, 130, 146, 146, 161])

In [134]:
def get_sim_cutoff_data(test,xgb_models,features_test,test_hash_cutoff_df,
                       is_imp_feats,imp_feats_count,gen_probs=True):
    if gen_probs:
        test_probs = generate_probs(True,xgb_models,test,features_test)
    else:
        test_probs = test
    print('Generate probs completed')
    test_probs['roll_actual_init']=test_probs['roll_actual'].copy()

    if is_imp_feats:
        hash_groups =[ [col for col in imp_df['feature'].values if (col.startswith('roll_'))][:imp_feats_count] ]
#         hash_groups = [['roll_mean_nonce']]
    else:
    #     hash_groups = [['roll_21_nonce', 'roll_37_nonce', 'roll_mean_nonce','roll_36_nonce','roll_48',
    #                    'roll_53_nonce','roll_32_nonce','roll_44_nonce','roll_38_nonce','roll_17']]
        hash_groups = [list(range(1,11)),list(range(25,36)),list(range(50,60)),list(range(100,111)) ]
#         hash_groups = [list(range(25,36)) ]
    hash_groups_flat = [item for sublist in hash_groups for item in sublist]
    print(hash_groups)
    print('list creation complete')
    

    top_k=[]
    sim_cutoff_df = pd.DataFrame(columns=['k','ratio','total','hash_no','hash_group'])
    df_size = len(test_hash_cutoff_df)
    for i,row in enumerate(test_hash_cutoff_df.itertuples()):
        k=row.Index
#         if (k%10)==0:
#             print(f'loop for k = {k}')
#         row = test_hash_cutoff_df.iloc[k]
    #     print(row['ratio'],row['cutoff'],row['cutoff_2'])
        mask = (test_probs['probs']>=row.cutoff) & (test_probs['probs']<=row.cutoff_2)
        test_filt = test_probs[mask]
        total = len(test_filt)
        ratios=[]

        for idx in hash_groups_flat:
#             print(f'k:{k} idx:{idx}')
            if is_imp_feats:
                test_filt['roll_actual']=test_filt[idx].copy()    
            else:
                test_filt['roll_actual']=test_filt[f'roll_actual_{idx}'].copy()
            mask2 = create_target_mask(test_filt)
            if total==0:
                success=0
                ratio=0
            else:
                success = len(test_filt[mask2])
                ratio = success / total
#             print(ratio,success,total,row['cutoff'],k)
            sim_cutoff_df.loc[len(sim_cutoff_df.index)] = [k,ratio, total, idx, np.nan]

    for group_no,group in enumerate(hash_groups):
        start =group[0]
        end = group[-1]
        mask = (sim_cutoff_df['hash_no']>=start) & (sim_cutoff_df['hash_no']<=end) 
        sim_cutoff_df.loc[mask,'hash_group']=group_no
        
    test_probs['roll_actual']=test_probs['roll_actual_init'].copy()
    
    return sim_cutoff_df,test_probs
        



In [135]:
def get_hash_prob_top_data(test,xgb_models,features_test,
                           test_hash_cutoff_df,is_imp_feats,imp_feats_count,
                           sim_cutoff_df = None,is_min_total=True,
                           total_cri=12,top_n=10):

    if sim_cutoff_df is None:
        sim_cutoff_df,test_probs= get_sim_cutoff_data(test,xgb_models,features_test,
                                                      test_hash_cutoff_df,
                                                     is_imp_feats,imp_feats_count)
    else:
        test_probs = generate_probs(True,xgb_models,test,features_test)
        
    print(len(sim_cutoff_df))
#     print(sim_cutoff_df)
    #filter only high total records
    if is_min_total:
        sim_cutoff_df_filt = sim_cutoff_df[sim_cutoff_df['total']>=total_cri]
    else:
        sim_cutoff_df_filt = sim_cutoff_df[sim_cutoff_df['total']<=total_cri]
        sim_cutoff_df_filt = sim_cutoff_df_filt[sim_cutoff_df_filt['total']>1]
    print(len(sim_cutoff_df_filt))

    sim_summary  = sim_cutoff_df_filt.groupby(['k']).agg( total =('total','mean'),
                                                         ratio_mean=('ratio', 'mean'), 
                                                          ratio_std=('ratio', 'std')).reset_index()
#     print(sim_summary.head())

    top_mean_data = sim_summary.sort_values('ratio_mean',ascending=False)[:top_n].reset_index(drop=True)
    print('--- Top 5 data ---')
    print(top_mean_data[:5])
    print('--- Top 6 to 10 data ---')
    print(top_mean_data[5:])
#     top_std_row = top_mean_data.sort_values('ratio_std',ascending=True).iloc[0]
#     print('--- Selected K ---')
#     print(top_std_row)
#     k_sel = int(top_std_row['k'])

#     k_sel = int(top_mean_data[0:1]['k'])
#     row = test_hash_cutoff_df.iloc[k_sel]
#     #     print(row['ratio'],row['cutoff'],row['cutoff_2'])
#     mask = (test_probs['probs']>=row['cutoff']) & (test_probs['probs']<=row['cutoff_2'])
#     test_sel = test_probs[mask]
#     print('Test Size:',len(test_sel))
    
    return top_mean_data


In [136]:
# k_sel,test_sel = get_test_sel_hash_prob(test,xgb_models,features_test,df,
#                           min_total=12, top_n=10)
# print(k_sel)

In [137]:
def create_cutoff_pos_df(test,models,features_test,test_hash_cutoff_df,
                         imp_feats_count_list,gen_probs=True,is_compute_matches=True,
                   test_ratio_limit=0.12,sim_cutoff_dfs=None,
                         low_high_record_condns=[(True,12,10),(False,10,10),(False,2,10)]):

    cutoff_pos_df =pd.DataFrame(columns = ['imp_feats_count','condn_max','condn_val',
                                           'k_top_1','k_top_2','k_top_3','k_top_4','k_top_5',
                                          'mean_top_1', 'mean_top_2','mean_top_3','mean_top_4',
                                          'mean_top_5','match_pos_1',
                                          'match_total_top_5','match_total_top_10','match_ks_top_5'])

    
    max_good_k= test_hash_cutoff_df[test_hash_cutoff_df['ratio']<test_ratio_limit].index[0]-1
    print(f'{max_good_k=}')
    is_imp_feats = True 
    gen_sim_cutoff=False
    if sim_cutoff_dfs is None:
        gen_sim_cutoff=True
        sim_cutoff_dfs=[]
    elif gen_probs:
        test_probs = generate_probs(True,xgb_models,test,features_test) 
    else:
        test_probs = test
    for i,imp_feats_count in enumerate(imp_feats_count_list):
        print()
        print(f'********************** Feature count {imp_feats_count} *****************')
        if gen_sim_cutoff:
            sim_cutoff_df,test_probs= get_sim_cutoff_data(test,models,features_test,test_hash_cutoff_df,
                                                     is_imp_feats,imp_feats_count,gen_probs)
            sim_cutoff_dfs.append(sim_cutoff_df)
        else:
            sim_cutoff_df=sim_cutoff_dfs[i]
        
        for condn in low_high_record_condns:
            print(f'******* Condition {condn} for feature count {imp_feats_count} *****************')
            top_mean_data = get_hash_prob_top_data(test,models,features_test,
                                       test_hash_cutoff_df,is_imp_feats,imp_feats_count,
                                       sim_cutoff_df = sim_cutoff_df,
                                        is_min_total=condn[0], total_cri=condn[1],
                                        top_n=condn[2])
            
#             print(top_mean_data)
            top_5_mean_data = top_mean_data[:5]
            k_tops=list(top_5_mean_data['k'].values)
            mean_tops=list(top_5_mean_data['ratio_mean'].values)
            
            match_pos_1=np.nan
            match_total_top_5=np.nan
            match_total_top_10=np.nan
            match_ks_top_5=np.nan
            
            if is_compute_matches:
                
                top_mean_data['rank']=top_mean_data['ratio_mean'].rank(method='average',
                                                                  ascending=False)
                top_5_mean_data = top_mean_data[:5]
               
                matches_top_5=top_5_mean_data[top_5_mean_data['k']<=max_good_k]
                matches_top_5=matches_top_5[matches_top_5['ratio_mean']>0]
                match_total_top_5=len(matches_top_5)

                matches_top=top_mean_data[top_mean_data['k']<=max_good_k]
                matches_top=matches_top[matches_top['ratio_mean']>0]
                match_total_top_10 = len(matches_top)

                if match_total_top_5==0:
                    match_ks_top_5=[]
                    if match_total_top_10==0:
                        match_pos_1=11
                    else:
                        match_pos_1=matches_top.iloc[0]['rank']
                else:
    #                 match_pos_1=matches_top_5.index[0]+1
                    match_pos_1=matches_top_5.iloc[0]['rank']
                    match_ks_top_5 = list(matches_top_5['k'].values)
 
            
            result = [imp_feats_count,condn[0],condn[1]]+k_tops+mean_tops \
                    +[match_pos_1,match_total_top_5,match_total_top_10,match_ks_top_5]
        
#             print(cutoff_pos_df.columns)
#             print(f'{len(cutoff_pos_df.columns),len(result),result}')
            next_idx = len(cutoff_pos_df.index)
            cutoff_pos_df.loc[next_idx] = result
        
            if is_compute_matches:
                match_cols = [col for col in cutoff_pos_df if col.startswith('match')]
                print(cutoff_pos_df.loc[next_idx][match_cols])
        
    return cutoff_pos_df,sim_cutoff_dfs,test_probs


In [138]:
import math

def get_test_sel_cutoff_pos(test_probs,models,features_test,test_hash_cutoff_df,
                            cutoff_pos_df_hash,
                            is_min_total=False,total_cri=2,top_n=10,
                           sim_cutoff_dfs=None,imp_feats_count_list=None):
    cutoff_pos_stat = cutoff_pos_df_hash.groupby(['imp_feats_count'])['match_pos_1',
                                               'match_total_top_5',
                                               'match_total_top_10'].mean().reset_index()
    print(' ........... Overall Positions of Features .........')
    print(cutoff_pos_stat)
    cutoff_pos_df_hash_filt=cutoff_pos_df_hash[cutoff_pos_df_hash['condn_max']==False]
    cutoff_pos_stat = cutoff_pos_df_hash_filt.groupby(['imp_feats_count'])['match_pos_1',
                                               'match_total_top_5',
                                               'match_total_top_10'].mean().reset_index()
    print(' ........... Low Positions of Features .........')
    print(cutoff_pos_stat)
    cutoff_pos_best= cutoff_pos_stat.sort_values(['match_pos_1','match_total_top_5'],
                                                ascending=[True,False])\
                                    .reset_index().iloc[0]
    print('*** Best cutoff position in hash **** ')
    print(cutoff_pos_best)
    predicted_raw_pos=cutoff_pos_best['match_pos_1']
    predicted_max_pos = math.ceil(predicted_raw_pos)
    
    if predicted_raw_pos==predicted_max_pos:
        predicted_max_pos+=1
    
    is_imp_feats=True
    imp_feats_count=cutoff_pos_best['imp_feats_count']
    is_min_total=False
    
    if sim_cutoff_dfs is None:
        sim_cutoff_df,test_probs= get_sim_cutoff_data(test_probs,models,features_test,test_hash_cutoff_df,
                                                 is_imp_feats,imp_feats_count,gen_probs=False)
    else:
        #to use existing buffer, read the matching sim cutoff df from the list of buffer
        idx = imp_feats_count_list.index(imp_feats_count)
        sim_cutoff_df=sim_cutoff_dfs[idx]

    top_mean_data = get_hash_prob_top_data(test_probs,models,features_test,
                           test_hash_cutoff_df,is_imp_feats,imp_feats_count,
                           sim_cutoff_df = sim_cutoff_df,
                            is_min_total=is_min_total, total_cri=total_cri,
                            top_n=top_n)
    
    top_mean_data['rank']=top_mean_data['ratio_mean'].rank(method='min',ascending=False)
    mask = (top_mean_data['rank']<=predicted_max_pos) & (top_mean_data['ratio_mean']>0)
    top_mean_sel = top_mean_data[mask]
    k_sel_list = list(top_mean_sel['k'].values)
    print(top_mean_data)
    print(top_mean_sel)
    print(f'{k_sel_list=}')
    

#     mask = cutoff_pos_df_actual['imp_feats_count']==cutoff_pos_best['cutoff_pos_best']
#     mask2 = cutoff_pos_df_actual['match_pos_1']<=predicted_max_pos
#     k_sel = cutoff_pos_df_actual.loc[mask & mask2, 'k' ]
    
    test_sel  = None
    for k_sel in k_sel_list:
        row = test_hash_cutoff_df.loc[k_sel]
    #     print(row)
        mask = (test_probs['probs']>=row['cutoff']) & (test_probs['probs']<=row['cutoff_2'])
        cur_test_sel = test_probs[mask]
        cur_test_sel['k']=k_sel
        if test_sel is None:
            test_sel = cur_test_sel
        else:
            test_sel = pd.concat([test_sel,cur_test_sel],axis=0)
        print('Test Size:',len(test_sel))

    return k_sel_list,test_sel,top_mean_sel



In [139]:
# k_sel_list=[218,198,201,190,184]
# print(df[df.index.isin(k_sel_list)])
# test_temp=None
# for k_sel in k_sel_list:
#     row = df.loc[k_sel]
# #     print(row)
#     mask = (test['probs']>=row['cutoff']) & (test['probs']<=row['cutoff_2'])
#     cur_test_sel = test[mask]
#     cur_test_sel['k']=k_sel
#     if test_temp is None:
#         test_temp = cur_test_sel
#     else:
#         test_temp = pd.concat([test_temp,cur_test_sel],axis=0)

# cols = [col for col in imp_df['feature'].values if (col.startswith('roll_'))][:20]        
# # test_temp
# test_temp['feats_mean']=test_temp[cols].mean(axis=1)
# test_temp['feats_std']=test_temp[cols].std(axis=1)
# test_temp['feats_count']=(test_temp[cols]<1000).sum(axis=1)
# test_temp[['k','roll_actual','feats_mean','feats_std','feats_count']+cols]

In [140]:
# test_orig= test.copy()
# test_hash_orig = test_hash.copy()

In [141]:
count_all_k = len(all_k)
if count_all_k<10:
    print(f'PROJECT WARNING: number of target records is {count_all_k} which is less than 10 ')

In [142]:
print(len(test))
mask = test.duplicated(subset='client_seed')
print(len(test[mask]))
print(test[mask]['client_seed'].nunique())

print(len(test_hash))
mask = test_hash.duplicated(subset='client_seed')
print(len(test_hash[mask]))
print(test_hash[mask]['client_seed'].nunique())

8115
0
0
8205
0
0


In [143]:
# test=test.drop_duplicates(subset='client_seed')
# test_hash=test_hash.drop_duplicates(subset='client_seed')

In [144]:
# %%time
# sim_cutoff_dfs_temp=None
# imp_feats_count_list = [2,5,10,15,20,25,30,40]
# cutoff_pos_df_temp,sim_cutoff_dfs_temp,test_probs=create_cutoff_pos_df(test,xgb_models,
#                     features_test,df[df.index.isin([2,22,70,100,178,200,486,
#                                                     488,525,604,672,681,747,
#                                                    812,1212,1392,1536])],
#                     imp_feats_count_list,is_compute_matches=False,
#                    test_ratio_limit=0.12,sim_cutoff_dfs=sim_cutoff_dfs_temp,
#                     low_high_record_condns=[(False,2,20)])
# cutoff_pos_df_temp

In [145]:
# %%time
# imp_feats_count_list = [30,40]
# sim_cutoff_dfs=None
# cutoff_pos_df_hash,sim_cutoff_dfs,test_probs_hash=create_cutoff_pos_df(test_hash,xgb_models,
#                     features_test,df[df['total']<=10],
#                     imp_feats_count_list,
#                    test_ratio_limit=0.12,sim_cutoff_dfs=sim_cutoff_dfs,
#                     low_high_record_condns=[(False,10,20),(False,2,20)])
# cutoff_pos_df_hash

In [146]:
# cutoff_pos_df_hash[cutoff_pos_df_hash['condn_val']==2]

In [147]:
# print(len(df))
# df_filt = df[(df['total']>=50)]
# print(len(df_filt))
# # df_filt[df_filt['ratio']==0.12]

In [148]:
# %%time
# imp_feats_count_list = [2,5,10,15,20,25,30,40]
# sim_cutoff_dfs=None
# cutoff_pos_df_hash,sim_cutoff_dfs,test_probs_hash=create_cutoff_pos_df(test_hash,xgb_models,
#                     features_test,df[(df['total']>=50)],
#                     imp_feats_count_list,
#                    test_ratio_limit=0.12,sim_cutoff_dfs=sim_cutoff_dfs,
#                     low_high_record_condns=[(True,50,10)])
# cutoff_pos_df_hash

In [149]:
%%time
imp_feats_count_list = [2,5,10,15,20,25,30,40]
sim_cutoff_dfs=None
min_normal_count=50
max_low_count = 5
cutoff_pos_df_hash,sim_cutoff_dfs,test_probs_hash=create_cutoff_pos_df(test_hash,xgb_models,
                    features_test,df[(df['total']<=max_low_count) | (df['total']>=min_normal_count)],
                    imp_feats_count_list,
                   test_ratio_limit=0.12,sim_cutoff_dfs=sim_cutoff_dfs,
                    low_high_record_condns=[(True,min_normal_count,10),
                                            (False,max_low_count,20),(False,2,20)])
cutoff_pos_df_hash

max_good_k=45

********************** Feature count 2 *****************
Generate probs completed
[['roll_mean_1_60', 'roll_42_nonce']]
list creation complete
******* Condition (True, 50, 10) for feature count 2 *****************
280
124
--- Top 5 data ---
     k  total  ratio_mean  ratio_std
0   28   65.0    0.092308   0.130543
1  107   67.0    0.089552   0.126646
2   30   81.0    0.080247   0.113486
3   62  119.0    0.079832   0.112899
4   63   70.0    0.078571   0.111117
--- Top 6 to 10 data ---
    k  total  ratio_mean  ratio_std
5  54   53.0    0.075472   0.106733
6  46   67.0    0.074627   0.105538
7  95  145.0    0.072414   0.102409
8  52  105.0    0.071429   0.101015
9  37  142.0    0.070423   0.099593
match_pos_1                1.0
match_total_top_5            2
match_total_top_10           3
match_ks_top_5        [28, 30]
Name: 0, dtype: object
******* Condition (False, 5, 20) for feature count 2 *****************
280
42
--- Top 5 data ---
     k  total  ratio_mean  ratio_std


,imp_feats_count,condn_max,condn_val,k_top_1,k_top_2,k_top_3,k_top_4,k_top_5,mean_top_1,mean_top_2,mean_top_3,mean_top_4,mean_top_5,match_pos_1,match_total_top_5,match_total_top_10,match_ks_top_5
0,2,True,50,28,107,30,62,63,0.092308,0.089552,0.080247,0.079832,0.078571,1.0,2,3,"[28, 30]"
1,2,False,5,5,169,170,11,2,0.166667,0.166667,0.100000,0.100000,0.000000,1.5,2,2,"[5, 11]"
2,2,False,2,139,161,164,166,171,0.000000,0.000000,0.000000,0.000000,0.000000,11.0,0,0,[]
3,5,True,50,24,44,54,95,35,0.100000,0.085714,0.083019,0.081379,0.081159,1.0,3,4,"[24, 44, 35]"
4,5,False,5,165,169,181,8,166,0.200000,0.133333,0.100000,0.100000,0.100000,4.0,1,3,[8]
5,5,False,2,166,181,139,161,164,0.100000,0.100000,0.000000,0.000000,0.000000,11.0,0,0,[]
6,10,True,50,24,35,54,44,109,0.068519,0.063768,0.060377,0.060204,0.059302,1.0,3,4,"[24, 35, 44]"
7,10,False,5,169,165,5,11,181,0.133333,0.133333,0.066667,0.060000,0.050000,3.0,2,4,"[5, 11]"
8,10,False,2,161,166,181,139,164,0.050000,0.050000,0.050000,0.000000,0.000000,11.0,0,0,[]
9,15,True,50,44,109,110,83,24,0.081633,0.079845,0.077778,0.076976,0.076543,1.0,2,3,"[44, 24]"


In [150]:
cutoff_pos_df_hash[cutoff_pos_df_hash['condn_val']==2]

,imp_feats_count,condn_max,condn_val,k_top_1,k_top_2,k_top_3,k_top_4,k_top_5,mean_top_1,mean_top_2,mean_top_3,mean_top_4,mean_top_5,match_pos_1,match_total_top_5,match_total_top_10,match_ks_top_5
2,2,False,2,139,161,164,166,171,0.000000,0.000000,0.000000,0.000000,0.000000,11.0,0,0,[]
5,5,False,2,166,181,139,161,164,0.100000,0.100000,0.000000,0.000000,0.000000,11.0,0,0,[]
8,10,False,2,161,166,181,139,164,0.050000,0.050000,0.050000,0.000000,0.000000,11.0,0,0,[]
11,15,False,2,181,139,161,166,171,0.066667,0.033333,0.033333,0.033333,0.033333,11.0,0,0,[]
14,20,False,2,181,139,171,161,166,0.075000,0.050000,0.050000,0.025000,0.025000,11.0,0,0,[]
17,25,False,2,139,181,161,166,171,0.080000,0.060000,0.040000,0.040000,0.040000,11.0,0,0,[]
20,30,False,2,139,181,161,164,166,0.066667,0.066667,0.050000,0.033333,0.033333,11.0,0,0,[]
23,40,False,2,161,181,139,166,171,0.100000,0.075000,0.062500,0.062500,0.062500,11.0,0,0,[]


In [155]:
all_k

[10, 130, 146, 161, 169, 174, 183, 186, 188, 192, 195]

In [151]:
%%time
# imp_feats_count_list = [2,5,10,15,20,25,30,40]
sim_cutoff_dfs_actual=None
cutoff_pos_df_actual,sim_cutoff_dfs_actual,test_probs=create_cutoff_pos_df(test,xgb_models,
                    features_test,df[df.index.isin(all_k)],
                    imp_feats_count_list,is_compute_matches=False,
                   test_ratio_limit=0.12,sim_cutoff_dfs=sim_cutoff_dfs_actual,
                    low_high_record_condns=[(False,2,20)])
cutoff_pos_df_actual

max_good_k=129

********************** Feature count 2 *****************
Generate probs completed
[['roll_mean_1_60', 'roll_42_nonce']]
list creation complete
******* Condition (False, 2, 20) for feature count 2 *****************
22
22
--- Top 5 data ---
     k  total  ratio_mean  ratio_std
0  192    2.0        0.25   0.353553
1   10    2.0        0.00   0.000000
2  130    2.0        0.00   0.000000
3  146    2.0        0.00   0.000000
4  161    2.0        0.00   0.000000
--- Top 6 to 10 data ---
      k  total  ratio_mean  ratio_std
5   169    2.0         0.0        0.0
6   174    2.0         0.0        0.0
7   183    2.0         0.0        0.0
8   186    2.0         0.0        0.0
9   188    2.0         0.0        0.0
10  195    2.0         0.0        0.0

********************** Feature count 5 *****************
Generate probs completed
[['roll_mean_1_60', 'roll_42_nonce', 'roll_60', 'roll_19_nonce', 'roll_mean_nonce']]
list creation complete
******* Condition (False, 2, 20) for feat

,imp_feats_count,condn_max,condn_val,k_top_1,k_top_2,k_top_3,k_top_4,k_top_5,mean_top_1,mean_top_2,mean_top_3,mean_top_4,mean_top_5,match_pos_1,match_total_top_5,match_total_top_10,match_ks_top_5
0,2,False,2,192,10,130,146,161,0.250,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN
1,5,False,2,10,192,130,146,161,0.100,0.100000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN
2,10,False,2,10,169,146,183,188,0.100,0.100000,0.050000,0.050000,0.050000,NaN,NaN,NaN,NaN
3,15,False,2,10,130,146,169,174,0.100,0.066667,0.066667,0.066667,0.033333,NaN,NaN,NaN,NaN
4,20,False,2,10,130,146,174,188,0.075,0.075000,0.075000,0.075000,0.075000,NaN,NaN,NaN,NaN
5,25,False,2,146,10,174,188,130,0.100,0.080000,0.080000,0.080000,0.060000,NaN,NaN,NaN,NaN
6,30,False,2,146,130,174,10,186,0.100,0.083333,0.083333,0.066667,0.066667,NaN,NaN,NaN,NaN
7,40,False,2,174,146,130,169,186,0.100,0.087500,0.075000,0.075000,0.075000,NaN,NaN,NaN,NaN


In [157]:
temp=df.loc[[10,130,146,192,174,188,161]]
temp['diff']=df.loc[10,'cutoff']-temp['cutoff']
temp

,cutoff,cutoff_2,ratio,total,diff
10,0.382469,0.383836,0.2,20,0.000000
130,0.270352,0.271720,0.0,0,0.112116
146,0.330512,0.331880,0.0,3,0.051956
192,0.303167,0.304534,0.0,0,0.079302
174,0.307269,0.308636,0.0,0,0.075200
188,0.297698,0.299065,0.0,0,0.084771
161,0.320942,0.322309,0.0,2,0.061527


In [153]:
# k_sel,test_sel,cutoff_pos_best =get_test_sel_cutoff_pos(test_probs,cutoff_pos_df_hash,cutoff_pos_df_actual)
# imp_feats_count_list = [2,5,10,15,20,25]

k_sel,test_sel,top_mean_best =get_test_sel_cutoff_pos(test_probs,xgb_models,features_test,
                                                      df,
                            cutoff_pos_df_hash,
                            is_min_total=False,total_cri=2,top_n=10,
                            sim_cutoff_dfs=sim_cutoff_dfs_actual,
                            imp_feats_count_list=imp_feats_count_list)
print(f'{k_sel=}')
print('cutoff best')
print(top_mean_best)
print(' ** selected test **')
test_sel[['k','client_seed','probs']]

 ........... Overall Positions of Features .........
   imp_feats_count  match_pos_1  match_total_top_5  match_total_top_10
0                2     4.500000           1.333333            1.666667
1                5     5.333333           1.333333            2.333333
2               10     5.000000           1.666667            2.666667
3               15     5.333333           1.000000            3.333333
4               20     5.000000           0.666667            4.000000
5               25     5.000000           1.666667            3.333333
6               30     5.166667           1.666667            3.000000
7               40     5.166667           1.333333            3.000000
 ........... Low Positions of Features .........
   imp_feats_count  match_pos_1  match_total_top_5  match_total_top_10
0                2         6.25                1.0                 1.0
1                5         7.50                0.5                 1.5
2               10         7.00               

,k,client_seed,probs
2657,146,woxpwoxpwoxpwoxpwoxp93e59a38c595212fa5b95439ff4bdaecdb87f9aa7113e3cce4cd781ff112551d,0.331719
2983,146,woxpwoxpwoxpwoxpwoxp549fd5dce873e1b1c023d784e5963c1b7394a5ad043f0ea21316eb41201e5bfc,0.330993
6228,10,woxpwoxpwoxpwoxpwoxp63a45b574bced9fd42be78db35abd2e02013cc9d10377e74f4cdfb6478e51c50,0.382476
6864,10,woxpwoxpwoxpwoxpwoxp18ee9a347ebb276f80fc9c7b4de104bcdb71fc1706ace5d3aa6cd2ee1d976771,0.382990
1117,174,woxpwoxpwoxpwoxpwoxpaabe3cb0b4eb6dffc77723c8a85536cbec0e92853f9e8b2fa1f371e1eed69396,0.307603
1455,174,woxpwoxpwoxpwoxpwoxp41165bc5bd2a349fddd109d702d372116eff8b6b4cf7bc1ab59fad7ed85fd216,0.308061
122,188,woxpwoxpwoxpwoxpwoxpf7be5771ccfedf1c72b734e09ccc2e076dd944a178afbad3c106ca3b30c61963,0.298555
5150,188,woxpwoxpwoxpwoxpwoxp24f9d068ae89851ad8d1fb08363c5d1f0ac0e6e00e2ec40dc4b020635037e072,0.298529
2012,130,woxpwoxpwoxpwoxpwoxpf6d59ab7a2adf220c9e90990441ad8e0b78e712e5752453ba7d733862abf49ca,0.271278
4683,130,woxpwoxpwoxpwoxpwoxp94b3e71b54a52abdd37b7516a7fb9568747a0baa91c45af22d9b6d77810dfab5,0.271584


In [159]:
test_sel=None
for k_sel in [130,146]:
    row = df.iloc[k_sel]
#     print(row)
    mask = (test_probs['probs']>=row['cutoff']) & (test_probs['probs']<=row['cutoff_2'])
    cur_test_sel = test_probs[mask]
    cur_test_sel['k']=k_sel
    if test_sel is None:
        test_sel = cur_test_sel
    else:
        test_sel = pd.concat([test_sel,cur_test_sel],axis=0)
    print('Test Size:',len(test_sel))
test_sel

Test Size: 2
Test Size: 4


,roll_actual,seed,hash,client_seed,roll_0,roll_1,roll_2,roll_3,roll_4,roll_5,roll_6,roll_7,roll_8,roll_9,roll_10,roll_11,roll_12,roll_13,roll_14,roll_15,roll_16,roll_17,roll_18,roll_19,roll_20,roll_21,roll_22,roll_23,roll_24,roll_25,roll_26,roll_27,roll_28,roll_29,roll_30,roll_31,roll_32,roll_33,roll_34,roll_35,roll_36,roll_37,roll_38,roll_39,roll_40,roll_41,roll_42,roll_43,roll_44,roll_45,roll_46,roll_47,roll_48,roll_49,roll_50,roll_51,roll_52,roll_53,roll_54,roll_55,roll_56,roll_57,roll_58,roll_59,roll_60,count_lt_1000,count_gt_9000,count_gt_8000,count_gt_7000,count_gt_6000,count_gt_5000,count_gt_4000,count_gt_3000,count_gt_2000,count_gt_1000,count_hi,count_lo,count_lo_hi,count_gt_9000_k,count_lt_1000_k,total_dig_9,total_dig_0,roll_mean,roll_std,count_lt_1000_25_50,count_gt_9000_25_50,count_gt_8000_25_50,count_gt_7000_25_50,count_gt_6000_25_50,count_gt_5000_25_50,count_gt_4000_25_50,count_gt_3000_25_50,count_gt_2000_25_50,count_gt_1000_25_50,count_hi_25_50,count_lo_25_50,count_lo_hi_25_50,count_gt_9000_k_25_50,count_lt_1000_k_25_50,total_dig_9_25_50,total_dig_0_25_50,roll_mean_25_50,roll_std_25_50,count_lt_1000_50_60,count_gt_9000_50_60,count_gt_8000_50_60,count_gt_7000_50_60,count_gt_6000_50_60,count_gt_5000_50_60,count_gt_4000_50_60,count_gt_3000_50_60,count_gt_2000_50_60,count_gt_1000_50_60,count_hi_50_60,count_lo_50_60,count_lo_hi_50_60,count_gt_9000_k_50_60,count_lt_1000_k_50_60,total_dig_9_50_60,total_dig_0_50_60,roll_mean_50_60,roll_std_50_60,count_lt_1000_1_60,count_gt_9000_1_60,count_gt_8000_1_60,count_gt_7000_1_60,count_gt_6000_1_60,count_gt_5000_1_60,count_gt_4000_1_60,count_gt_3000_1_60,count_gt_2000_1_60,count_gt_1000_1_60,count_hi_1_60,count_lo_1_60,count_lo_hi_1_60,count_gt_9000_k_1_60,count_lt_1000_k_1_60,total_dig_9_1_60,total_dig_0_1_60,roll_mean_1_60,roll_std_1_60,roll_0_nonce,roll_1_nonce,roll_2_nonce,roll_3_nonce,roll_4_nonce,roll_5_nonce,roll_6_nonce,roll_7_nonce,roll_8_nonce,roll_9_nonce,roll_10_nonce,roll_11_nonce,roll_12_nonce,roll_13_nonce,roll_14_nonce,roll_15_nonce,roll_16_nonce,roll_17_nonce,roll_18_nonce,roll_19_nonce,roll_20_nonce,roll_21_nonce,roll_22_nonce,roll_23_nonce,roll_24_nonce,roll_25_nonce,roll_26_nonce,roll_27_nonce,roll_28_nonce,roll_29_nonce,roll_30_nonce,roll_31_nonce,roll_32_nonce,roll_33_nonce,roll_34_nonce,roll_35_nonce,roll_36_nonce,roll_37_nonce,roll_38_nonce,roll_39_nonce,roll_40_nonce,roll_41_nonce,roll_42_nonce,roll_43_nonce,roll_44_nonce,roll_45_nonce,roll_46_nonce,roll_47_nonce,roll_48_nonce,roll_49_nonce,roll_50_nonce,roll_51_nonce,roll_52_nonce,roll_53_nonce,roll_54_nonce,roll_55_nonce,roll_56_nonce,roll_57_nonce,roll_58_nonce,roll_59_nonce,roll_60_nonce,count_lt_1000_nonce,count_gt_9000_nonce,count_gt_8000_nonce,count_gt_7000_nonce,count_gt_6000_nonce,count_gt_5000_nonce,count_gt_4000_nonce,count_gt_3000_nonce,count_gt_2000_nonce,count_gt_1000_nonce,count_hi_nonce,count_lo_nonce,count_lo_hi_nonce,count_gt_9000_k_nonce,count_lt_1000_k_nonce,total_dig_9_nonce,total_dig_0_nonce,roll_mean_nonce,roll_std_nonce,count_lt_1000_25_50_nonce,count_gt_9000_25_50_nonce,count_gt_8000_25_50_nonce,count_gt_7000_25_50_nonce,count_gt_6000_25_50_nonce,count_gt_5000_25_50_nonce,count_gt_4000_25_50_nonce,count_gt_3000_25_50_nonce,count_gt_2000_25_50_nonce,count_gt_1000_25_50_nonce,count_hi_25_50_nonce,count_lo_25_50_nonce,count_lo_hi_25_50_nonce,count_gt_9000_k_25_50_nonce,count_lt_1000_k_25_50_nonce,total_dig_9_25_50_nonce,total_dig_0_25_50_nonce,roll_mean_25_50_nonce,roll_std_25_50_nonce,count_lt_1000_50_60_nonce,count_gt_9000_50_60_nonce,count_gt_8000_50_60_nonce,count_gt_7000_50_60_nonce,count_gt_6000_50_60_nonce,count_gt_5000_50_60_nonce,count_gt_4000_50_60_nonce,count_gt_3000_50_60_nonce,count_gt_2000_50_60_nonce,count_gt_1000_50_60_nonce,...,roll_actual_14,roll_actual_15,roll_actual_16,roll_actual_17,roll_actual_18,roll_actual_19,roll_actual_20,roll_actual_21,roll_actual_22,roll_actual_23,roll_actual_24,roll_actual_25,roll_actual_26,roll_actual_27,roll_actual_28,roll_actual_29,roll

In [172]:
imp_feats = [col for col in imp_df['feature'].values if (col.startswith('roll_'))][:10] 
test_sel[['k']+imp_feats]

,k,roll_mean_1_60,roll_42_nonce,roll_60,roll_19_nonce,roll_mean_nonce,roll_mean,roll_mean_25_50_nonce,roll_mean_1_60_nonce,roll_7_nonce,roll_32_nonce
2012,130,4302.066667,9624,9386,5737,5313.9,3959.95,5269.384615,4814.133333,8607,7325
4683,130,5999.000000,1275,9665,7312,6164.5,5416.00,5277.230769,5704.983333,7651,6917
2657,146,4368.300000,9696,9990,7769,5958.4,3656.20,5253.230769,5364.416667,3299,6928
2983,146,4582.866667,7868,9864,6686,5012.5,4039.75,4458.269231,5040.666667,301,4690


In [175]:
sim_hash_group[sim_hash_group['k']==24]

,k,hash_group,ratio_mean,ratio_std
5,24,0.0,0.130841,0.0


In [176]:
sim_cutoff_df[sim_cutoff_df['k']==24]

,k,ratio,total,hash_no,hash_group
48,24,0.130841,214,roll_21_nonce,0.0
49,24,0.130841,214,roll_37_nonce,0.0


In [586]:
def compute_total(data,cutoffs,cols):
    mask = None
    for cutoff,col in zip(cutoffs,cols):
        cur_mask =(data[col]>=cutoff) 
        if mask is None:
            mask = cur_mask
        else:
            mask = mask & cur_mask
    data_filt = data[mask]
    total = len(data_filt)
    return total,data_filt

In [587]:
cols=['roll_20_nonce','roll_14_nonce','roll_3_nonce','roll_16_nonce','roll_11_nonce']
cols=['roll_43_nonce','roll_56_nonce','roll_31_nonce','roll_20_nonce','roll_45_nonce']
cols=['roll_31_nonce','roll_52_nonce','roll_15_nonce','roll_45_nonce','roll_2_nonce']
cols=['roll_31_nonce','roll_52_nonce']
cols=['roll_52_nonce','roll_31_nonce','roll_24_nonce','roll_49_nonce','roll_41_nonce']
cols=['roll_mean_25_50_nonce','roll_52_nonce','roll_31_nonce']
cols=['roll_52_nonce','roll_31_nonce']
cols=['roll_46_nonce','roll_25_nonce']
cols=['roll_42_nonce','roll_37_nonce']

# cols=['roll_20_nonce','roll_15_nonce','roll_13_nonce','roll_12_nonce','roll_18_nonce']
# cols=['roll_20_nonce','roll_15_nonce','roll_13_nonce']
# cols=['roll_6_nonce','roll_11_nonce','roll_12_nonce','roll_10_nonce',]




In [588]:
def get_cutoff(data,data_hash,col,quantile):
    mask=create_target_mask(data_hash)
#     cutoff = data[col].mean()-data_hash[col].mean()+data_hash[mask][col].mean()
#     cutoff = data_hash[mask][col].quantile(0.5)
    cutoff = data[col].mean()-data_hash[col].mean()+data_hash[mask][col].quantile(quantile)
    return cutoff



best_ratio =0
best_success = 0
best_total = 0
best_cutoffs = []
best_quantile = 0
best_actual = test_filt
# for quantile in np.linspace(0.01,0.95,50):
for quantile in np.linspace(0.01,0.95,50):
    cutoffs=[]
    for col in cols:
        #use entire hash to determine cutoff
        cutoff = get_cutoff(test,test_hash,col,quantile)
        cutoffs.append(cutoff)
    
#     print(cutoffs)
    total,filt_inter = compute_total(test_filt_hash,cutoffs,cols)
    mask2=create_target_mask(filt_inter)
#     mask2 = (filt_inter['roll_actual']>=9000)
    success = len(filt_inter[mask2])
    
    total_actual,filt_inter_actual = compute_total(test_filt,cutoffs,cols)
    mask2=create_target_mask(filt_inter_actual)

#     mask2 = (filt_inter_actual['roll_actual']>=9000)
    
    if total_actual==0:
        success_actual=0
        ratio_actual=0
    else:
        success_actual = len(filt_inter_actual[mask2])
        ratio_actual = success_actual / total_actual
    print('\nquantile:',quantile)
    print('Actuals:',ratio_actual,success_actual,total_actual)
    
    if total==0:
        ratio=0
    else:
        ratio = success / total
    print('Hash:',ratio,success,total)
#     print(quantile,cutoffs)
    print(total,total_actual)
#     print(total,success)
    if (total==0) or (success==0) : #or (total_actual<1):
        break

#     print(ratio,success,total)
    if ratio >= best_ratio:
        best_ratio = ratio
        best_success = success
        best_total = total
        best_cutoffs = cutoffs
        best_quantile = quantile
        best_actual = filt_inter_actual
print(best_quantile)        
print(best_ratio,best_success,best_total)        
print(best_cutoffs)
# print(test_filt[mask]['roll_actual'].describe())


quantile: 0.01
Actuals: 0.05 1 20
Hash: 0.16666666666666666 2 12
12 20

quantile: 0.02918367346938775
Actuals: 0.05263157894736842 1 19
Hash: 0.1 1 10
10 19

quantile: 0.04836734693877551
Actuals: 0.05263157894736842 1 19
Hash: 0.1 1 10
10 19

quantile: 0.06755102040816326
Actuals: 0.058823529411764705 1 17
Hash: 0.1 1 10
10 17

quantile: 0.086734693877551
Actuals: 0.058823529411764705 1 17
Hash: 0.1111111111111111 1 9
9 17

quantile: 0.10591836734693875
Actuals: 0.0625 1 16
Hash: 0.1111111111111111 1 9
9 16

quantile: 0.12510204081632653
Actuals: 0.0625 1 16
Hash: 0.1111111111111111 1 9
9 16

quantile: 0.14428571428571427
Actuals: 0.0625 1 16
Hash: 0.1111111111111111 1 9
9 16

quantile: 0.16346938775510203
Actuals: 0.0625 1 16
Hash: 0.1111111111111111 1 9
9 16

quantile: 0.1826530612244898
Actuals: 0.0625 1 16
Hash: 0.1111111111111111 1 9
9 16

quantile: 0.20183673469387753
Actuals: 0.0625 1 16
Hash: 0.125 1 8
8 16

quantile: 0.2210204081632653
Actuals: 0.0625 1 16
Hash: 0.125 1 8
8 

In [589]:
mask2=create_target_mask(best_actual)
success = len(best_actual[mask2])
total = len(best_actual)
ratio = success / total
print(ratio,success,total)

0.05 1 20


In [559]:
test_sel = best_actual.copy()

In [261]:


def get_actual_based_data(data,data_hash,data_filt,cols,total_cutoff=3):

    best_filt_inter = data
    best_total= 0
    best_quantile = 0
    for quantile in np.linspace(0.01,0.95,50):
        cutoffs=[]
        for col in cols:
            cutoff = get_cutoff(data,data_hash,col,quantile)
        #     cutoff = get_cutoff(test,test_hash,col)
            cutoffs.append(cutoff)
#         print(quantile,cutoffs)

        total,filt_inter=compute_total(data_filt,cutoffs,cols)
        if total<total_cutoff:
            break
        best_filt_inter = filt_inter
        best_total = total
        best_quantile = quantile
        
    return best_filt_inter,best_total,best_quantile

In [590]:
for cutoff in range(0,11):

    best_filt_inter,best_total,best_quantile=get_actual_based_data(test,test_hash,test_filt,cols,
                                                                        total_cutoff=cutoff)
#     print('quantile:',best_quantile)
    mask2=create_target_mask(best_filt_inter)
    # mask2 = (best_filt_inter['roll_actual']>=9000)
    success = len(best_filt_inter[mask2])
    if best_total==0:
        ratio=0
        success=0
    else:
        success = len(best_filt_inter[mask2])
        ratio = success / best_total
    print(cutoff, ratio,success,best_total)
    # print(test_filt[mask]['roll_actual'].describe())

0 0 0 0
1 0.0 0 1
2 0.0 0 2
3 0.0 0 3
4 0.0 0 5
5 0.0 0 5
6 0.0 0 6
7 0.0 0 8
8 0.0 0 8
9 0.0 0 9
10 0.08333333333333333 1 12


In [ ]:
test_sel =best_filt_inter.copy()

In [ ]:
best_filt_inter,best_total,best_quantile=get_actual_based_data(test_filt,test_filt_hash,test_filt,cols,
                                                              total_cutoff=4)
print('quantile:',best_quantile)
mask2=create_target_mask(best_filt_inter)
# mask2 = (best_filt_inter['roll_actual']>=9000)
success = len(best_filt_inter[mask2])
if best_total==0:
    ratio=0
    success=0
else:
    success = len(best_filt_inter[mask2])
    ratio = success / best_total
print(ratio,success,best_total)
# print(test_filt[mask]['roll_actual'].describe())

In [ ]:
# mask_actual = (test_probs['probs']>=row['cutoff']) & (test_probs['probs']<=row['cutoff_2'])
# idx= test_probs[mask_actual].index
# mask_hash = test_hash_probs.index.isin(idx)
# test_filt_hash_pos = test_hash_probs.loc[mask_hash]

mask_hash = (test_hash_probs['probs']>=row['cutoff']) & (test_hash_probs['probs']<=row['cutoff_2'])
idx= test_hash_probs[mask_hash].index
mask_actual = test_probs.index.isin(idx)
test_filt_pos = test_probs.loc[mask_actual]

best_filt_inter,best_total,best_quantile=get_actual_based_data(test,test_hash,
                                                               test_filt_pos,cols,
                                                              total_cutoff=37)
print('quantile:',best_quantile)
mask2=create_target_mask(best_filt_inter)
# mask2 = (best_filt_inter['roll_actual']>=9000)
if best_total==0:
    ratio=0
    success=0
else:
    success = len(best_filt_inter[mask2])
    ratio = success / best_total
print(ratio,success,best_total)
# print(test_filt[mask]['roll_actual'].describe())

In [ ]:
best_filt_inter

In [ ]:
test_sel=best_filt_inter.copy()

In [ ]:
# analysis_df_copy  = analysis_df.copy()

In [166]:
def gen_analysis(nonce,train_manual,test_manual,test,models,tr_last_cutoffs,val_cutoffs,
                multi_models=True,file_suffix="",save_files=True,gen_probs=True,
                analysis_folder="analysis"):
    if CONT_ANALYSIS:
        analysis_df=pd.read_csv(f'data/{analysis_folder}/analysis_df_{nonce}{file_suffix}.csv')
    else:
        analysis_df=None
    print(f'Analysis for PATTERN_99:{PATTERN_99} HIGH_ANALYSIS:{HIGH_ANALYSIS}')
    for zero_cutoff in range(0,6):
        analysis_df=save_train_stats(train_manual,nonce,zero_cutoff,PATTERN_99,HIGH_ANALYSIS,0,
                            analysis_df=analysis_df)
    print('Train completed')
    analysis_df = save_test_stats(analysis_df, test_manual,PATTERN_99,HIGH_ANALYSIS,is_prod=False)
    print('Test completed')   
    
    if save_files:
        joblib.dump(xgb_models,f'data/models/models_{nonce}{file_suffix}.dump')
        print('Models Save completed')   

    features_test = [col for col in features if col not in ['target','probs']]
    
    if gen_probs:
        test=generate_probs(True,models,test,features_test)
    else:
        test['probs'] = np.load(f'data/probs/probs_{nonce}{file_suffix}.npy')

    if save_files:
        np.save(f'data/probs/probs_{nonce}{file_suffix}.npy',test['probs'].values)
    print('Probability Prediction completed')   

    test_filt_all,analysis_df = save_test_ml_probs(analysis_df,test,
                                                models,
                                               np.mean(np.array(tr_last_cutoffs)),
                                                 np.mean(np.array(val_cutoffs)), 
                                                PATTERN_99,HIGH_ANALYSIS,multi_models=multi_models,
                                                  gen_probs=False)
    print(len(analysis_df))
    analysis_df.to_csv(f'data/{analysis_folder}/analysis_df_{nonce}{file_suffix}.csv',index=False)  
    print('Analysis Generation completed')  
    return analysis_df,test

In [131]:
test['roll_actual'].head()

0    1119
1    1638
2    2244
3    5933
4    3730
Name: roll_actual, dtype: int64

In [167]:
%%time
analysis_df,test = gen_analysis(nonce,train_manual,test_manual,test,xgb_models,tr_last_cutoffs,val_cutoffs,
                multi_models=True,file_suffix="",
                save_files=True,analysis_folder="analysis_25_to_50_seeds"
#                 gen_probs=False
                    )
analysis_df.head()

Analysis for PATTERN_99:False HIGH_ANALYSIS:False
Train completed
Test completed
Models Save completed
Probability Prediction completed
No Record for zero_col_cutoff:4 digit:7 cutoff:5 after_val:8
No Record for zero_col_cutoff:5 digit:7 cutoff:4 after_val:8
No Record for zero_col_cutoff:5 digit:7 cutoff:5 after_val:8
No Record for zero_col_cutoff:5 digit:6 cutoff:5 after_val:8
Function time:7.605552673339844
972
Analysis Generation completed
CPU times: user 14.8 s, sys: 168 ms, total: 14.9 s
Wall time: 12 s


,nonce,zero_col_cutoff,pattern_99,target_high,digit,cutoff,afterval,tr_ratio,te_ratio,aft_all,aft_1,aft_2,pr_mean,pr_med,pr_1,pr_2,pr_3,pr_4,pr_5,pr_tr_cutoff,pr_val_cutoff,tr_total,tr_success,te_total,te_success,te_pr_1_count,te_pr_2_count,te_pr_3_count,te_pr_4_count,te_pr_5_count,te_mean_count,te_med_count,te_tr_count,te_val_count
0,2649,0,False,False,9,0,25,0.0,0.0,0.0,0.0,0.0,13.636364,15.000000,12.345679,12.345679,12.345679,12.000000,11.111111,11.538462,12.500000,0.0,0.0,0.0,0.0,81.0,81.0,81.0,75.0,18.0,44.0,40.0,78.0,80.0
1,2649,0,False,False,9,0,8,0.0,0.0,0.0,0.0,0.0,8.333333,7.692308,11.538462,11.538462,11.538462,9.090909,0.000000,8.695652,12.000000,0.0,0.0,0.0,0.0,26.0,26.0,26.0,22.0,3.0,12.0,13.0,23.0,25.0
2,2649,0,False,False,9,0,10,0.0,0.0,0.0,0.0,0.0,6.896552,7.407407,10.909091,10.909091,10.909091,10.000000,0.000000,9.615385,11.111111,0.0,0.0,0.0,0.0,55.0,55.0,55.0,50.0,9.0,29.0,27.0,52.0,54.0
3,2649,0,False,False,9,1,25,0.0,0.0,0.0,0.0,0.0,11.363636,11.965812,11.489362,11.489362,11.489362,10.810811,2.380952,10.869565,11.538462,0.0,0.0,0.0,0.0,235.0,235.0,235.0,222.0,42.0,132.0,117.0,230.0,234.0
4,2649,0,False,False,9,1,8,0.0,0.0,0.0,0.0,0.0,17.857143,20.408163,17.171717,17.171717,17.171717,17.391304,0.000000,16.666667,17.346939,0.0,0.0,0.0,0.0,99.0,99.0,99.0,92.0,10.0,56.0,49.0,96.0,98.0


In [168]:
%%time
analysis_df_hash,test_hash = gen_analysis(nonce,train_manual,test_manual_hash,test_hash,
                                xgb_models,tr_last_cutoffs,val_cutoffs,
                                multi_models=True,file_suffix="_hash",
                                save_files=True,
                                         )
analysis_df_hash.head()

Analysis for PATTERN_99:False HIGH_ANALYSIS:False
Train completed
Test completed
Models Save completed
Probability Prediction completed
No Record for zero_col_cutoff:5 digit:7 cutoff:5 after_val:8
Function time:6.502852916717529
972
Analysis Generation completed
CPU times: user 13.8 s, sys: 204 ms, total: 14 s
Wall time: 10.9 s


,nonce,zero_col_cutoff,pattern_99,target_high,digit,cutoff,afterval,tr_ratio,te_ratio,aft_all,aft_1,aft_2,pr_mean,pr_med,pr_1,pr_2,pr_3,pr_4,pr_5,pr_tr_cutoff,pr_val_cutoff,tr_total,tr_success,te_total,te_success,te_pr_1_count,te_pr_2_count,te_pr_3_count,te_pr_4_count,te_pr_5_count,te_mean_count,te_med_count,te_tr_count,te_val_count
0,2649,0,False,False,9,0,25,0.0,0.0,0.0,0.0,0.0,9.615385,8.695652,7.608696,7.608696,7.608696,7.954545,0.000000,7.777778,7.692308,0.0,0.0,0.0,0.0,92.0,92.0,92.0,88.0,20.0,52.0,46.0,90.0,91.0
1,2649,0,False,False,9,0,8,0.0,0.0,0.0,0.0,0.0,11.111111,12.500000,9.375000,9.375000,9.375000,9.375000,0.000000,9.375000,9.375000,0.0,0.0,0.0,0.0,32.0,32.0,32.0,32.0,6.0,18.0,16.0,32.0,32.0
2,2649,0,False,False,9,0,10,0.0,0.0,0.0,0.0,0.0,5.882353,6.666667,6.666667,6.666667,6.666667,6.779661,0.000000,6.779661,6.779661,0.0,0.0,0.0,0.0,60.0,60.0,60.0,59.0,13.0,34.0,30.0,59.0,59.0
3,2649,0,False,False,9,1,25,0.0,0.0,0.0,0.0,0.0,10.156250,7.964602,10.176991,10.176991,10.176991,10.628019,10.526316,10.138249,10.360360,0.0,0.0,0.0,0.0,226.0,226.0,226.0,207.0,38.0,128.0,113.0,217.0,222.0
4,2649,0,False,False,9,1,8,0.0,0.0,0.0,0.0,0.0,5.882353,6.818182,5.617978,5.617978,5.617978,6.097561,14.285714,5.813953,5.747126,0.0,0.0,0.0,0.0,89.0,89.0,89.0,82.0,14.0,51.0,44.0,86.0,87.0


In [ ]:
# %%time
# if CONT_ANALYSIS:
#     analysis_df=pd.read_csv(f'data/analysis_df_{nonce}.csv')
# else:
#     analysis_df=None
# print(f'Analysis for PATTERN_99:{PATTERN_99} HIGH_ANALYSIS:{HIGH_ANALYSIS}')
# for zero_cutoff in range(0,6):
#     analysis_df=save_train_stats(train_manual,nonce,zero_cutoff,PATTERN_99,HIGH_ANALYSIS,0,
#                         analysis_df=analysis_df)
# print('Train completed')
# analysis_df = save_test_stats(analysis_df, test_manual,PATTERN_99,HIGH_ANALYSIS,is_prod=True)
# print('Test completed')

In [ ]:
# %%time

# # model_no=5
# # analysis_df = save_test_ml_probs(analysis_df,test,
# #                                 xgb_models[model_no],np.mean(np.array(tr_last_cutoffs)),
# #                                 np.mean(np.array(val_cutoffs)))


# #Save the Models
# joblib.dump(xgb_models,f'models_{nonce}.dump')

# features_test = [col for col in features if col not in ['target','probs']
# test=generate_probs(True,xgb_models,test,features_test)
# np.save(f'probs/probs_{nonce}.npy',test['probs'].values)
                 
# test_filt_all,analysis_df = save_test_ml_probs(analysis_df,test,
#                                             xgb_models,
#                                            np.mean(np.array(tr_last_cutoffs)),
#                                              np.mean(np.array(val_cutoffs)), 
#                                             PATTERN_99,HIGH_ANALYSIS,multi_models=True,
#                                               gen_probs=False)
# print(len(analysis_df))
# analysis_df.to_csv(f'data/analysis/analysis_df_{nonce}.csv',index=False)

In [ ]:
# xgb_models= joblib.load(f'data/models/models_{nonce}.dump')
# for model in xgb_models:
#     temp_probs = model.predict_proba(test[0:5][features_test])
#     print(temp_probs)

In [ ]:
# len(analysis_df[(analysis_df['pattern_99']==True) & (analysis_df['target_high']==False)])

In [ ]:
# analysis_df=pd.read_csv(f'data/analysis/analysis_df_2582.csv')

In [169]:
def report_exceptional_teratio(analysis_df,digitlist,
                     ratio_cutoff,count_cutoff_min,count_cutoff_max,
                     pattern,bHighCheck,
                    zerocolcutoffs,is_ratio_cri_lessthan=False,
                               is_aftval_opp=False,is_aftval_sum=False):
    mask1 = analysis_df['digit'].isin(digitlist)
    if is_ratio_cri_lessthan:
        mask2= (analysis_df['tr_ratio']<ratio_cutoff) 
        if is_aftval_opp:
            mask2= mask2 & ((analysis_df['aft_all']>=ratio_cutoff) | (analysis_df['aft_1']>=ratio_cutoff) | (analysis_df['aft_2']>=ratio_cutoff)) 
    else:
        mask2=  (analysis_df['tr_ratio']>=ratio_cutoff) 
        if is_aftval_opp:
            mask2= mask2 & ((analysis_df['aft_all']<ratio_cutoff) | (analysis_df['aft_1']<ratio_cutoff) | (analysis_df['aft_2']<ratio_cutoff)) \
    
        
    mask= mask1 & mask2 \
             & ((analysis_df['aft_all']!=0) & (analysis_df['aft_1']!=0) & (analysis_df['aft_2']!=0)) \
                & (analysis_df['te_total']>=count_cutoff_min) \
                & (analysis_df['te_total']<=count_cutoff_max) \
                & (analysis_df['pattern_99']==pattern) \
                & (analysis_df['target_high']==bHighCheck) \
                & (analysis_df['zero_col_cutoff'].isin(zerocolcutoffs))
        
    if is_aftval_sum:
        mask= mask & ((analysis_df['aft_all']+analysis_df['aft_1']+analysis_df['aft_2'])>=36) \

    filtered = analysis_df[mask]
    
    records  = len(filtered)
    cols = ['tr_ratio','tr_total','te_total']
    
    return filtered

In [ ]:
# %%time

# suffix='_hash'
# digit_list=[1,2,3,4]
# # digit_list=[9,8,7,5]
# zero_col_cutoff_list=[0,1]

# cols = ['te_ratio','pr_mean','pr_med','pr_1','pr_2','pr_3','pr_4','pr_5',
#        'pr_tr_cutoff','pr_val_cutoff']
# analysis_list_hash=[]
# mean_list=[]
# model_means=[]
# mask = analysis_df['digit'].isin(digit_list)
# mask = mask & analysis_df['zero_col_cutoff'].isin(zero_col_cutoff_list)
# cutoffs = [(13,20),(13,50),(13,100),(14,20),
#            (14,100),(14,50),(15,20),(15,50),(15,100)]
# cutoffs =[(18,3)]

# # if CONT_ANALYSIS:
# #     analysis_mean_df=pd.read_csv(f'data/analysis/analysis_mean_df_{nonce}{suffix}.csv')
# # else:
# #     analysis_mean_df = None

# analysis_mean_df_hash =None    
# mean_df_hash=None

# for (ratio_cutoff,count_cutoff) in cutoffs:

#     filtered = report_exceptional_teratio(analysis_df_hash,digit_list,
#                      ratio_cutoff,count_cutoff,10000,
#                      PATTERN_99,HIGH_ANALYSIS,zero_col_cutoff_list,is_aftval_sum=True,
#                                           is_aftval_opp=True,
#                                          )
#     print(len(filtered))
# #     test_filt_all,filtered = save_test_ml_probs(filtered,test_hash,
# #                                     xgb_models,
# #                                    np.mean(np.array(tr_last_cutoffs)),
# #                                      np.mean(np.array(val_cutoffs)), 
# #                                   PATTERN_99,HIGH_ANALYSIS,
# #                                   gen_probs=False,
# #                                   multi_models=True )
#     analysis_list_hash.append(filtered)
#     records  = len(filtered)
# #     print('Records:',records)
# #     cur_mean = filtered[cols].mean(axis=0)
#     cur_mean = pd.Series()
#     pr_count_dict = {'pr_1':'te_pr_1_count','pr_2':'te_pr_2_count','pr_3':'te_pr_3_count',
#                        'pr_4':'te_pr_4_count','pr_5':'te_pr_5_count',
#                       'pr_mean':'te_mean_count','pr_med':'te_med_count',
#                      'pr_tr_cutoff':'te_tr_count','pr_val_cutoff':'te_val_count',
#                      'te_ratio':'te_total'}
    
#     for pr_col,pr_count_col in pr_count_dict.items():
#         cur_mean.loc[pr_col]= (filtered[pr_col]*filtered[pr_count_col]).round(0).sum() / filtered[pr_count_col].sum()
    
#     model_mean = cur_mean.mean()
#     te_total_sum = filtered['te_total'].sum(axis=0)
# #     model_mean = filtered[cols].mean(axis=None)
# #     print('Overall mean:',model_mean)
# #     print(cur_mean)

#     cur_mean.loc['overall_mean']=model_mean
#     cur_mean.loc['ratio_cutoff']=ratio_cutoff
#     cur_mean.loc['total_cutoff']=count_cutoff
#     cur_mean.loc['records']=records
#     cur_mean.loc['te_total_sum']=te_total_sum

#     if mean_df_hash is None:
#         mean_df_hash = pd.DataFrame({'temp':cur_mean})    
#     else:
#         mean_df_hash=pd.concat([mean_df_hash,cur_mean],axis=1)

# mean_df_hash=mean_df_hash.transpose().reset_index(drop=True)
# mean_df_hash['nonce']=nonce
# mean_df_hash['pattern_99']=PATTERN_99
# mean_df_hash['target_high']=HIGH_ANALYSIS
# key_cols = ['nonce','pattern_99','target_high','ratio_cutoff','total_cutoff', 
#             'records','te_total_sum','te_ratio','pr_tr_cutoff','pr_val_cutoff','pr_mean','pr_med',
#             'overall_mean']
# cols_reorder = key_cols+ [col for col in mean_df_hash.columns if col not in key_cols]

# mean_df_hash=mean_df_hash[cols_reorder]
# # mean_df
# # temp_df
# if analysis_mean_df_hash is None:
#     analysis_mean_df_hash=mean_df_hash 
# else:
#     analysis_mean_df_hash = pd.concat([analysis_mean_df,mean_df_hash],axis=0,ignore_index=True)
# analysis_mean_df_hash

In [ ]:
# %%time

# if not IS_PROD:

#     cols = ['te_ratio','pr_mean','pr_med','pr_1','pr_2','pr_3','pr_4','pr_5',
#            'pr_tr_cutoff','pr_val_cutoff']
#     analysis_list=[]
#     mean_list=[]
#     model_means=[]
#     mask = analysis_df['digit'].isin(digit_list)
#     mask = mask & analysis_df['zero_col_cutoff'].isin(zero_col_cutoff_list)
#     cutoffs = [(13,20),(13,50),(13,100),(14,20),
#                (14,100),(14,50),(15,20),(15,50),(15,100)]
#     # cutoffs = [(9,20),(9,50),(9,100),(10,20),(10,50),(10,100),(11,20),(11,50),(11,100),(12,20),(12,50),(13,20),(13,50),(13,100),(14,20),
#     #            (14,100),(14,50),(15,20),(15,50),(15,100)]
#     # cutoffs = [(11,20),(11,50),(11,100),(12,20),(12,50),(12,100),
#     #            (13,20),(13,50),(13,100),(14,20),(14,50),(14,100),(15,20),(15,50),]
#     # cutoffs = [(12,20),(12,50)]
#     cutoffs =[(18,3)]

# #     if CONT_ANALYSIS:
# #         analysis_mean_df=pd.read_csv(f'data/analysis/analysis_mean_df_{nonce}.csv')
# #     else:
# #         analysis_mean_df = None

#     analysis_mean_df =None    
#     mean_df=None

#     for (ratio_cutoff,count_cutoff) in cutoffs:

#         filtered = report_exceptional_teratio(analysis_df,digit_list,
#                      ratio_cutoff,count_cutoff,10000,
#                      PATTERN_99,HIGH_ANALYSIS,zero_col_cutoff_list,is_aftval_sum=True,
#                                           is_aftval_opp=True,
#                                              )
   
    
# #         filtered = analysis_df[mask1]
#         print(len(filtered))
# #         test_filt_all,filtered = save_test_ml_probs(filtered,test,
# #                                         xgb_models,
# #                                        np.mean(np.array(tr_last_cutoffs)),
# #                                          np.mean(np.array(val_cutoffs)), 
# #                                       PATTERN_99,HIGH_ANALYSIS,
# #                                       gen_probs=False,
# #                                       multi_models=True )
#         analysis_list.append(filtered)
#         records  = len(filtered)
#     #     print('Records:',records)
# #         cur_mean = filtered[cols].mean(axis=0)
#         cur_mean = pd.Series()
#         pr_count_dict = {'pr_1':'te_pr_1_count','pr_2':'te_pr_2_count','pr_3':'te_pr_3_count',
#                            'pr_4':'te_pr_4_count','pr_5':'te_pr_5_count',
#                           'pr_mean':'te_mean_count','pr_med':'te_med_count',
#                          'pr_tr_cutoff':'te_tr_count','pr_val_cutoff':'te_val_count',
#                          'te_ratio':'te_total'}

#         for pr_col,pr_count_col in pr_count_dict.items():
#             cur_mean.loc[pr_col]= (filtered[pr_col]*filtered[pr_count_col]).round(0).sum() / filtered[pr_count_col].sum()

        
        
        
#         model_mean = cur_mean.mean()
#         te_total_sum = filtered['te_total'].sum(axis=0)
        
        
#     #     model_mean = filtered[cols].mean(axis=None)
#     #     print('Overall mean:',model_mean)
#     #     print(cur_mean)

#         cur_mean.loc['te_total_sum']=te_total_sum
#         cur_mean.loc['overall_mean']=model_mean
#         cur_mean.loc['ratio_cutoff']=ratio_cutoff
#         cur_mean.loc['total_cutoff']=count_cutoff
#         cur_mean.loc['records']=records

#         if mean_df is None:
#             mean_df = pd.DataFrame({'temp':cur_mean})    
#         else:
#             mean_df=pd.concat([mean_df,cur_mean],axis=1)

#     mean_df=mean_df.transpose().reset_index(drop=True)
#     mean_df['nonce']=nonce
#     mean_df['pattern_99']=PATTERN_99
#     mean_df['target_high']=HIGH_ANALYSIS
#     key_cols = ['nonce','pattern_99','target_high','ratio_cutoff','total_cutoff', 
#                 'records','te_total_sum','te_ratio','pr_tr_cutoff','pr_val_cutoff',
#                 'pr_mean','pr_med','overall_mean']
#     cols_reorder = key_cols+ [col for col in mean_df.columns if col not in key_cols]

#     mean_df=mean_df[cols_reorder]
#     # mean_df
#     # temp_df
#     if analysis_mean_df is None:
#         analysis_mean_df=mean_df 
#     else:
#         analysis_mean_df = pd.concat([analysis_mean_df,mean_df],axis=0,ignore_index=True)
# analysis_mean_df

In [171]:
%%time

suffix='_hash'
digit_list=[1,2,3,4]
digit_list=[9,8,7,5]
# digit_list=[6]
zero_col_cutoff_list=[0,1]
only_better_probs = False
better_prob_cutoff= 11
cutoff_col = 'te_ratio'


cols = ['te_ratio','pr_mean','pr_med','pr_1','pr_2','pr_3','pr_4','pr_5',
       'pr_tr_cutoff','pr_val_cutoff']
analysis_list_hash=[]
mean_list=[]
model_means=[]
mask = analysis_df['digit'].isin(digit_list)
mask = mask & analysis_df['zero_col_cutoff'].isin(zero_col_cutoff_list)
cutoffs = [(13,15),(13,20),(13,50),(13,100),(14,15),(14,20),
           (14,100),(14,50),(15,15),(15,20),(15,50),(15,100),(18,50),(20,50),(25,50),]
# cutoffs =[(14,20)]

# if CONT_ANALYSIS:
#     analysis_mean_df=pd.read_csv(f'data/analysis/analysis_mean_df_{nonce}{suffix}.csv')
# else:
#     analysis_mean_df = None

analysis_mean_df_hash =None    
mean_df_hash=None

for (ratio_cutoff,count_cutoff) in cutoffs:

    mask1= mask & (analysis_df_hash['tr_ratio']>=ratio_cutoff) \
                & (analysis_df_hash['te_total']>=count_cutoff) \
                & (analysis_df_hash['pattern_99']==PATTERN_99) \
                & (analysis_df_hash['target_high']==HIGH_ANALYSIS) \
    
    if only_better_probs:
        mask1 = mask1 & (analysis_df_hash[cutoff_col]>=better_prob_cutoff)

    filtered = analysis_df_hash[mask1]
    print(len(filtered))
#     test_filt_all,filtered = save_test_ml_probs(filtered,test_hash,
#                                     xgb_models,
#                                    np.mean(np.array(tr_last_cutoffs)),
#                                      np.mean(np.array(val_cutoffs)), 
#                                   PATTERN_99,HIGH_ANALYSIS,
#                                   gen_probs=False,
#                                   multi_models=True )
    analysis_list_hash.append(filtered)
    records  = len(filtered)
#     print('Records:',records)
#     cur_mean = filtered[cols].mean(axis=0)
    cur_mean = pd.Series()
    pr_count_dict = {'pr_1':'te_pr_1_count','pr_2':'te_pr_2_count','pr_3':'te_pr_3_count',
                       'pr_4':'te_pr_4_count','pr_5':'te_pr_5_count',
                      'pr_mean':'te_mean_count','pr_med':'te_med_count',
                     'pr_tr_cutoff':'te_tr_count','pr_val_cutoff':'te_val_count',
                     'te_ratio':'te_total'}
    
    for pr_col,pr_count_col in pr_count_dict.items():
        cur_mean.loc[pr_col]= (filtered[pr_col]*filtered[pr_count_col]).round(0).sum() / filtered[pr_count_col].sum()
    
    model_mean = cur_mean.mean()
    te_total_sum = filtered['te_total'].sum(axis=0)
#     model_mean = filtered[cols].mean(axis=None)
#     print('Overall mean:',model_mean)
#     print(cur_mean)

    cur_mean.loc['overall_mean']=model_mean
    cur_mean.loc['ratio_cutoff']=ratio_cutoff
    cur_mean.loc['total_cutoff']=count_cutoff
    cur_mean.loc['records']=records
    cur_mean.loc['te_total_sum']=te_total_sum

    if mean_df_hash is None:
        mean_df_hash = pd.DataFrame({'temp':cur_mean})    
    else:
        mean_df_hash=pd.concat([mean_df_hash,cur_mean],axis=1)

mean_df_hash=mean_df_hash.transpose().reset_index(drop=True)
mean_df_hash['nonce']=nonce
mean_df_hash['pattern_99']=PATTERN_99
mean_df_hash['target_high']=HIGH_ANALYSIS
key_cols = ['nonce','pattern_99','target_high','ratio_cutoff','total_cutoff', 
            'records','te_total_sum','te_ratio','pr_tr_cutoff','pr_val_cutoff','pr_mean','pr_med',
            'overall_mean']
cols_reorder = key_cols+ [col for col in mean_df_hash.columns if col not in key_cols]

mean_df_hash=mean_df_hash[cols_reorder]
# mean_df
# temp_df
if analysis_mean_df_hash is None:
    analysis_mean_df_hash=mean_df_hash 
else:
    analysis_mean_df_hash = pd.concat([analysis_mean_df,mean_df_hash],axis=0,ignore_index=True)
analysis_mean_df_hash

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
CPU times: user 304 ms, sys: 15 µs, total: 304 ms
Wall time: 303 ms


,nonce,pattern_99,target_high,ratio_cutoff,total_cutoff,records,te_total_sum,te_ratio,pr_tr_cutoff,pr_val_cutoff,pr_mean,pr_med,overall_mean,pr_1,pr_2,pr_3,pr_4,pr_5
0,2649,False,False,13.0,15.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2649,False,False,13.0,20.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2649,False,False,13.0,50.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2649,False,False,13.0,100.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2649,False,False,14.0,15.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2649,False,False,14.0,20.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2649,False,False,14.0,100.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2649,False,False,14.0,50.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2649,False,False,15.0,15.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2649,False,False,15.0,20.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
analysis_list_hash[0]

In [144]:
print(len(analysis_df_hash))
len(analysis_df_hash[analysis_df_hash['te_ratio']!=0])

972


97

In [145]:
print(len(analysis_df))
len(analysis_df[analysis_df['te_ratio']!=0])

972


95

In [ ]:
%%time

if not False: #IS_PROD:

    cols = ['te_ratio','pr_mean','pr_med','pr_1','pr_2','pr_3','pr_4','pr_5',
           'pr_tr_cutoff','pr_val_cutoff']
    analysis_list=[]
    mean_list=[]
    model_means=[]
    mask = analysis_df['digit'].isin(digit_list)
    mask = mask & analysis_df['zero_col_cutoff'].isin(zero_col_cutoff_list)
#     cutoffs = [(13,20),(13,50),(13,100),(14,20),
#                (14,100),(14,50),(15,20),(15,50),(15,100)]
    # cutoffs = [(9,20),(9,50),(9,100),(10,20),(10,50),(10,100),(11,20),(11,50),(11,100),(12,20),(12,50),(13,20),(13,50),(13,100),(14,20),
    #            (14,100),(14,50),(15,20),(15,50),(15,100)]
    # cutoffs = [(11,20),(11,50),(11,100),(12,20),(12,50),(12,100),
    #            (13,20),(13,50),(13,100),(14,20),(14,50),(14,100),(15,20),(15,50),]
    # cutoffs = [(12,20),(12,50)]
#     cutoffs =[(14,20)]

    if CONT_ANALYSIS:
        analysis_mean_df=pd.read_csv(f'data/analysis/analysis_mean_df_{nonce}.csv')
    else:
        analysis_mean_df = None

    analysis_mean_df =None    
    mean_df=None

    for i,(ratio_cutoff,count_cutoff) in enumerate(cutoffs):

    #     mask1= mask & (analysis_df['cutoff']==5) \
    #                 & (analysis_df['te_total']>=count_cutoff) \
    #                 & (analysis_df['tr_ratio']>=ratio_cutoff) \
    #                 & (analysis_df['zero_col_cutoff']==3) \
        mask1= mask & (analysis_df['tr_ratio']>=ratio_cutoff) \
                    & (analysis_df['te_total']<=count_cutoff) \
                    & (analysis_df['pattern_99']==PATTERN_99) \
                    & (analysis_df['target_high']==HIGH_ANALYSIS) \
    #                  & (analysis_df['zero_col_cutoff'].isin([0,1])) \
    #                  & (analysis_df['cutoff'].isin([4,5])) \
       #                 & ((analysis_df['aft_all']>13) | (analysis_df['aft_1']>13) | (analysis_df['aft_2']>13)) \
        #                 & ((analysis_df['aft_all']<13) | (analysis_df['aft_1']<13) | (analysis_df['aft_2']<13)) \
        #                  & ((analysis_df['aft_all']+analysis_df['aft_1']+analysis_df['aft_2'])>=36) \

        if only_better_probs:
    #         mask1 = mask1 & (analysis_df.index.isin(analysis_list_hash[i].index))
            filtered=analysis_df.loc[analysis_list_hash[i].index][mask1]
        else:
            filtered = analysis_df[mask1]
            
        print(len(filtered))
        test_filt_all,filtered = save_test_ml_probs(filtered,test,
                                        xgb_models,
                                       np.mean(np.array(tr_last_cutoffs)),
                                         np.mean(np.array(val_cutoffs)), 
                                      PATTERN_99,HIGH_ANALYSIS,
                                      gen_probs=False,
                                      multi_models=True )
        analysis_list.append(filtered)
        records  = len(filtered)
    #     print('Records:',records)
#         cur_mean = filtered[cols].mean(axis=0)
        cur_mean = pd.Series()
        pr_count_dict = {'pr_1':'te_pr_1_count','pr_2':'te_pr_2_count','pr_3':'te_pr_3_count',
                           'pr_4':'te_pr_4_count','pr_5':'te_pr_5_count',
                          'pr_mean':'te_mean_count','pr_med':'te_med_count',
                         'pr_tr_cutoff':'te_tr_count','pr_val_cutoff':'te_val_count',
                         'te_ratio':'te_total'}

        for pr_col,pr_count_col in pr_count_dict.items():
            cur_mean.loc[pr_col]= (filtered[pr_col]*filtered[pr_count_col]).round(0).sum() / filtered[pr_count_col].sum()

        
        
        
        model_mean = cur_mean.mean()
        te_total_sum = filtered['te_total'].sum(axis=0)
        
        
    #     model_mean = filtered[cols].mean(axis=None)
    #     print('Overall mean:',model_mean)
    #     print(cur_mean)

        cur_mean.loc['te_total_sum']=te_total_sum
        cur_mean.loc['overall_mean']=model_mean
        cur_mean.loc['ratio_cutoff']=ratio_cutoff
        cur_mean.loc['total_cutoff']=count_cutoff
        cur_mean.loc['records']=records

        if mean_df is None:
            mean_df = pd.DataFrame({'temp':cur_mean})    
        else:
            mean_df=pd.concat([mean_df,cur_mean],axis=1)

    mean_df=mean_df.transpose().reset_index(drop=True)
    mean_df['nonce']=nonce
    mean_df['pattern_99']=PATTERN_99
    mean_df['target_high']=HIGH_ANALYSIS
    key_cols = ['nonce','pattern_99','target_high','ratio_cutoff','total_cutoff', 
                'records','te_total_sum','te_ratio','pr_tr_cutoff','pr_val_cutoff',
                'pr_mean','pr_med','overall_mean']
    cols_reorder = key_cols+ [col for col in mean_df.columns if col not in key_cols]

    mean_df=mean_df[cols_reorder]
    # mean_df
    # temp_df
    if analysis_mean_df is None:
        analysis_mean_df=mean_df 
    else:
        analysis_mean_df = pd.concat([analysis_mean_df,mean_df],axis=0,ignore_index=True)
    analysis_mean_df

In [ ]:
analysis_mean_df

In [ ]:
analysis_list[0]

In [ ]:
print(test_hash['probs'].describe())
cutoff = test_hash['probs'].quantile(q=0.97)
print(cutoff)
mask = test_hash['probs']>cutoff
total = len(test_hash[mask])
mask2 = test_hash['roll_actual']>=8944
success = len(test_hash[mask & mask2])
ratio = success / total
print(ratio,success,total)

In [ ]:
print(test['probs'].describe())
cutoff = test['probs'].quantile(q=0.97)
print(cutoff)
mask = test['probs']>cutoff
total = len(test[mask])
mask2 = test['roll_actual']>=8944
success = len(test[mask & mask2])
ratio = success / total
print(ratio,success,total)

In [ ]:
if not IS_PROD:
    analysis_mean_df.to_csv(f'data/analysis/analysis_mean_df_{nonce}.csv',index=False)

##### Production Code

In [ ]:
# digitlist=[9,8,7,5]
# zerocolcutoffs=list(range(0,6))

# digitlist=[1,2,3,4]
digitlist=[9,8,7,5]
zerocolcutoffs=[4,5]
ratio_cutoff=15
count_cutoff_min=20
count_cutoff_max=10000
analysis_filt = report_exceptional_teratio(analysis_df,digitlist,
                     ratio_cutoff,count_cutoff_min,count_cutoff_max,
                     PATTERN_99,HIGH_ANALYSIS,zerocolcutoffs)
anal_size=len(analysis_filt)
test_size=analysis_filt['te_total'].sum()
print(f'Analysis Selection Size: {anal_size} Total Test Size:{test_size}')

In [ ]:
analysis_filt

In [ ]:
#Fetch test selection based on te ratio 

def get_test_from_analysis(analysis_df,test,
              pattern,bHighCheck,
              is_prod=False,):
    
    start = time.time()
    features_test = [col for col in features if col not in ['target','probs']]
    if  is_prod:
        analysis_df_filt= analysis_df
    else:
        mask = (analysis_df['pattern_99']==pattern) & (analysis_df['target_high']==bHighCheck) 
        analysis_df_filt = analysis_df[mask]
    
    test_filt_all = None
    for row in analysis_df_filt.itertuples():
        digit = row.digit
        cutoff = row.cutoff
        after_val = row.afterval
        bHighCheck = row.target_high
        zero_col_cutoff = row.zero_col_cutoff

        test_filt_df = validate_pattern(test,bHighCheck,zero_col_cutoff,
                                        digit,cutoff,after_val)
        filt_size = len(test_filt_df)
        print(f'{filt_size=}')
        mask=create_target_mask(test_filt_df)
        success = len(test_filt_df[mask])
#         success=len(test_filt_df[test_filt_df['roll_actual']>HIGH_TARGET])
        print(f'{success=}')        
        
        if filt_size==0:
            print(f'No Record for zero_col_cutoff:{zero_col_cutoff} digit:{digit} cutoff:{cutoff} after_val:{after_val}')
            continue

        if is_prod:
            if test_filt_all is None:
                test_filt_all=test_filt_df
            else:
                test_filt_all=pd.concat([test_filt_all,test_filt_df],axis=0)
            
            
    end = time.time()
    print(f'Function time:{end-start}')   

    return test_filt_all

In [ ]:
# temp_sel = save_test_stats(analysis_filt, test_manual,PATTERN_99,HIGH_ANALYSIS,is_prod=False)
# print(len(temp_sel))

In [ ]:
# %%time
# test_sel = get_test_from_analysis(analysis_filt,test,PATTERN_99,HIGH_ANALYSIS,
#                                   is_prod=True)
# print(len(test_sel))

In [ ]:
%%time
test_sel,ret_analysis_df = save_test_ml_probs(analysis_filt,
                                              test,
                                    xgb_models,
                                   np.mean(np.array(tr_last_cutoffs)),
                                  np.mean(np.array(val_cutoffs)), 
                                  PATTERN_99,HIGH_ANALYSIS,
                                  is_prod=True,
                                 gen_probs=False,
                                  prob_cutoff=0.3,
                                  multi_models=True )
print(f'Selected Test Size:{len(test_sel)}')

In [ ]:
if not IS_PROD:
    success=len(test_sel[test_sel['roll_actual']>=9000])
    print(success)
    print(success / len(test_sel))

In [ ]:
test_sel.head()

In [165]:
# def get_random_client_seed(test):
#     size = len(test)
#     test = test.reset_index(drop=True).sample(n=size).reset_index(drop=True)
#     rand_pos = random.randint(0,size-1)
#     print(f'rand_pos:{rand_pos}')
#     return test.iloc[rand_pos]['client_seed'],test

def get_random_client_seed(test):
    size = len(test)
    final_test= test.sample(n=size)
    rand_pos=random.randint(0,size-1)
    sample=final_test.reset_index(drop=True).loc[rand_pos]
#     rand_pos = sample.index[0]
    print(f'rand_pos:{rand_pos}')
    return sample['client_seed'],final_test

In [166]:
client_seed,final_test = get_random_client_seed(test_sel)
print('Client Seed Selected')
print(client_seed)
# final_test.to_csv(f'data/analysis/final_test_{nonce}.csv')
# final_test.head()

rand_pos:0
Client Seed Selected
woxpwoxpwoxpwoxpwoxp94b3e71b54a52abdd37b7516a7fb9568747a0baa91c45af22d9b6d77810dfab5


In [167]:
final_test

,roll_actual,seed,hash,client_seed,roll_0,roll_1,roll_2,roll_3,roll_4,roll_5,roll_6,roll_7,roll_8,roll_9,roll_10,roll_11,roll_12,roll_13,roll_14,roll_15,roll_16,roll_17,roll_18,roll_19,roll_20,roll_21,roll_22,roll_23,roll_24,roll_25,roll_26,roll_27,roll_28,roll_29,roll_30,roll_31,roll_32,roll_33,roll_34,roll_35,roll_36,roll_37,roll_38,roll_39,roll_40,roll_41,roll_42,roll_43,roll_44,roll_45,roll_46,roll_47,roll_48,roll_49,roll_50,roll_51,roll_52,roll_53,roll_54,roll_55,roll_56,roll_57,roll_58,roll_59,roll_60,count_lt_1000,count_gt_9000,count_gt_8000,count_gt_7000,count_gt_6000,count_gt_5000,count_gt_4000,count_gt_3000,count_gt_2000,count_gt_1000,count_hi,count_lo,count_lo_hi,count_gt_9000_k,count_lt_1000_k,total_dig_9,total_dig_0,roll_mean,roll_std,count_lt_1000_25_50,count_gt_9000_25_50,count_gt_8000_25_50,count_gt_7000_25_50,count_gt_6000_25_50,count_gt_5000_25_50,count_gt_4000_25_50,count_gt_3000_25_50,count_gt_2000_25_50,count_gt_1000_25_50,count_hi_25_50,count_lo_25_50,count_lo_hi_25_50,count_gt_9000_k_25_50,count_lt_1000_k_25_50,total_dig_9_25_50,total_dig_0_25_50,roll_mean_25_50,roll_std_25_50,count_lt_1000_50_60,count_gt_9000_50_60,count_gt_8000_50_60,count_gt_7000_50_60,count_gt_6000_50_60,count_gt_5000_50_60,count_gt_4000_50_60,count_gt_3000_50_60,count_gt_2000_50_60,count_gt_1000_50_60,count_hi_50_60,count_lo_50_60,count_lo_hi_50_60,count_gt_9000_k_50_60,count_lt_1000_k_50_60,total_dig_9_50_60,total_dig_0_50_60,roll_mean_50_60,roll_std_50_60,count_lt_1000_1_60,count_gt_9000_1_60,count_gt_8000_1_60,count_gt_7000_1_60,count_gt_6000_1_60,count_gt_5000_1_60,count_gt_4000_1_60,count_gt_3000_1_60,count_gt_2000_1_60,count_gt_1000_1_60,count_hi_1_60,count_lo_1_60,count_lo_hi_1_60,count_gt_9000_k_1_60,count_lt_1000_k_1_60,total_dig_9_1_60,total_dig_0_1_60,roll_mean_1_60,roll_std_1_60,roll_0_nonce,roll_1_nonce,roll_2_nonce,roll_3_nonce,roll_4_nonce,roll_5_nonce,roll_6_nonce,roll_7_nonce,roll_8_nonce,roll_9_nonce,roll_10_nonce,roll_11_nonce,roll_12_nonce,roll_13_nonce,roll_14_nonce,roll_15_nonce,roll_16_nonce,roll_17_nonce,roll_18_nonce,roll_19_nonce,roll_20_nonce,roll_21_nonce,roll_22_nonce,roll_23_nonce,roll_24_nonce,roll_25_nonce,roll_26_nonce,roll_27_nonce,roll_28_nonce,roll_29_nonce,roll_30_nonce,roll_31_nonce,roll_32_nonce,roll_33_nonce,roll_34_nonce,roll_35_nonce,roll_36_nonce,roll_37_nonce,roll_38_nonce,roll_39_nonce,roll_40_nonce,roll_41_nonce,roll_42_nonce,roll_43_nonce,roll_44_nonce,roll_45_nonce,roll_46_nonce,roll_47_nonce,roll_48_nonce,roll_49_nonce,roll_50_nonce,roll_51_nonce,roll_52_nonce,roll_53_nonce,roll_54_nonce,roll_55_nonce,roll_56_nonce,roll_57_nonce,roll_58_nonce,roll_59_nonce,roll_60_nonce,count_lt_1000_nonce,count_gt_9000_nonce,count_gt_8000_nonce,count_gt_7000_nonce,count_gt_6000_nonce,count_gt_5000_nonce,count_gt_4000_nonce,count_gt_3000_nonce,count_gt_2000_nonce,count_gt_1000_nonce,count_hi_nonce,count_lo_nonce,count_lo_hi_nonce,count_gt_9000_k_nonce,count_lt_1000_k_nonce,total_dig_9_nonce,total_dig_0_nonce,roll_mean_nonce,roll_std_nonce,count_lt_1000_25_50_nonce,count_gt_9000_25_50_nonce,count_gt_8000_25_50_nonce,count_gt_7000_25_50_nonce,count_gt_6000_25_50_nonce,count_gt_5000_25_50_nonce,count_gt_4000_25_50_nonce,count_gt_3000_25_50_nonce,count_gt_2000_25_50_nonce,count_gt_1000_25_50_nonce,count_hi_25_50_nonce,count_lo_25_50_nonce,count_lo_hi_25_50_nonce,count_gt_9000_k_25_50_nonce,count_lt_1000_k_25_50_nonce,total_dig_9_25_50_nonce,total_dig_0_25_50_nonce,roll_mean_25_50_nonce,roll_std_25_50_nonce,count_lt_1000_50_60_nonce,count_gt_9000_50_60_nonce,count_gt_8000_50_60_nonce,count_gt_7000_50_60_nonce,count_gt_6000_50_60_nonce,count_gt_5000_50_60_nonce,count_gt_4000_50_60_nonce,count_gt_3000_50_60_nonce,count_gt_2000_50_60_nonce,count_gt_1000_50_60_nonce,...,roll_actual_14,roll_actual_15,roll_actual_16,roll_actual_17,roll_actual_18,roll_actual_19,roll_actual_20,roll_actual_21,roll_actual_22,roll_actual_23,roll_actual_24,roll_actual_25,roll_actual_26,roll_actual_27,roll_actual_28,roll_actual_29,roll

In [147]:
final_test.reset_index()[final_test.reset_index()['index']==6826]

,index,roll_actual,seed,hash,client_seed,roll_0,roll_1,roll_2,roll_3,roll_4,roll_5,roll_6,roll_7,roll_8,roll_9,roll_10,roll_11,roll_12,roll_13,roll_14,roll_15,roll_16,roll_17,roll_18,roll_19,roll_20,roll_21,roll_22,roll_23,roll_24,roll_25,roll_26,roll_27,roll_28,roll_29,roll_30,roll_31,roll_32,roll_33,roll_34,roll_35,roll_36,roll_37,roll_38,roll_39,roll_40,roll_41,roll_42,roll_43,roll_44,roll_45,roll_46,roll_47,roll_48,roll_49,roll_50,roll_51,roll_52,roll_53,roll_54,roll_55,roll_56,roll_57,roll_58,roll_59,roll_60,count_lt_1000,count_gt_9000,count_gt_8000,count_gt_7000,count_gt_6000,count_gt_5000,count_gt_4000,count_gt_3000,count_gt_2000,count_gt_1000,count_hi,count_lo,count_lo_hi,count_gt_9000_k,count_lt_1000_k,total_dig_9,total_dig_0,roll_mean,roll_std,count_lt_1000_25_50,count_gt_9000_25_50,count_gt_8000_25_50,count_gt_7000_25_50,count_gt_6000_25_50,count_gt_5000_25_50,count_gt_4000_25_50,count_gt_3000_25_50,count_gt_2000_25_50,count_gt_1000_25_50,count_hi_25_50,count_lo_25_50,count_lo_hi_25_50,count_gt_9000_k_25_50,count_lt_1000_k_25_50,total_dig_9_25_50,total_dig_0_25_50,roll_mean_25_50,roll_std_25_50,count_lt_1000_50_60,count_gt_9000_50_60,count_gt_8000_50_60,count_gt_7000_50_60,count_gt_6000_50_60,count_gt_5000_50_60,count_gt_4000_50_60,count_gt_3000_50_60,count_gt_2000_50_60,count_gt_1000_50_60,count_hi_50_60,count_lo_50_60,count_lo_hi_50_60,count_gt_9000_k_50_60,count_lt_1000_k_50_60,total_dig_9_50_60,total_dig_0_50_60,roll_mean_50_60,roll_std_50_60,count_lt_1000_1_60,count_gt_9000_1_60,count_gt_8000_1_60,count_gt_7000_1_60,count_gt_6000_1_60,count_gt_5000_1_60,count_gt_4000_1_60,count_gt_3000_1_60,count_gt_2000_1_60,count_gt_1000_1_60,count_hi_1_60,count_lo_1_60,count_lo_hi_1_60,count_gt_9000_k_1_60,count_lt_1000_k_1_60,total_dig_9_1_60,total_dig_0_1_60,roll_mean_1_60,roll_std_1_60,roll_0_nonce,roll_1_nonce,roll_2_nonce,roll_3_nonce,roll_4_nonce,roll_5_nonce,roll_6_nonce,roll_7_nonce,roll_8_nonce,roll_9_nonce,roll_10_nonce,roll_11_nonce,roll_12_nonce,roll_13_nonce,roll_14_nonce,roll_15_nonce,roll_16_nonce,roll_17_nonce,roll_18_nonce,roll_19_nonce,roll_20_nonce,roll_21_nonce,roll_22_nonce,roll_23_nonce,roll_24_nonce,roll_25_nonce,roll_26_nonce,roll_27_nonce,roll_28_nonce,roll_29_nonce,roll_30_nonce,roll_31_nonce,roll_32_nonce,roll_33_nonce,roll_34_nonce,roll_35_nonce,roll_36_nonce,roll_37_nonce,roll_38_nonce,roll_39_nonce,roll_40_nonce,roll_41_nonce,roll_42_nonce,roll_43_nonce,roll_44_nonce,roll_45_nonce,roll_46_nonce,roll_47_nonce,roll_48_nonce,roll_49_nonce,roll_50_nonce,roll_51_nonce,roll_52_nonce,roll_53_nonce,roll_54_nonce,roll_55_nonce,roll_56_nonce,roll_57_nonce,roll_58_nonce,roll_59_nonce,roll_60_nonce,count_lt_1000_nonce,count_gt_9000_nonce,count_gt_8000_nonce,count_gt_7000_nonce,count_gt_6000_nonce,count_gt_5000_nonce,count_gt_4000_nonce,count_gt_3000_nonce,count_gt_2000_nonce,count_gt_1000_nonce,count_hi_nonce,count_lo_nonce,count_lo_hi_nonce,count_gt_9000_k_nonce,count_lt_1000_k_nonce,total_dig_9_nonce,total_dig_0_nonce,roll_mean_nonce,roll_std_nonce,count_lt_1000_25_50_nonce,count_gt_9000_25_50_nonce,count_gt_8000_25_50_nonce,count_gt_7000_25_50_nonce,count_gt_6000_25_50_nonce,count_gt_5000_25_50_nonce,count_gt_4000_25_50_nonce,count_gt_3000_25_50_nonce,count_gt_2000_25_50_nonce,count_gt_1000_25_50_nonce,count_hi_25_50_nonce,count_lo_25_50_nonce,count_lo_hi_25_50_nonce,count_gt_9000_k_25_50_nonce,count_lt_1000_k_25_50_nonce,total_dig_9_25_50_nonce,total_dig_0_25_50_nonce,roll_mean_25_50_nonce,roll_std_25_50_nonce,count_lt_1000_50_60_nonce,count_gt_9000_50_60_nonce,count_gt_8000_50_60_nonce,count_gt_7000_50_60_nonce,count_gt_6000_50_60_nonce,count_gt_5000_50_60_nonce,count_gt_4000_50_60_nonce,count_gt_3000_50_60_nonce,count_gt_2000_50_60_nonce,...,roll_actual_13,roll_actual_14,roll_actual_15,roll_actual_16,roll_actual_17,roll_actual_18,roll_actual_19,roll_actual_20,roll_actual_21,roll_actual_22,roll_actual_23,roll_actual_24,roll_actual_25,roll_actual_26,roll_actual_27,roll_actual_28,roll_actual_29,roll_actu

After Actual SEED is known

In [168]:
actual_seed="f8abc42960d2c5c172b5c1fbe605f4eed29dd808f974640ebca01e1b00383b0b"
# actual_seed=cur_hash_list[4]
# print(f'{actual_seed=}')

In [169]:
cur_hash_list[25]

'809d6a154ad7c7dcb33d1389bad8f809197e7b78ecb9b1c859fae02150615ecf'

In [170]:
# Vectorize the function
vectorized_calculate_roll = np.vectorize(calculate_roll)

# Compute the roll values for the input arrays
roll_array = vectorized_calculate_roll(actual_seed,
                                       final_test['client_seed'],
                                       nonce)
final_test['roll_actual_dummy']=final_test['roll_actual'].copy()
final_test['roll_actual']=roll_array
# final_test.to_csv(f'data/analysis/final_test_{nonce}.csv')

In [171]:
final_test

,roll_actual,seed,hash,client_seed,roll_0,roll_1,roll_2,roll_3,roll_4,roll_5,roll_6,roll_7,roll_8,roll_9,roll_10,roll_11,roll_12,roll_13,roll_14,roll_15,roll_16,roll_17,roll_18,roll_19,roll_20,roll_21,roll_22,roll_23,roll_24,roll_25,roll_26,roll_27,roll_28,roll_29,roll_30,roll_31,roll_32,roll_33,roll_34,roll_35,roll_36,roll_37,roll_38,roll_39,roll_40,roll_41,roll_42,roll_43,roll_44,roll_45,roll_46,roll_47,roll_48,roll_49,roll_50,roll_51,roll_52,roll_53,roll_54,roll_55,roll_56,roll_57,roll_58,roll_59,roll_60,count_lt_1000,count_gt_9000,count_gt_8000,count_gt_7000,count_gt_6000,count_gt_5000,count_gt_4000,count_gt_3000,count_gt_2000,count_gt_1000,count_hi,count_lo,count_lo_hi,count_gt_9000_k,count_lt_1000_k,total_dig_9,total_dig_0,roll_mean,roll_std,count_lt_1000_25_50,count_gt_9000_25_50,count_gt_8000_25_50,count_gt_7000_25_50,count_gt_6000_25_50,count_gt_5000_25_50,count_gt_4000_25_50,count_gt_3000_25_50,count_gt_2000_25_50,count_gt_1000_25_50,count_hi_25_50,count_lo_25_50,count_lo_hi_25_50,count_gt_9000_k_25_50,count_lt_1000_k_25_50,total_dig_9_25_50,total_dig_0_25_50,roll_mean_25_50,roll_std_25_50,count_lt_1000_50_60,count_gt_9000_50_60,count_gt_8000_50_60,count_gt_7000_50_60,count_gt_6000_50_60,count_gt_5000_50_60,count_gt_4000_50_60,count_gt_3000_50_60,count_gt_2000_50_60,count_gt_1000_50_60,count_hi_50_60,count_lo_50_60,count_lo_hi_50_60,count_gt_9000_k_50_60,count_lt_1000_k_50_60,total_dig_9_50_60,total_dig_0_50_60,roll_mean_50_60,roll_std_50_60,count_lt_1000_1_60,count_gt_9000_1_60,count_gt_8000_1_60,count_gt_7000_1_60,count_gt_6000_1_60,count_gt_5000_1_60,count_gt_4000_1_60,count_gt_3000_1_60,count_gt_2000_1_60,count_gt_1000_1_60,count_hi_1_60,count_lo_1_60,count_lo_hi_1_60,count_gt_9000_k_1_60,count_lt_1000_k_1_60,total_dig_9_1_60,total_dig_0_1_60,roll_mean_1_60,roll_std_1_60,roll_0_nonce,roll_1_nonce,roll_2_nonce,roll_3_nonce,roll_4_nonce,roll_5_nonce,roll_6_nonce,roll_7_nonce,roll_8_nonce,roll_9_nonce,roll_10_nonce,roll_11_nonce,roll_12_nonce,roll_13_nonce,roll_14_nonce,roll_15_nonce,roll_16_nonce,roll_17_nonce,roll_18_nonce,roll_19_nonce,roll_20_nonce,roll_21_nonce,roll_22_nonce,roll_23_nonce,roll_24_nonce,roll_25_nonce,roll_26_nonce,roll_27_nonce,roll_28_nonce,roll_29_nonce,roll_30_nonce,roll_31_nonce,roll_32_nonce,roll_33_nonce,roll_34_nonce,roll_35_nonce,roll_36_nonce,roll_37_nonce,roll_38_nonce,roll_39_nonce,roll_40_nonce,roll_41_nonce,roll_42_nonce,roll_43_nonce,roll_44_nonce,roll_45_nonce,roll_46_nonce,roll_47_nonce,roll_48_nonce,roll_49_nonce,roll_50_nonce,roll_51_nonce,roll_52_nonce,roll_53_nonce,roll_54_nonce,roll_55_nonce,roll_56_nonce,roll_57_nonce,roll_58_nonce,roll_59_nonce,roll_60_nonce,count_lt_1000_nonce,count_gt_9000_nonce,count_gt_8000_nonce,count_gt_7000_nonce,count_gt_6000_nonce,count_gt_5000_nonce,count_gt_4000_nonce,count_gt_3000_nonce,count_gt_2000_nonce,count_gt_1000_nonce,count_hi_nonce,count_lo_nonce,count_lo_hi_nonce,count_gt_9000_k_nonce,count_lt_1000_k_nonce,total_dig_9_nonce,total_dig_0_nonce,roll_mean_nonce,roll_std_nonce,count_lt_1000_25_50_nonce,count_gt_9000_25_50_nonce,count_gt_8000_25_50_nonce,count_gt_7000_25_50_nonce,count_gt_6000_25_50_nonce,count_gt_5000_25_50_nonce,count_gt_4000_25_50_nonce,count_gt_3000_25_50_nonce,count_gt_2000_25_50_nonce,count_gt_1000_25_50_nonce,count_hi_25_50_nonce,count_lo_25_50_nonce,count_lo_hi_25_50_nonce,count_gt_9000_k_25_50_nonce,count_lt_1000_k_25_50_nonce,total_dig_9_25_50_nonce,total_dig_0_25_50_nonce,roll_mean_25_50_nonce,roll_std_25_50_nonce,count_lt_1000_50_60_nonce,count_gt_9000_50_60_nonce,count_gt_8000_50_60_nonce,count_gt_7000_50_60_nonce,count_gt_6000_50_60_nonce,count_gt_5000_50_60_nonce,count_gt_4000_50_60_nonce,count_gt_3000_50_60_nonce,count_gt_2000_50_60_nonce,count_gt_1000_50_60_nonce,...,roll_actual_15,roll_actual_16,roll_actual_17,roll_actual_18,roll_actual_19,roll_actual_20,roll_actual_21,roll_actual_22,roll_actual_23,roll_actual_24,roll_actual_25,roll_actual_26,roll_actual_27,roll_actual_28,roll_actual_29,roll_actual_30,roll

In [170]:
final_test.to_csv(f'data/analysis/final_test_{nonce}_{file_pattern_str}_pattern.csv')

In [162]:
LOW_TARGET

1000

In [159]:
mask = create_target_mask(final_test)
success_pos = final_test[mask].index

if HIGH_ANALYSIS:
    success_pos = final_test[final_test['roll_actual']>=HIGH_TARGET].index
else:
    success_pos = final_test[final_test['roll_actual']<LOW_TARGET].index
print(len(success_pos))
print(f'success_pos:{success_pos}')

# target = set([17, 19, 34, 35, 44, 47])
# matched  = set(list(success_pos)).intersection(target)
# print(matched)

# print(len(matched)/len(success_pos))

1
success_pos:Int64Index([6865], dtype='int64')


In [ ]:
test_sel.head()

In [ ]:
final_test

In [ ]:
# Vectorize the function
vectorized_calculate_roll = np.vectorize(calculate_roll)

# Compute the roll values for the input arrays
roll_array = vectorized_calculate_roll(actual_seed,
                                       results_df_test['client_seed'],
                                       nonce)

results_df_test['roll_actual']=roll_array
test_manual = generate_test_features(results_df_test)
test = generate_features_full(results_df_test,False,feature_chain_length)


analysis_df,test = gen_analysis(nonce,train_manual,test_manual,test,xgb_models,tr_last_cutoffs,val_cutoffs,
                multi_models=True,file_suffix="")
analysis_df.head()

In [ ]:
# ratio_cutoff=18
# count_cutoff_min=3
# # test_sel_options= [([1],[0,1],10000),
# #                     ([2],[0,1],10000),
# #                    ([3],[0,1],10000),
# #                    ([4],[0,1],10000),
# #                     ([5],[0,1],10000),
# # #                     ([6],[0,1],10000),
# #                    ([7],[0,1],10000),
# #                    ([8],[0,1],10000),
# #                    ([9],[0,1],10000),
# #                     ([1],[4,5],10000),
# #                     ([2],[4,5],10000),
# #                    ([3],[4,5],10000),
# #                    ([4],[4,5],10000),
# #                     ([5],[4,5],10000),
# # #                     ([6],[4,5],10000),
# #                    ([7],[4,5],10000),
# #                    ([8],[4,5],10000),
# #                    ([9],[4,5],10000),
# #                   ]

# test_sel_options= [([1,2,3,4],[0,1],10000),
#                     ([9,8,7,5],[0,1],10000),
#                    ([6],[0,1],10000),
#                     ([1,2,3,4],[4,5],10000),
#                     ([9,8,7,5],[4,5],10000),
#                    ([6],[4,5],10000),
#                   ]
# test_sel_options= [([1],[0],10000),]

#                    #                     ([6],[0,1],10000),
# #                     ([9,8,7,5],[0,1],10000),
# #                     ([1,2,3,4],[0,1],10000),

# # test_sel_options= [([6],[4,5],10000)]

# # test_sel_options= [([1,2,3,4],[0,1],10),
# #                     ([9,8,7,5,6],[0,1],10),
# #                     ([1,2,3,4],[4,5],10),
# #                     ([1,2,3,4],[4,5],15),
# #                     ([1,2,3,4],[4,5],20),
# #                     ([1,2,3,4],[4,5],25),
# #                     ([6],[0,1],10000),
# #                     ([9,8,7,5],[0,1],10000),
# #                     ([1,2,3,4],[0,1],10000),
# #                   ]


# # test_sel_options= [([9,8,7,6,5,1,2,3,4],[0,1,4,5],10000)]
# # test_sel_options= [([9,8,7,5],[4,5],10000)]
# # test_sel_options= [([6],[4,5],10000)]

# # test_sel_options= [([1,2,3,4],[0,1],10),
# #                     ([9,8,7,5,6],[0,1],10),
# #                     ([1,2,3,4],[4,5],10),
# #                     ([1,2,3,4],[4,5],15),
# #                     ([1,2,3,4],[4,5],20),
# #                     ([1,2,3,4],[4,5],25),
# #                     ([6],[0,1],10000),
# #                     ([9,8,7,5],[0,1],10000),
# #                     ([1,2,3,4],[0,1],10000),
# #                   ]

# anal_sizes =[]
# analysis_filts=[]
# test_sizes=[]
# filt_options=[]
# for option in test_sel_options:
#     digitlist=option[0]
#     zerocolcutoffs=option[1]
#     count_cutoff_max=option[2]
#     analysis_filt = report_exceptional_teratio(analysis_df,digitlist,
#                          ratio_cutoff,count_cutoff_min,count_cutoff_max,
#                          PATTERN_99,HIGH_ANALYSIS,zerocolcutoffs)
#     anal_size=len(analysis_filt)
#     if anal_size!=0:
#         test_size=analysis_filt['te_total'].sum()
#         print(f'option: {option} Analysis Selection Size: {anal_size} Total Test Size:{test_size}')
#         break
# #         anal_sizes.append(anal_size)
# #         analysis_filts.append(analysis_filt)
# #         filt_options.append(option)
# #         test_sizes.append(test_size)
#     else:
#         print(f'No records for option:{option}')

# # sel_option = np.argmin(np.array(test_sizes))
# # print(f'selected option: {filt_options[sel_option]} \
# #       Analysis Selection Size: {anal_sizes[sel_option]} Total Test Size:{test_sizes[sel_option]}')

# # analysis_filt = analysis_filts[sel_option]

#### END

Finalize Training

In [ ]:
tr_index

In [ ]:
train.index

In [ ]:
# tr_index = train[30 * train_client_size:(30 + 5) * train_client_size].index
tr_index = train[train_client_size:].index
train_ignored = train[0:30*train_client_size]
# tr_index = list(tr_index) + list(train_ignored[train_ignored['target']==1].index)
print(len(tr_index))
xgb_model_final = xgb.XGBClassifier(**params)

X_tr,y_tr = X.iloc[tr_index],y.iloc[tr_index]
oversample = RandomOverSampler(sampling_strategy='minority',random_state=123)
# fit and apply the transform
X_tr, y_tr = oversample.fit_resample(X_tr, y_tr)
print(pd.Series(y_tr).value_counts())
xgb_model_final.fit(X_tr,y_tr)

In [ ]:
len(X_tr)

In [ ]:
test_probs = xgb_model_final.predict_proba(test)[:,1]

test_labels = convert_probtolabels(test_probs)
subm=pd.DataFrame()
subm['client_seed']= results_df_test['client_seed']
subm['preds'] = test_labels
subm['prob'] = test_probs
score = accuracy_score(y_test,test_labels)
print(f'test score:{score}')

In [ ]:
mask = (subm['prob']<0.52) & (subm['preds']==1)
print(len(subm[mask]))
score = accuracy_score(y_test[mask],test_labels[mask])
print(f'test score:{score}')